## Import Necessary Libraries

In [ ]:
# 1.0

!pip install fredapi
import yfinance as yf
from fredapi import Fred
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Fetch the Sector ETF Data

In [ ]:
# 2.0

tickers = ["XLK", "XLC", "XLV", "XLF", "XLI", "XLB", "XLU", "XLE", "XLY", "XLP", "XLRE", "SPY"]

# Mapping from tickers to desired sector names
ticker_to_sector = {
    "XLK": "Technology",
    "XLC": "Communications",
    "XLV": "Healthcare",
    "XLF": "Financial",
    "XLI": "Industrial",
    "XLB": "Material",
    "XLU": "Utilities",
    "XLE": "Energy",
    "XLY": "CDiscretionary",
    "XLP": "CStaples",
    "XLRE": "RealEst",
    "SPY": "S&P 500"
}

# Set the date range
start_date = "2015-01-01"
end_date = "2025-12-30"

# Initialize an empty dictionary to store ETF data
sector_data = {}

# Download historical data for each ETF, explicitly setting auto_adjust=True.
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date, interval="1wk", auto_adjust=True, progress=False)
    if not data.empty:
        sector_data[ticker] = data["Close"]

# If valid data exists, combine it into a DataFrame and flatten the headers.
if sector_data:
    # Concatenate the Series objects; using keys creates a MultiIndex header.
    df_sector = pd.concat(sector_data.values(), axis=1, keys=sector_data.keys())

    # Flatten the MultiIndex columns by extracting the first level (tickers) and mapping to sector names.
    df_sector.columns = [ticker_to_sector.get(ticker, ticker) for ticker in df_sector.columns.get_level_values(0)]

    # Save the DataFrame to CSV
    df_sector.to_csv("Sector_ETF_Data.csv")
    print("Sector ETF data saved successfully!")
else:
    print("No valid data retrieved for any tickers.")

Sector ETF data saved successfully!


In [ ]:
# 3.0

# Filter signals for January 2026
jan_2026_signals = signals_monthly.loc['2026-01-01':'2026-01-31']

buy_sectors = jan_2026_signals[jan_2026_signals == 'Buy'].index.tolist()
sell_sectors = jan_2026_signals[jan_2026_signals == 'Sell'].index.tolist()
hold_sectors = jan_2026_signals[jan_2026_signals == 'Hold'].index.tolist()

# Create a DataFrame for the recommendations
recommendations_data = {
    'Signal Type': ['Buy', 'Sell', 'Hold'],
    'Recommended Sectors (Jan 2026)': [
        ', '.join(buy_sectors) if buy_sectors else 'None',
        ', '.join(sell_sectors) if sell_sectors else 'None',
        ', '.join(hold_sectors) if hold_sectors else 'None'
    ]
}
recommendations_df = pd.DataFrame(recommendations_data)

print("Sector Recommendations for January 2026:")
display(recommendations_df)

print("\nNote: The current models in this notebook are designed to predict macroeconomic indicators (GDP, Inflation, Unemployment Rate, Interest Rate) and economic phases for sector rotation strategy. They do not directly output a predicted S&P 500 percentage return for future months.")

NameError: name 'signals_monthly' is not defined

In [ ]:
# 4.0

import matplotlib.pyplot as plt
import seaborn as sns

# Filter signals for January 2026
jan_2026_signals = signals_monthly.loc['2026-01-01':'2026-01-31']

# Count the occurrences of 'Buy', 'Sell', 'Hold' for each sector
signal_counts = pd.DataFrame(index=jan_2026_signals.columns)
for signal_type in ['Buy', 'Sell', 'Hold']:
    signal_counts[signal_type] = (jan_2026_signals == signal_type).sum()

# Transpose for plotting
signal_counts_transposed = signal_counts.T

# Plotting
plt.figure(figsize=(14, 7))
signal_counts_transposed.plot(kind='bar', stacked=True, figsize=(14, 7), cmap='viridis')
plt.title('Buy, Sell, Hold Signals for Each Sector in January 2026')
plt.xlabel('Signal Type')
plt.ylabel('Number of Occurrences')
plt.xticks(rotation=45, ha='right')
plt.legend(title='Sector', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
# 5.0

# Load sector data (if not already loaded or to ensure latest version)
df_sector = pd.read_csv("Sector_ETF_Data.csv", index_col="Date", parse_dates=True)

In [ ]:
# 6.0

# Correct for missing values using ffill() and then bfill()
df_sector = df_sector.ffill()
df_sector = df_sector.bfill()

print("Missing values after correction:")
display(df_sector.isnull().sum())

print("DataFrame head after missing value correction:")
display(df_sector.head())

In [ ]:
# 7.0

pd.set_option('display.max_rows', 10)
display(df_sector.head(10))

## Fetch the Economic Data

In [ ]:
# 8.0

# FRED API key
fred = Fred(api_key='e8fb9c414f6ab8f1a06a5472466a31dc')

# Define weekly date range
weekly_index = pd.date_range(start='2010-01-01', end='2025-12-30', freq='W-FRI').strftime('%Y-%m-%d')

# Define FRED series IDs
series_dict = {
    'GDP': 'GDP',
    'Inflation (CPI)': 'CPIAUCSL',
    'Unemployment Rate': 'UNRATE',
    'Interest Rate (Fed Funds)': 'FEDFUNDS'
}

# Create a DataFrame with weekly index
macro_data = pd.DataFrame(index=weekly_index)

# Set the index name explicitly
macro_data.index.name = "Date"

# Fetch and interpolate data
for label, series_id in series_dict.items():
    data = fred.get_series(series_id)
    data = data.to_frame(name=label)
    data.index = pd.to_datetime(data.index)

    # Resample to daily if not daily, then reindex to weekly and interpolate
    if label == 'Interest Rate (Fed Funds)':
        daily = data.resample('D').ffill()
    else:
        daily = data.resample('D').interpolate()

    # Align with weekly index
    weekly = daily.reindex(pd.to_datetime(weekly_index)).interpolate()
    macro_data[label] = weekly[label]

# Show and save
macro_data.to_csv("Macroeconomic_Data.csv", index=True)

print("Macroeconomic Data saved successfully!")

Macroeconomic Data saved successfully!


In [ ]:
# 9.0

pd.set_option('display.max_rows', 10)
display(macro_data.head(10))

## Statistical Analysis

In [ ]:
# 10.0

# Load sector and macroeconomic data
df_sector = pd.read_csv("Sector_ETF_Data.csv", index_col="Date", parse_dates=True)
df_macro = pd.read_csv("Macroeconomic_Data.csv", index_col="Date", parse_dates=True)

# Ensure both dataframes are aligned to the same weekly frequency (W-FRI)
# Resample df_sector to 'W-FRI' and take the last value of the week
df_sector = df_sector.resample('W-FRI').last()

# Correct for any missing values introduced by resampling or initial load
df_sector = df_sector.ffill().bfill()

# Merge datasets
df = df_sector.join(df_macro, how="inner")

In [ ]:
#1. Statistical Analysis

# Summary statistics
print(df.describe())

In [ ]:
print("Missing values in the combined DataFrame (df) after join:")
display(df.isnull().sum())

In [ ]:
# 11.0

# Calculate weekly percent change for df_sector
df_sector_pct_change = df_sector.pct_change().dropna()

# Rolling statistics for trend analysis on percent change
rolling_mean_pct_change = df_sector_pct_change.rolling(window=12).mean()
rolling_std_pct_change = df_sector_pct_change.rolling(window=12).std()

plt.figure(figsize=(12, 6))
plt.plot(df_sector_pct_change.index, rolling_mean_pct_change["Technology"], label="Rolling Mean (Technology)", color='blue')
# plt.plot(df_sector_pct_change.index, rolling_std_pct_change["Technology"], label="Rolling Std (Technology)", linestyle="dashed", color='blue')

plt.plot(df_sector_pct_change.index, rolling_mean_pct_change["S&P 500"], label="Rolling Mean (S&P 500)", color='red')
# plt.plot(df_sector_pct_change.index, rolling_std_pct_change["S&P 500"], label="Rolling Std (S&P 500)", linestyle="dashed", color='red')

plt.legend()
plt.title("Sector Performance Trends (Percent Change): Technology vs. S&P 500 (Rolling Mean Only)")
plt.xlabel("Date")
plt.ylabel("Percent Change")
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

In [ ]:
# 12.0

print("--- Weekly Trade Breakdown (Fixed-Threshold Strategy) ---")
weekly_fixed_trades_per_sector = pd.DataFrame(index=signals_aligned.index, columns=signals_aligned.columns, dtype=int)

for i in range(1, len(signals_aligned)):
    current_week_signals = signals_aligned.iloc[i]
    previous_week_signals = signals_aligned.iloc[i-1]

    for sector in signals_aligned.columns:
        if current_week_signals[sector] != previous_week_signals[sector]:
            weekly_fixed_trades_per_sector.loc[signals_aligned.index[i], sector] = 1
        else:
            weekly_fixed_trades_per_sector.loc[signals_aligned.index[i], sector] = 0

# Set the first row to 0 as there's no previous week to compare
weekly_fixed_trades_per_sector.iloc[0] = 0

print("First 10 weeks of fixed strategy trades:")
display(weekly_fixed_trades_per_sector.head(10))

print("\n--- Weekly Trade Breakdown (Dynamic-Threshold Strategy) ---")
weekly_dynamic_trades_per_sector = pd.DataFrame(index=signals_aligned_dynamic.index, columns=signals_aligned_dynamic.columns, dtype=int)

for i in range(1, len(signals_aligned_dynamic)):
    current_week_signals_dynamic = signals_aligned_dynamic.iloc[i]
    previous_week_signals_dynamic = signals_aligned_dynamic.iloc[i-1]

    for sector in signals_aligned_dynamic.columns:
        if current_week_signals_dynamic[sector] != previous_week_signals_dynamic[sector]:
            weekly_dynamic_trades_per_sector.loc[signals_aligned_dynamic.index[i], sector] = 1
        else:
            weekly_dynamic_trades_per_sector.loc[signals_aligned_dynamic.index[i], sector] = 0

# Set the first row to 0
weekly_dynamic_trades_per_sector.iloc[0] = 0

print("First 10 weeks of dynamic strategy trades:")
display(weekly_dynamic_trades_per_sector.head(10))

--- Weekly Trade Breakdown (Fixed-Threshold Strategy) ---


NameError: name 'signals_aligned' is not defined

In [ ]:
# 13.0

weekly_start_date = portfolio_returns.index.min()
weekly_end_date = portfolio_returns.index.max()

monthly_start_date = portfolio_returns_monthly.index.min()
monthly_end_date = portfolio_returns_monthly.index.max()

print(f"--- Backtesting Date Ranges ---")
print(f"Weekly Strategy:   {weekly_start_date.strftime('%Y-%m-%d')} to {weekly_end_date.strftime('%Y-%m-%d')}")
print(f"Monthly Strategy:  {monthly_start_date.strftime('%Y-%m-%d')} to {monthly_end_date.strftime('%Y-%m-%d')}")

print("\n--- Explanation ---")
print("The monthly strategy's backtesting period starts later than the weekly strategy's.")
print("This is because the monthly strategy uses a 3-month rolling mean on monthly percent changes.")
print("For the first 3 months of data to be available for the rolling mean calculation, and then for that to be used to generate signals, the effective start date for the monthly strategy's signals and portfolio returns is delayed compared to the weekly strategy's 12-week rolling mean, even though both use a rolling window.")

In [ ]:
# 14.0

# Analyze trade frequency for the Fixed Threshold Strategy
fixed_signal_changes = 0
for sector in signals_aligned.columns:
    # Compare current signal to previous signal. Shift() moves previous signal to current row
    changes = (signals_aligned[sector] != signals_aligned[sector].shift(1))
    fixed_signal_changes += changes.sum()

print(f"Fixed Threshold Strategy: Total number of signal changes across all sectors = {fixed_signal_changes}")

# Analyze trade frequency for the Dynamic Threshold Strategy
dynamic_signal_changes = 0
for sector in signals_aligned_dynamic.columns:
    # Compare current signal to previous signal
    changes = (signals_aligned_dynamic[sector] != signals_aligned_dynamic[sector].shift(1))
    dynamic_signal_changes += changes.sum()

print(f"Dynamic Threshold Strategy: Total number of signal changes across all sectors = {dynamic_signal_changes}")

num_weeks_total = len(signals_aligned)
num_sectors = len(signals_aligned.columns)

print(f"\nTotal weeks in backtest period: {num_weeks_total}")
print(f"Number of sectors (excluding S&P 500): {num_sectors}")

print(f"\nAverage signal changes per sector per week (Fixed): {fixed_signal_changes / (num_weeks_total * num_sectors):.2f}")
print(f"Average signal changes per sector per week (Dynamic): {dynamic_signal_changes / (num_weeks_total * num_sectors):.2f}")

Fixed Threshold Strategy: Total number of signal changes across all sectors = 1302
Dynamic Threshold Strategy: Total number of signal changes across all sectors = 1325

Total weeks in backtest period: 558
Number of sectors (excluding S&P 500): 10

Average signal changes per sector per week (Fixed): 0.23
Average signal changes per sector per week (Dynamic): 0.24


In [ ]:
# 15.0

# Extract correlations of 'Interest Rate (Fed Funds)' with other variables
interest_rate_correlations = corr_matrix['Interest Rate (Fed Funds)'].drop(['GDP', 'Inflation (CPI)', 'Unemployment Rate', 'Interest Rate (Fed Funds)'])

# Sort the correlations for better visualization
interest_rate_correlations = interest_rate_correlations.sort_values(ascending=False)

plt.figure(figsize=(12, 7))
sns.barplot(x=interest_rate_correlations.index, y=interest_rate_correlations.values, palette='viridis')
plt.title("Correlation between 'Interest Rate (Fed Funds)' and Sector Performance")
plt.xlabel("Sector")
plt.ylabel("Correlation Coefficient")
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

- Technology sector steadily grew, with rapid rise post-2016.

- Volatility remained mostly low, except for a bump in recent years, likely due to economic shocks (e.g., COVID-19, inflation, Fed policy shifts).

In [ ]:
#16.0

#2. Correlation Analysis

# Pearson correlation matrix
corr_matrix = df.corr()
plt.figure(figsize=(12, 8))
sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Matrix")
plt.show()

## Classify Economic Phases

In [ ]:
# 17.0

# Load macroeconomic data
df_macro = pd.read_csv("Macroeconomic_Data.csv", index_col="Date", parse_dates=True)

# Calculate percentage changes
df_macro["GDP Growth (%)"] = df_macro["GDP"].pct_change() * 100
df_macro["Inflation Rate (%)"] = df_macro["Inflation (CPI)"].pct_change() * 100

# Plot histogram for GDP Growth (%)
plt.figure(figsize=(10, 5))
plt.hist(df_macro["GDP Growth (%)"].dropna(), bins=30, color='skyblue', edgecolor='black')
plt.title("Histogram of GDP Growth (%)")
plt.xlabel("GDP Growth (%)")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()

# Plot histogram for Inflation Rate (%)
plt.figure(figsize=(10, 5))
plt.hist(df_macro["Inflation Rate (%)"].dropna(), bins=30, color='salmon', edgecolor='black')
plt.title("Histogram of Inflation Rate (%)")
plt.xlabel("Inflation Rate (%)")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()

In [ ]:
# 18.0

# Define classification logic using percentage changes
def classify_economic_phase(row):
    gdp = row["GDP Growth (%)"]
    inflation = row["Inflation Rate (%)"]
    unemployment = row["Unemployment Rate"]
    interest = row["Interest Rate (Fed Funds)"]

    # Expansion
    if gdp > 0.1 and unemployment < 4:
        return "Expansion"

    # Peak
    elif gdp > 0  and inflation > 0.1 and  interest > 3:
        return "Peak"

    # Recession / Contraction
    elif gdp < 0 and unemployment > 5:
        return "Recession"

    # Recovery / Trough
    else:
        return "Recovery"

# Apply classification to dataset
df_macro["Economic Phase"] = df_macro.apply(classify_economic_phase, axis=1)

# Save classification results to a CSV file
df_macro.to_csv("Economic_Phases.csv", index=True)

print("Economic phases saved successfully!")

Economic phases saved successfully!


In [ ]:
#19.0

# Display results
df_macro.head(10)

## Data Preprocessing

In [ ]:
# 20.0

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler
#from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:

# 21.0

# Load and preprocess macroeconomic data
df = pd.read_csv("Macroeconomic_Data.csv", index_col="Date", parse_dates=True)

# Normalize data
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df)

# Convert data to time-series format
def create_sequences(data, seq_length=12):  # 12-month sequences
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length])
    return np.array(X), np.array(y)

seq_length = 12
X, y = create_sequences(df_scaled, seq_length)

# Split into train & test
split = int(0.8 * len(X))
X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]

In [ ]:
# 22.0

import joblib

# Save the fitted scaler to a file
joblib.dump(scaler, "scaler.pkl")
print("Scaler saved as 'scaler.pkl'")

## LSTM Model

In [ ]:
# 23.0

# 1. LSTM Model for Time-Series Forecasting
lstm_model = Sequential([
    LSTM(64, activation="relu", return_sequences=True, input_shape=(seq_length, X.shape[2])),
    LSTM(64, activation="relu"),
    Dropout(0.01),
    Dense(y.shape[1])  # Output layer
])

lstm_model.compile(optimizer="adam", loss="mse", metrics=["mae"])
lstm_model.fit(X_train, y_train, epochs=40, batch_size=16, validation_data=(X_test, y_test))

# Predictions
y_pred_lstm = lstm_model.predict(X_test)

In [ ]:
# 24.0

# Saved the lstm model
lstm_model.save("lstm_model.keras")
print("LSTM Model is saved successfully!")

LSTM Model is saved successfully!


##  Transformer Model

In [ ]:
# 25.0

# 2. Transformer Model for Long-Term Dependencies
class TransformerTimeSeries(tf.keras.Model):
    def __init__(self, seq_length=12, feature_dim=4, **kwargs):
        super().__init__(**kwargs)
        self.seq_length = seq_length
        self.feature_dim = feature_dim
        self.encoder_layer = tf.keras.layers.MultiHeadAttention(num_heads=4, key_dim=feature_dim)
        self.dense_layer = tf.keras.layers.Dense(feature_dim, activation="relu")

    def call(self, inputs):
        x = self.encoder_layer(inputs, inputs)
        x = self.dense_layer(x)
        return x[:, -1, :]

    def get_config(self):
        config = super().get_config()
        config.update({
            "seq_length": self.seq_length,
            "feature_dim": self.feature_dim,
        })
        return config

    @classmethod
    def from_config(cls, config):
        return cls(**config)

# Instantiate transformer model
transformer_model = TransformerTimeSeries(seq_length, X.shape[2])
transformer_model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Train Transformer model
transformer_model.fit(X_train, y_train, epochs=40, batch_size=16, validation_data=(X_test, y_test))

# Predictions
y_pred_transformer = transformer_model(X_test)

In [ ]:
# 26.0

# Saved the Transformer Model
transformer_model.save_weights("transformer_model.weights.h5")
print("Transformer Model is saved successfully!")

## Model Evaluation

In [ ]:
# 27.0

# 3. Model Evaluation
def evaluate_model(y_true, y_pred, model_name):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    print(f"{model_name} - RMSE: {rmse:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}")

evaluate_model(y_test, y_pred_lstm, "LSTM")
evaluate_model(y_test, y_pred_transformer.numpy(), "Transformer")

In [ ]:
# 28.0

# 4. Visualization
plt.figure(figsize=(10, 5))
plt.plot(y_test[:, 0], label="Actual GDP", linestyle="dashed")
plt.plot(y_pred_lstm[:, 0], label="LSTM Predicted GDP")
plt.plot(y_pred_transformer[:, 0], label="Transformer Predicted GDP")
plt.legend()
plt.title("GDP Prediction Comparison")
plt.show()

## Simplified Q-Learning for Sector Allocation

In [ ]:
# 29.0

import random

# Load data: macroeconomic phases
df_phases = pd.read_csv("Economic_Phases.csv", index_col="Date", parse_dates=True)

# Compute weekly percentage returns
returns = df_sector.pct_change().dropna()

# Merge with economic phases
df_combined = returns.join(df_phases["Economic Phase"]).dropna()

# Calculate average sector returns per economic phase
reward_matrix = df_combined.groupby("Economic Phase").mean()
reward_map = reward_matrix.to_dict(orient="index")

states = reward_matrix.index.tolist()  # [Phases]
actions = reward_matrix.columns.tolist()  # [sectors]

Q = np.zeros((len(states), len(actions)))

alpha = 0.1   # learning rate(0.1)
gamma = 0.99   # discount factor(0.9)
episodes = 10000 # more episodes = more stable learning

for ep in range(episodes):
    state = random.choice(states)
    action_idx = random.randint(0, len(actions)-1)
    action = actions[action_idx]

    reward = reward_map.get(state, {}).get(action, 0.1)
    next_state = random.choice(states)
    best_next = np.max(Q[states.index(next_state)])
    Q[states.index(state), action_idx] += alpha * (
        reward + gamma * best_next - Q[states.index(state), action_idx]
    )

# Extract learned policy
best_actions = np.argmax(Q, axis=1)

policy = {states[i]: actions[best_actions[i]] for i in range(len(states))}

# Display final policy
print("Learned Policy (based on historical sector performance):")
for state, action in policy.items():
    print(f"{state}: {action}")

In [ ]:
# 30.0
import pickle

# Save Q-matrix (as .npy)
np.save("Q_matrix.npy", Q)

# Save policy dictionary (as .pkl)
with open("sector_policy.pkl", "wb") as f:
    pickle.dump(policy, f)

print("Q-matrix saved to 'Q_matrix.npy'")
print("Policy saved to 'sector_policy.pkl'")

## Portfolio Optimization (Modern Portfolio Theory)

In [ ]:
# 31.0

from scipy.optimize import minimize

# Load weekly sector ETF data
df = pd.read_csv("Sector_ETF_Data.csv", index_col="Date", parse_dates=True)
returns = np.log(df / df.shift(1)).dropna()
returns = returns.rolling(12).mean().dropna()  # 12-week moving average

# Annualize mean returns and covariance matrix
mean_returns = returns.mean() * 52  # Weekly to annual
cov_matrix = returns.cov() * 52

# Risk-free rate (e.g., 13-week T-bill ~1% annualized)
rf = 0.01

# Number of assets (ETFs)
n = len(mean_returns)
init_weights = np.ones(n) / n  # Start with equal weights

# Bounds and constraint: weights must sum to 1 and be between 0 and 1
bounds = tuple((0, 1) for _ in range(n))
constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})

# Objective: Maximize Sharpe Ratio (i.e., minimize negative)
def negative_sharpe(weights):
    port_return = np.dot(weights, mean_returns)
    port_vol = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe = (port_return - rf) / port_vol
    return -sharpe

# Run optimization
result = minimize(negative_sharpe, init_weights, method='SLSQP', bounds=bounds, constraints=constraints)
opt_weights = result.x

# Compute portfolio stats
annual_return = np.dot(opt_weights, mean_returns)
annual_volatility = np.sqrt(np.dot(opt_weights.T, np.dot(cov_matrix, opt_weights)))
sharpe_ratio = (annual_return - rf) / annual_volatility

# Create portfolio return series
portfolio_returns_series = returns.dot(opt_weights)
cumulative_return = (1 + portfolio_returns_series).prod() - 1

# Display results
print("Optimized Portfolio Performance Metrics:\n")
print(f"Portfolio Return (Annualized)    : {annual_return:.4f}")
print(f"Portfolio Volatility (Annualized): {annual_volatility:.4f}")
print(f"Sharpe Ratio                     : {sharpe_ratio:.4f}")
print(f"Cumulative Return (Total)        : {cumulative_return:.4f}")

## Economic Phase Prediction & Sector Allocation

In [ ]:
# 32.0

# 1. Load macroeconomic data
df_macro = pd.read_csv("Macroeconomic_Data.csv", index_col="Date", parse_dates=True)

features = ["GDP", "Inflation (CPI)", "Unemployment Rate", "Interest Rate (Fed Funds)"]
recent_data = df_macro[features].dropna()

# 2. Scale and prepare input
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(recent_data)

def create_latest_sequence(data, seq_length=12):
    return np.expand_dims(data[-seq_length:], axis=0)  # (1, seq_len, n_features)

seq_input = create_latest_sequence(data_scaled)

# 3. Used Transformer model to predict
predicted_values = transformer_model.predict(seq_input)
predicted_values = scaler.inverse_transform(predicted_values)[0]
predicted_dict = dict(zip(features, predicted_values))

# 4. Classify phase based on prediction
def classify_economic_phase(row_dict):
    gdp = row_dict["GDP"]
    unemployment = row_dict["Unemployment Rate"]
    inflation = row_dict["Inflation (CPI)"]
    interest = row_dict["Interest Rate (Fed Funds)"]

    if gdp > 2.5 and unemployment < 4:
        return "Expansion"
    elif gdp > 0 and inflation > 3 and interest > 4:
        return "Peak"
    elif gdp < 0 and unemployment > 5:
        return "Recession"
    else:
        return "Recovery"


predicted_dict = dict(zip(features, predicted_values))
economic_phase = classify_economic_phase(predicted_dict)

# Define FRED macroeconomic indicators
fred_symbols = {
    'GDP': 'GDP',
    'Inflation (CPI)': 'CPIAUCSL',
    'Unemployment Rate': 'UNRATE',
    'Interest Rate (Fed Funds)': 'FEDFUNDS'
}

import pandas_datareader.data as web # Import pandas_datareader

# Fetch the most recent available real values from FRED
actual_values = {}
for feature, symbol in fred_symbols.items():
    try:
        data = web.DataReader(symbol, "fred")
        latest_val = data.dropna().iloc[-1].values[0]
        actual_values[feature] = latest_val
    except Exception as e:
        print(f"Error fetching {feature}: {e}")
        actual_values[feature] = None


# Predicted Values
print("Predicted Macroeconomic Values")
for feature in features:
    print(f"{feature}: {predicted_dict[feature]:.2f}")

predicted_phase = classify_economic_phase(predicted_dict)
predicted_sector = policy.get(predicted_phase, "S&P 500")

print(f"\nPredicted Economic Phase: {predicted_phase}")
print(f"Recommended Sector to Overweight: {predicted_sector}")
print("__________________________________________________________________")

# Actual Values
print("\nActual Macroeconomic Values (Most Recent from FRED)")
actual_dict = {}
for feature in features:
    actual_val = actual_values.get(feature, None)
    if actual_val is not None:
        actual_dict[feature] = actual_val
        print(f"{feature}: {actual_val:.2f}")
    else:
        print(f"{feature}: Not Available")

if len(actual_dict) == len(features):
    actual_phase = classify_economic_phase(actual_dict)
    actual_sector = policy.get(actual_phase, "S&P 500")

    print(f"\nActual Economic Phase: {actual_phase}")
    print(f"Recommended Sector to Overweight: {actual_sector}")
else:
    print("\nActual Economic Phase: Cannot determine (missing data)")

print("__________________________________________________________________")

# Optional: Error Comparison
print("\nError (Predicted - Actual)")
for feature in features:
    pred_val = predicted_dict[feature]
    actual_val = actual_values.get(feature, None)
    if actual_val is not None:
        error = pred_val - actual_val
        print(f"{feature}: Error = {error:.2f}")
    else:
        print(f"{feature}: Error = Cannot compute (Actual not available)")

## Predicted vs Actual Macroeconomic Indicators

In [ ]:
# 33.0

# Prepare data
feature_labels = features
predicted_vals = [predicted_dict[feature] for feature in features]
actual_vals = [actual_values.get(feature, np.nan) for feature in features]

# Create a single row of subplots
fig, axes = plt.subplots(1, len(feature_labels), figsize=(12, 5))  # Width x Height

for i, ax in enumerate(axes):
    feature = feature_labels[i]
    pred = predicted_vals[i]
    actual = actual_vals[i]

    # Convert actual to np.nan if it's None, for robust plotting
    plot_actual = actual if actual is not None else np.nan

    # Bar chart for each feature
    bars = ax.bar(['Predicted', 'Actual'], [pred, plot_actual], color=['skyblue', 'salmon'])

    ax.set_title(f'{feature}', fontsize=10)
    ax.tick_params(axis='both', labelsize=8)
    ax.grid(axis='y', linestyle='--', alpha=0.6)

    # Annotate values on bars, handling potential NaN heights
    for bar in bars:
        height = bar.get_height()
        if not np.isnan(height):
            ax.annotate(f'{height:.2f}',
                        xy=(bar.get_x() + bar.get_width() / 2, height),
                        xytext=(0, 3),
                        textcoords="offset points",
                        ha='center', va='bottom', fontsize=8)

# Adjust layout
plt.tight_layout()
plt.show()

In [ ]:
# 34.0

sector_etfs = [col for col in rolling_mean_pct_change.columns if col != "S&P 500"]

for sector in sector_etfs:
    plt.figure(figsize=(12, 6))
    plt.plot(rolling_mean_pct_change.index, rolling_mean_pct_change[sector], label=f"Rolling Mean ({sector})", color='blue')
    plt.plot(rolling_mean_pct_change.index, rolling_mean_pct_change["S&P 500"], label="Rolling Mean (S&P 500)", color='red')
    plt.legend()
    plt.title(f"Sector Performance Trends (Percent Change): {sector} vs. S&P 500 (Rolling Mean)")
    plt.xlabel("Date")
    plt.ylabel("Percent Change")
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()

In [ ]:
# 36.0

relative_performance = pd.DataFrame(index=rolling_mean_pct_change.index)

sector_etfs = [col for col in rolling_mean_pct_change.columns if col != "S&P 500"]

for sector in sector_etfs:
    relative_performance[sector] = rolling_mean_pct_change[sector] - rolling_mean_pct_change["S&P 500"]

print("Relative performance DataFrame created successfully!")
print("First 5 rows of relative_performance:")
display(relative_performance.head())

In [ ]:
# 37.0

rolling_mean_pct_change_cleaned = rolling_mean_pct_change.dropna()

relative_performance = pd.DataFrame(index=rolling_mean_pct_change_cleaned.index)

sector_etfs = [col for col in rolling_mean_pct_change_cleaned.columns if col != "S&P 500"]

for sector in sector_etfs:
    relative_performance[sector] = rolling_mean_pct_change_cleaned[sector] - rolling_mean_pct_change_cleaned["S&P 500"]

print("Relative performance DataFrame created successfully after cleaning NaNs!")
print("First 5 rows of relative_performance:")
display(relative_performance.head())

## Define Buy/Sell Signals

### Subtask:
Based on the calculated relative performance, define thresholds to generate 'buy', 'sell', or 'hold' signals for each sector. For instance, a sector might be a 'buy' if its relative performance is above a certain positive threshold, and a 'sell' if below a certain negative threshold.


In [ ]:
# 38.0

buy_threshold = 0.001
sell_threshold = -0.001

signals = pd.DataFrame(index=relative_performance.index)

for sector in relative_performance.columns:
    signals[sector] = 'Hold'  # Initialize all to 'Hold'
    signals.loc[relative_performance[sector] > buy_threshold, sector] = 'Buy'
    signals.loc[relative_performance[sector] < sell_threshold, sector] = 'Sell'

print("Signals DataFrame created successfully!")
print("First 5 rows of signals:")
display(signals.head())

## Backtest Strategy

### Subtask:
Simulate the sector rotation strategy over the historical period. This will involve applying the generated buy/sell signals to construct a hypothetical portfolio and calculating its returns. We'll start with an equal-weighted portfolio for 'hold' signals and adjust weights for 'buy'/'sell' signals.


In [ ]:
# 39.0

import pandas as pd
import numpy as np

# 1. Initialize an empty DataFrame called portfolio_returns
portfolio_returns = pd.DataFrame(index=signals.index, columns=['Portfolio Return'])

# Get list of sector ETFs (excluding S&P 500)
sector_etfs = [col for col in df_sector_pct_change.columns if col != "S&P 500"]
num_tradeable_sectors = len(sector_etfs)

# Align returns and signals to ensure they cover the same time period
# This ensures both DataFrames start and end on the same dates.
returns_aligned = df_sector_pct_change[sector_etfs].align(signals, join='inner', axis=0)[0]
signals_aligned = signals.align(returns_aligned, join='inner', axis=0)[0]

# 2. Initialize portfolio_weights. Start with an equal weight for all sectors
# These are the weights assumed at the start of the *first* period for which we have signals.
current_portfolio_weights_at_start_of_period = {sector: 1.0 / num_tradeable_sectors for sector in sector_etfs}

# Create a DataFrame to store historical weights for analysis (optional, but good for inspection)
historical_weights = pd.DataFrame(columns=sector_etfs, index=signals_aligned.index)

# 3. Iterate through each time step (row) in the aligned signals DataFrame
for i in range(len(signals_aligned)):
    current_date = signals_aligned.index[i]
    current_signals = signals_aligned.loc[current_date] # Signals for the current period
    current_week_returns = returns_aligned.loc[current_date] # Returns for the current period

    # `weights_for_this_period` will hold the actual weights used for the current period's return calculation.
    # Initialize all to 0.0, and then assign based on signals.
    weights_for_this_period = {sector: 0.0 for sector in sector_etfs}

    buy_sectors = [s for s in sector_etfs if current_signals[s] == 'Buy']
    sell_sectors = [s for s in sector_etfs if current_signals[s] == 'Sell']
    hold_sectors = [s for s in sector_etfs if current_signals[s] == 'Hold']

    # Determine the 'active' sectors for this period (those not getting a 'Sell' signal).
    # The prompt's instructions ("new, higher weight for buy", "maintain previous weight for hold",
    # and "re-normalize all weights") imply that capital is re-allocated among active sectors.
    # The most consistent interpretation ensuring weights sum to 1 is to equally weight active sectors.
    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period[sector] = weight_per_active_sector
        # Sectors not in active_sectors_this_period (i.e., sell_sectors) implicitly retain 0.0 weight.
    else:
        # If all sectors are 'Sell', or no active signals, all weights remain 0.0.
        # Portfolio return for this week will be 0.0.
        pass # weights_for_this_period is already initialized to 0.0

    # Store the weights used for this period calculation in the historical_weights DataFrame.
    historical_weights.loc[current_date] = list(weights_for_this_period.values())

    # Calculate portfolio return for the current week using the determined weights.
    weekly_port_return = sum(weights_for_this_period[s] * current_week_returns[s] for s in sector_etfs)
    portfolio_returns.loc[current_date, 'Portfolio Return'] = weekly_port_return

    # Update `current_portfolio_weights_at_start_of_period` for the *next* iteration.
    # This reflects the portfolio composition at the end of the current period, which is the start of the next.
    current_portfolio_weights_at_start_of_period = weights_for_this_period

# 4. Calculate the cumulative returns of the portfolio_returns
# (1 + Return) cumprod() - 1 gives the cumulative return based on compounding.
portfolio_returns['Cumulative Return'] = (1 + portfolio_returns['Portfolio Return']).cumprod() - 1

print("Portfolio simulation complete.")
print("First 5 rows of portfolio_returns:")
display(portfolio_returns.head())
print("Last 5 rows of portfolio_returns:")
display(portfolio_returns.tail())

In [ ]:
# 40.0

plt.figure(figsize=(14, 7))
plt.plot(portfolio_returns.index, portfolio_returns['Cumulative Return'], label='Portfolio Cumulative Return', color='blue')
plt.title('Cumulative Returns of Sector Rotation Strategy')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend()
plt.show()

## Evaluate Strategy vs. S&P 500

### Subtask:
Compare the performance of the simulated sector rotation strategy against the S&P 500 (SPY) using key metrics like total cumulative return, annualized return, and potentially other risk-adjusted measures.


**Reasoning**:
First, I will calculate the cumulative return for the S&P 500 over the same period as the portfolio. This involves aligning the S&P 500 weekly percent changes with the portfolio's returns and then computing the cumulative product.



In [ ]:
# 41.0

s_p_500_returns_aligned = df_sector_pct_change["S&P 500"].align(portfolio_returns, join='inner', axis=0)[0]
s_p_500_cumulative_return = (1 + s_p_500_returns_aligned).cumprod() - 1

print("S&P 500 Cumulative Returns calculated.")
print("First 5 rows of S&P 500 cumulative returns:")
display(s_p_500_cumulative_return.head())
print("Last 5 rows of S&P 500 cumulative returns:")
display(s_p_500_cumulative_return.tail())

In [ ]:
# 42.0

num_weeks = len(portfolio_returns)
num_years = num_weeks / 52.0

# Portfolio Metrics
portfolio_total_cumulative_return = portfolio_returns['Cumulative Return'].iloc[-1]
portfolio_annualized_return = (1 + portfolio_total_cumulative_return)**(1/num_years) - 1
portfolio_annualized_volatility = portfolio_returns['Portfolio Return'].std() * np.sqrt(52)

# S&P 500 Metrics
s_p_500_total_cumulative_return = s_p_500_cumulative_return.iloc[-1]
s_p_500_annualized_return = (1 + s_p_500_total_cumulative_return)**(1/num_years) - 1
s_p_500_annualized_volatility = s_p_500_returns_aligned.std() * np.sqrt(52)

# Risk-free rate (already defined as rf = 0.01)

# Sharpe Ratio
portfolio_sharpe_ratio = (portfolio_annualized_return - rf) / portfolio_annualized_volatility
s_p_500_sharpe_ratio = (s_p_500_annualized_return - rf) / s_p_500_annualized_volatility

# Print Summary
print("\n--- Strategy Performance vs. S&P 500 ---")
print(f"Total Cumulative Return (Strategy): {portfolio_total_cumulative_return:.4f}")
print(f"Total Cumulative Return (S&P 500): {s_p_500_total_cumulative_return:.4f}")
print("\n")
print(f"Annualized Return (Strategy):       {portfolio_annualized_return:.4f}")
print(f"Annualized Return (S&P 500):        {s_p_500_annualized_return:.4f}")
print("\n")
print(f"Annualized Volatility (Strategy):   {portfolio_annualized_volatility:.4f}")
print(f"Annualized Volatility (S&P 500):    {s_p_500_annualized_volatility:.4f}")
print("\n")
print(f"Sharpe Ratio (Strategy):            {portfolio_sharpe_ratio:.4f}")
print(f"Sharpe Ratio (S&P 500):             {s_p_500_sharpe_ratio:.4f}")
print("------------------------------------")

## Visualize Strategy Performance

### Subtask:
Plot the cumulative returns of the sector rotation strategy alongside the cumulative returns of the S&P 500 to visually assess the outperformance or underperformance.


In [ ]:
# 43.0

plt.figure(figsize=(14, 7))
plt.plot(portfolio_returns.index, portfolio_returns['Cumulative Return'], label='Portfolio Cumulative Return', color='blue')
plt.plot(s_p_500_cumulative_return.index, s_p_500_cumulative_return, label='S&P 500 Cumulative Return', color='red')
plt.title('Cumulative Returns: Strategy vs. S&P 500')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend()
plt.show()

## Calculate Dynamic Thresholds

### Subtask:
Calculate dynamic buy/sell thresholds based on a rolling measure of volatility of the relative performance.


In [ ]:
# 44.0

rolling_std_relative_performance = relative_performance.rolling(window=12).std()

multiplier = 0.5 # Adjust this multiplier as needed

dynamic_buy_thresholds = multiplier * rolling_std_relative_performance
dynamic_sell_thresholds = -multiplier * rolling_std_relative_performance

print("Dynamic buy thresholds calculated successfully!\nFirst 5 rows:")
display(dynamic_buy_thresholds.head())

print("\nDynamic sell thresholds calculated successfully!\nFirst 5 rows:")
display(dynamic_sell_thresholds.head())

## Generate Signals with Dynamic Thresholds

### Subtask:
Apply the newly calculated dynamic thresholds to the `relative_performance` DataFrame to generate updated 'buy', 'sell', or 'hold' signals for each sector.


In [ ]:
# 45.0

signals_dynamic = pd.DataFrame(index=relative_performance.index)

for sector in relative_performance.columns:
    signals_dynamic[sector] = 'Hold'  # Initialize all to 'Hold'
    # Apply 'Buy' signal where relative performance is above the dynamic buy threshold
    buy_condition = relative_performance[sector] > dynamic_buy_thresholds[sector]
    signals_dynamic.loc[buy_condition, sector] = 'Buy'

    # Apply 'Sell' signal where relative performance is below the dynamic sell threshold
    sell_condition = relative_performance[sector] < dynamic_sell_thresholds[sector]
    signals_dynamic.loc[sell_condition, sector] = 'Sell'

print("Dynamic signals DataFrame created successfully!")
print("First 5 rows of signals_dynamic:")
display(signals_dynamic.head())

## Backtest Strategy with Dynamic Thresholds

### Subtask:
Re-simulate the sector rotation strategy using these new dynamic signals. This involves iterating through historical time steps, adjusting portfolio weights based on the dynamic signals, and calculating weekly portfolio returns.


In [ ]:
# 46.0

import pandas as pd
import numpy as np

# 1. Initialize an empty DataFrame called portfolio_returns_dynamic
portfolio_returns_dynamic = pd.DataFrame(index=signals_dynamic.index, columns=['Portfolio Return'])

# Get list of sector ETFs (excluding S&P 500)
sector_etfs = [col for col in df_sector_pct_change.columns if col != "S&P 500"]
num_tradeable_sectors = len(sector_etfs)

# 3. Align returns and signals_dynamic to ensure they cover the same time period
returns_aligned_dynamic = df_sector_pct_change[sector_etfs].align(signals_dynamic, join='inner', axis=0)[0]
signals_aligned_dynamic = signals_dynamic.align(returns_aligned_dynamic, join='inner', axis=0)[0]

# Remove any NaNs that might have resulted from the rolling std calculation at the beginning
# This ensures that signals_aligned_dynamic only contains periods where dynamic thresholds could be calculated.
valid_indices = signals_aligned_dynamic.dropna(how='all').index
signals_aligned_dynamic = signals_aligned_dynamic.loc[valid_indices]
returns_aligned_dynamic = returns_aligned_dynamic.loc[valid_indices]
portfolio_returns_dynamic = portfolio_returns_dynamic.loc[valid_indices]

# 4. Initialize current_portfolio_weights_at_start_of_period_dynamic
# These are the weights assumed at the start of the *first* period for which we have signals.
current_portfolio_weights_at_start_of_period_dynamic = {sector: 1.0 / num_tradeable_sectors for sector in sector_etfs}

# Create a DataFrame to store historical weights for analysis (optional)
historical_weights_dynamic = pd.DataFrame(columns=sector_etfs, index=signals_aligned_dynamic.index)

# 5. Iterate through each time step (row) in the aligned signals_dynamic DataFrame
for i in range(len(signals_aligned_dynamic)):
    current_date = signals_aligned_dynamic.index[i]
    current_signals = signals_aligned_dynamic.loc[current_date] # Signals for the current period
    current_week_returns = returns_aligned_dynamic.loc[current_date] # Returns for the current period

    # `weights_for_this_period_dynamic` will hold the actual weights used for the current period's return calculation.
    # Initialize all to 0.0, and then assign based on signals.
    weights_for_this_period_dynamic = {sector: 0.0 for sector in sector_etfs}

    buy_sectors = [s for s in sector_etfs if current_signals[s] == 'Buy']
    sell_sectors = [s for s in sector_etfs if current_signals[s] == 'Sell']
    hold_sectors = [s for s in sector_etfs if current_signals[s] == 'Hold']

    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_dynamic[sector] = weight_per_active_sector
    else:
        # If all sectors are 'Sell', or no active signals, all weights remain 0.0.
        pass # weights_for_this_period_dynamic is already initialized to 0.0

    # Store the weights used for this period calculation
    historical_weights_dynamic.loc[current_date] = list(weights_for_this_period_dynamic.values())

    # Calculate portfolio return for the current week using the determined weights.
    weekly_port_return = sum(weights_for_this_period_dynamic[s] * current_week_returns[s] for s in sector_etfs)
    portfolio_returns_dynamic.loc[current_date, 'Portfolio Return'] = weekly_port_return

    # Update `current_portfolio_weights_at_start_of_period_dynamic` for the *next* iteration.
    current_portfolio_weights_at_start_of_period_dynamic = weights_for_this_period_dynamic

# 6. Calculate the cumulative returns of the portfolio_returns_dynamic
portfolio_returns_dynamic['Cumulative Return'] = (1 + portfolio_returns_dynamic['Portfolio Return']).cumprod() - 1

print("Dynamic portfolio simulation complete.")
# 7. Print the first 5 and last 5 rows of the portfolio_returns_dynamic DataFrame
print("First 5 rows of portfolio_returns_dynamic:")
display(portfolio_returns_dynamic.head())
print("Last 5 rows of portfolio_returns_dynamic:")
display(portfolio_returns_dynamic.tail())

## Evaluate Strategy Performance (Dynamic vs. Fixed)

### Subtask:
Compare the performance of the strategy using dynamic thresholds against the original strategy with fixed thresholds and the S&P 500, using metrics like total cumulative return, annualized return, annualized volatility, and Sharpe Ratio.


In [ ]:
# 47.0

num_weeks_dynamic = len(portfolio_returns_dynamic)
num_years_dynamic = num_weeks_dynamic / 52.0

# Dynamic Strategy Metrics
portfolio_total_cumulative_return_dynamic = portfolio_returns_dynamic['Cumulative Return'].iloc[-1]
portfolio_annualized_return_dynamic = (1 + portfolio_total_cumulative_return_dynamic)**(1/num_years_dynamic) - 1
portfolio_annualized_volatility_dynamic = portfolio_returns_dynamic['Portfolio Return'].std() * np.sqrt(52)
portfolio_sharpe_ratio_dynamic = (portfolio_annualized_return_dynamic - rf) / portfolio_annualized_volatility_dynamic

# Print Summary Comparison
print("\n--- Strategy Performance Comparison (Fixed vs. Dynamic vs. S&P 500) ---")
print(f"Metric                        | Fixed Threshold | Dynamic Threshold | S&P 500       ")
print(f"---------------------------------------------------------------------------------")
print(f"Total Cumulative Return       | {portfolio_total_cumulative_return:.4f}        | {portfolio_total_cumulative_return_dynamic:.4f}         | {s_p_500_total_cumulative_return:.4f}       ")
print(f"Annualized Return             | {portfolio_annualized_return:.4f}        | {portfolio_annualized_return_dynamic:.4f}         | {s_p_500_annualized_return:.4f}       ")
print(f"Annualized Volatility         | {portfolio_annualized_volatility:.4f}        | {portfolio_annualized_volatility_dynamic:.4f}         | {s_p_500_annualized_volatility:.4f}       ")
print(f"Sharpe Ratio                  | {portfolio_sharpe_ratio:.4f}        | {portfolio_sharpe_ratio_dynamic:.4f}         | {s_p_500_sharpe_ratio:.4f}       ")
print("---------------------------------------------------------------------------------")

## Visualize Strategy Performance (Dynamic vs. Fixed)

### Subtask:
Plot the cumulative returns of the strategy with dynamic thresholds alongside the original strategy (fixed thresholds) and the S&P 500 to visually assess the impact of dynamic thresholds.


In [ ]:
# 48.0

plt.figure(figsize=(14, 7))
plt.plot(portfolio_returns_dynamic.index, portfolio_returns_dynamic['Cumulative Return'], label='Dynamic Strategy Cumulative Return', color='green')
plt.plot(portfolio_returns.index, portfolio_returns['Cumulative Return'], label='Fixed Strategy Cumulative Return', color='blue')
plt.plot(s_p_500_cumulative_return.index, s_p_500_cumulative_return, label='S&P 500 Cumulative Return', color='red')
plt.title('Cumulative Returns: Dynamic vs. Fixed Strategy vs. S&P 500')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend()
plt.show()

In [ ]:
# 49.0

import pandas as pd
import numpy as np

# 1. Initialize an empty DataFrame named portfolio_returns_dynamic_with_costs
portfolio_returns_dynamic_with_costs = pd.DataFrame(index=signals_aligned_dynamic.index, columns=['Portfolio Return'])

# 2. Define the transaction_cost_rate
transaction_cost_rate = 0.001  # 0.1% transaction cost

# Get list of sector ETFs (excluding S&P 500)
sector_etfs = [col for col in df_sector_pct_change.columns if col != "S&P 500"]
num_tradeable_sectors = len(sector_etfs)

# 3. Initialize previous_period_weights
previous_period_weights = {sector: 1.0 / num_tradeable_sectors for sector in sector_etfs}

# Create a DataFrame to store historical weights for analysis (optional)
historical_weights_with_costs = pd.DataFrame(columns=sector_etfs, index=signals_aligned_dynamic.index)

# 4. Iterate through each time step (row) in the aligned signals_dynamic DataFrame
for i in range(len(signals_aligned_dynamic)):
    current_date = signals_aligned_dynamic.index[i]
    current_signals = signals_aligned_dynamic.loc[current_date] # Signals for the current period
    current_week_returns = returns_aligned_dynamic.loc[current_date] # Returns for the current period

    # Determine the weights_for_this_period_dynamic
    weights_for_this_period_dynamic = {sector: 0.0 for sector in sector_etfs}
    buy_sectors = [s for s in sector_etfs if current_signals[s] == 'Buy']
    hold_sectors = [s for s in sector_etfs if current_signals[s] == 'Hold']
    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_dynamic[sector] = weight_per_active_sector

    # Calculate transaction_costs for this period
    transaction_costs = 0.0
    for sector in sector_etfs:
        weight_change = abs(weights_for_this_period_dynamic[sector] - previous_period_weights[sector])
        transaction_costs += weight_change * transaction_cost_rate

    # Calculate weekly_port_return_before_costs
    weekly_port_return_before_costs = sum(weights_for_this_period_dynamic[s] * current_week_returns[s] for s in sector_etfs)

    # Deduct transaction_costs
    weekly_port_return_after_costs = weekly_port_return_before_costs - transaction_costs
    portfolio_returns_dynamic_with_costs.loc[current_date, 'Portfolio Return'] = weekly_port_return_after_costs

    # Update previous_period_weights for the next iteration
    previous_period_weights = weights_for_this_period_dynamic

# 5. Calculate the cumulative returns
portfolio_returns_dynamic_with_costs['Cumulative Return'] = (1 + portfolio_returns_dynamic_with_costs['Portfolio Return']).cumprod() - 1

print("Dynamic portfolio simulation with transaction costs complete.")
# 6. Print the first 5 and last 5 rows of the portfolio_returns_dynamic_with_costs DataFrame
print("First 5 rows of portfolio_returns_dynamic_with_costs:")
display(portfolio_returns_dynamic_with_costs.head())
print("Last 5 rows of portfolio_returns_dynamic_with_costs:")
display(portfolio_returns_dynamic_with_costs.tail())

In [ ]:
# 50.0

num_weeks_dynamic_costs = len(portfolio_returns_dynamic_with_costs)
num_years_dynamic_costs = num_weeks_dynamic_costs / 52.0

# Dynamic Strategy with Costs Metrics
portfolio_total_cumulative_return_dynamic_costs = portfolio_returns_dynamic_with_costs['Cumulative Return'].iloc[-1]
portfolio_annualized_return_dynamic_costs = (1 + portfolio_total_cumulative_return_dynamic_costs)**(1/num_years_dynamic_costs) - 1
portfolio_annualized_volatility_dynamic_costs = portfolio_returns_dynamic_with_costs['Portfolio Return'].std() * np.sqrt(52)
portfolio_sharpe_ratio_dynamic_costs = (portfolio_annualized_return_dynamic_costs - rf) / portfolio_annualized_volatility_dynamic_costs

# Print Summary Comparison
print("\n--- Strategy Performance Comparison (Dynamic with Costs vs. Dynamic vs. Fixed vs. S&P 500) ---")
print(f"Metric                        | Dynamic Costs   | Dynamic         | Fixed Threshold | S&P 500       ")
print(f"--------------------------------------------------------------------------------------------------")
print(f"Total Cumulative Return       | {portfolio_total_cumulative_return_dynamic_costs:.4f}        | {portfolio_total_cumulative_return_dynamic:.4f}         | {portfolio_total_cumulative_return:.4f}        | {s_p_500_total_cumulative_return:.4f}       ")
print(f"Annualized Return             | {portfolio_annualized_return_dynamic_costs:.4f}        | {portfolio_annualized_return_dynamic:.4f}         | {portfolio_annualized_return:.4f}        | {s_p_500_annualized_return:.4f}       ")
print(f"Annualized Volatility         | {portfolio_annualized_volatility_dynamic_costs:.4f}        | {portfolio_annualized_volatility_dynamic:.4f}         | {portfolio_annualized_volatility:.4f}        | {s_p_500_annualized_volatility:.4f}       ")
print(f"Sharpe Ratio                  | {portfolio_sharpe_ratio_dynamic_costs:.4f}        | {portfolio_sharpe_ratio_dynamic:.4f}         | {portfolio_sharpe_ratio:.4f}        | {s_p_500_sharpe_ratio:.4f}       ")
print("--------------------------------------------------------------------------------------------------")

In [ ]:
# 51.0

plt.figure(figsize=(14, 7))
plt.plot(portfolio_returns_dynamic_with_costs.index, portfolio_returns_dynamic_with_costs['Cumulative Return'], label='Dynamic Strategy (with Costs) Cumulative Return', color='purple')
plt.plot(portfolio_returns_dynamic.index, portfolio_returns_dynamic['Cumulative Return'], label='Dynamic Strategy (without Costs) Cumulative Return', color='green')
plt.plot(portfolio_returns.index, portfolio_returns['Cumulative Return'], label='Fixed Strategy Cumulative Return', color='blue')
plt.plot(s_p_500_cumulative_return.index, s_p_500_cumulative_return, label='S&P 500 Cumulative Return', color='red')
plt.title('Cumulative Returns: Dynamic (with/without Costs) vs. Fixed Strategy vs. S&P 500')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend()
plt.show()

In [ ]:
# 52.0

fixed_strategy_weekly_trades = pd.Series(index=signals_aligned.index, dtype=int)
dynamic_strategy_weekly_trades = pd.Series(index=signals_aligned_dynamic.index, dtype=int)

# Calculate weekly trade counts for the fixed-threshold strategy
for i in range(1, len(signals_aligned)):
    current_week_signals = signals_aligned.iloc[i]
    previous_week_signals = signals_aligned.iloc[i-1]

    # Count signal changes for the current week compared to the previous week
    # A change occurs if the signal for a sector is different from the previous week's signal
    num_changes = (current_week_signals != previous_week_signals).sum()
    fixed_strategy_weekly_trades.iloc[i] = num_changes

# The first week has no previous week to compare to, so we can set its trade count to 0 or NaN
# For simplicity, let's set it to 0 as no 'trade' would occur based on a change.
fixed_strategy_weekly_trades.iloc[0] = 0

# Calculate weekly trade counts for the dynamic-threshold strategy
for i in range(1, len(signals_aligned_dynamic)):
    current_week_signals_dynamic = signals_aligned_dynamic.iloc[i]
    previous_week_signals_dynamic = signals_aligned_dynamic.iloc[i-1]

    # Count signal changes for the current week compared to the previous week
    num_changes_dynamic = (current_week_signals_dynamic != previous_week_signals_dynamic).sum()
    dynamic_strategy_weekly_trades.iloc[i] = num_changes_dynamic

# The first week for dynamic strategy (after NaN removal) also has no previous week
dynamic_strategy_weekly_trades.iloc[0] = 0

print("Fixed Strategy Weekly Trade Counts (First 5 rows):")
display(fixed_strategy_weekly_trades.head())

print("\nDynamic Strategy Weekly Trade Counts (First 5 rows):")
display(dynamic_strategy_weekly_trades.head())

20251230 14:52 MODEL RUN W/ DEC DATA

In [ ]:
# 114.0

import pandas as pd

df_sector = pd.read_csv("Sector_ETF_Data.csv", index_col="Date", parse_dates=True)

# Correct for missing values using ffill() and then bfill()
df_sector = df_sector.ffill()
df_sector = df_sector.bfill()

print("Missing values after correction:")
display(df_sector.isnull().sum())

print("DataFrame head after missing value correction:")
display(df_sector.head())

In [ ]:
# 115.0

import pandas as pd
from fredapi import Fred

# FRED API key - Ensure this is correctly set or retrieved from a secure source.
# For demonstration, using the API key from the provided notebook context.
fred = Fred(api_key='e8fb9c414f6ab8f1a06a5472466a31dc')

# Define date range for weekly index, consistent with previous data fetches
start_date = "2015-01-01"
end_date = "2025-11-30"
weekly_index = pd.date_range(start=start_date, end=end_date, freq='W-FRI').strftime('%Y-%m-%d')

# Define FRED series IDs and their corresponding labels
series_dict = {
    'GDP': 'GDP',
    'Inflation (CPI)': 'CPIAUCSL',
    'Unemployment Rate': 'UNRATE',
    'Interest Rate (Fed Funds)': 'FEDFUNDS'
}

# Create a DataFrame with weekly index to store macroeconomic data
macro_data = pd.DataFrame(index=pd.to_datetime(weekly_index))
macro_data.index.name = "Date"

# Fetch and interpolate data for each series
for label, series_id in series_dict.items():
    data = fred.get_series(series_id)
    if data is None: # Handle cases where FRED API might return None
        print(f"Warning: Could not fetch series {series_id} for {label}")
        continue

    data = data.to_frame(name=label)
    data.index = pd.to_datetime(data.index)

    # Resample to daily, interpolating or forward-filling as specified
    if label == 'Interest Rate (Fed Funds)':
        daily = data.resample('D').ffill()
    else:
        # For other series, interpolate missing values to daily frequency
        daily = data.resample('D').interpolate(method='linear')

    # Reindex the daily data to the predefined weekly index and interpolate remaining missing values
    # Use limit_direction='both' for interpolation to handle NaNs at ends of series
    weekly_processed = daily.reindex(macro_data.index).interpolate(method='linear', limit_direction='both')
    macro_data[label] = weekly_processed[label]

# Save the completed macroeconomic DataFrame to a CSV file
macro_data.to_csv("Macroeconomic_Data.csv", index=True)

print("Macroeconomic Data fetched, processed, and saved successfully to 'Macroeconomic_Data.csv'!")

In [ ]:
# 116.0

df_sector_pct_change = df_sector.pct_change().dropna()

rolling_mean_pct_change = df_sector_pct_change.rolling(window=12).mean().dropna()

relative_performance = pd.DataFrame(index=rolling_mean_pct_change.index)

sector_etfs = [col for col in rolling_mean_pct_change.columns if col != "S&P 500"]

for sector in sector_etfs:
    relative_performance[sector] = rolling_mean_pct_change[sector] - rolling_mean_pct_change["S&P 500"]

print("df_sector_pct_change, rolling_mean_pct_change, and relative_performance recalculated successfully!")
print("First 5 rows of relative_performance:")
display(relative_performance.head())

In [ ]:
# 117.0

buy_threshold = 0.001
sell_threshold = -0.001

signals = pd.DataFrame(index=relative_performance.index)

for sector in relative_performance.columns:
    signals[sector] = 'Hold'  # Initialize all to 'Hold'
    signals.loc[relative_performance[sector] > buy_threshold, sector] = 'Buy'
    signals.loc[relative_performance[sector] < sell_threshold, sector] = 'Sell'

print("Fixed-threshold signals DataFrame regenerated successfully!")
print("First 5 rows of signals:")
display(signals.head())

# Re-run the backtesting simulation for the fixed-threshold strategy
# 1. Initialize an empty DataFrame called portfolio_returns
portfolio_returns = pd.DataFrame(index=signals.index, columns=['Portfolio Return'])

# Get list of sector ETFs (excluding S&P 500)
sector_etfs = [col for col in df_sector_pct_change.columns if col != "S&P 500"]
num_tradeable_sectors = len(sector_etfs)

# Align returns and signals to ensure they cover the same time period
# This ensures both DataFrames start and end on the same dates.
returns_aligned = df_sector_pct_change[sector_etfs].align(signals, join='inner', axis=0)[0]
signals_aligned = signals.align(returns_aligned, join='inner', axis=0)[0]

# 2. Initialize current_portfolio_weights_at_start_of_period
# These are the weights assumed at the start of the *first* period for which we have signals.
current_portfolio_weights_at_start_of_period = {sector: 1.0 / num_tradeable_sectors for sector in sector_etfs}

# Create a DataFrame to store historical weights for analysis (optional, but good for inspection)
historical_weights = pd.DataFrame(columns=sector_etfs, index=signals_aligned.index)

# 3. Iterate through each time step (row) in the aligned signals DataFrame
for i in range(len(signals_aligned)):
    current_date = signals_aligned.index[i]
    current_signals = signals_aligned.loc[current_date] # Signals for the current period
    current_week_returns = returns_aligned.loc[current_date] # Returns for the current period

    # `weights_for_this_period` will hold the actual weights used for the current period's return calculation.
    # Initialize all to 0.0, and then assign based on signals.
    weights_for_this_period = {sector: 0.0 for sector in sector_etfs}

    buy_sectors = [s for s in sector_etfs if current_signals[s] == 'Buy']
    sell_sectors = [s for s in sector_etfs if current_signals[s] == 'Sell']
    hold_sectors = [s for s in sector_etfs if current_signals[s] == 'Hold']

    # Determine the 'active' sectors for this period (those not getting a 'Sell' signal).
    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period[sector] = weight_per_active_sector
        # Sectors not in active_sectors_this_period (i.e., sell_sectors) implicitly retain 0.0 weight.
    else:
        # If all sectors are 'Sell', or no active signals, all weights remain 0.0.
        # Portfolio return for this week will be 0.0.
        pass # weights_for_this_period is already initialized to 0.0

    # Store the weights used for this period calculation in the historical_weights DataFrame.
    historical_weights.loc[current_date] = list(weights_for_this_period.values())

    # Calculate portfolio return for the current week using the determined weights.
    weekly_port_return = sum(weights_for_this_period[s] * current_week_returns[s] for s in sector_etfs)
    portfolio_returns.loc[current_date, 'Portfolio Return'] = weekly_port_return

    # Update `current_portfolio_weights_at_start_of_period` for the *next* iteration.
    # This reflects the portfolio composition at the end of the current period, which is the start of the next.
    current_portfolio_weights_at_start_of_period = weights_for_this_period

# 4. Calculate the cumulative returns of the portfolio_returns
# (1 + Return) cumprod() - 1 gives the cumulative return based on compounding.
portfolio_returns['Cumulative Return'] = (1 + portfolio_returns['Portfolio Return']).cumprod() - 1

print("Fixed-threshold backtesting simulation complete.")
print("First 5 rows of portfolio_returns:")
display(portfolio_returns.head())

In [ ]:
# 118.0

rolling_std_relative_performance = relative_performance.rolling(window=12).std()

multiplier = 0.5 # Adjust this multiplier as needed

dynamic_buy_thresholds = multiplier * rolling_std_relative_performance
dynamic_sell_thresholds = -multiplier * rolling_std_relative_performance

signals_dynamic = pd.DataFrame(index=relative_performance.index)

for sector in relative_performance.columns:
    signals_dynamic[sector] = 'Hold'  # Initialize all to 'Hold'
    # Apply 'Buy' signal where relative performance is above the dynamic buy threshold
    buy_condition = relative_performance[sector] > dynamic_buy_thresholds[sector]
    signals_dynamic.loc[buy_condition, sector] = 'Buy'

    # Apply 'Sell' signal where relative performance is below the dynamic sell threshold
    sell_condition = relative_performance[sector] < dynamic_sell_thresholds[sector]
    signals_dynamic.loc[sell_condition, sector] = 'Sell'

print("Dynamic signals DataFrame regenerated successfully!")
print("First 5 rows of signals_dynamic:")
display(signals_dynamic.head())

# Re-run the backtesting simulation for the dynamic-threshold strategy without costs
portfolio_returns_dynamic = pd.DataFrame(index=signals_dynamic.index, columns=['Portfolio Return'])

sector_etfs = [col for col in df_sector_pct_change.columns if col != "S&P 500"]
num_tradeable_sectors = len(sector_etfs)

returns_aligned_dynamic = df_sector_pct_change[sector_etfs].align(signals_dynamic, join='inner', axis=0)[0]
signals_aligned_dynamic = signals_dynamic.align(returns_aligned_dynamic, join='inner', axis=0)[0]

valid_indices = signals_aligned_dynamic.dropna(how='all').index
signals_aligned_dynamic = signals_aligned_dynamic.loc[valid_indices]
returns_aligned_dynamic = returns_aligned_dynamic.loc[valid_indices]
portfolio_returns_dynamic = portfolio_returns_dynamic.loc[valid_indices]

current_portfolio_weights_at_start_of_period_dynamic = {sector: 1.0 / num_tradeable_sectors for sector in sector_etfs}

historical_weights_dynamic = pd.DataFrame(columns=sector_etfs, index=signals_aligned_dynamic.index)

for i in range(len(signals_aligned_dynamic)):
    current_date = signals_aligned_dynamic.index[i]
    current_signals = signals_aligned_dynamic.loc[current_date]
    current_week_returns = returns_aligned_dynamic.loc[current_date]

    weights_for_this_period_dynamic = {sector: 0.0 for sector in sector_etfs}

    buy_sectors = [s for s in sector_etfs if current_signals[s] == 'Buy']
    sell_sectors = [s for s in sector_etfs if current_signals[s] == 'Sell']
    hold_sectors = [s for s in sector_etfs if current_signals[s] == 'Hold']

    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_dynamic[sector] = weight_per_active_sector

    historical_weights_dynamic.loc[current_date] = list(weights_for_this_period_dynamic.values())

    weekly_port_return = sum(weights_for_this_period_dynamic[s] * current_week_returns[s] for s in sector_etfs)
    portfolio_returns_dynamic.loc[current_date, 'Portfolio Return'] = weekly_port_return

    current_portfolio_weights_at_start_of_period_dynamic = weights_for_this_period_dynamic

portfolio_returns_dynamic['Cumulative Return'] = (1 + portfolio_returns_dynamic['Portfolio Return']).cumprod() - 1

print("Dynamic portfolio simulation (without costs) complete.")
print("First 5 rows of portfolio_returns_dynamic:")
display(portfolio_returns_dynamic.head())

# Re-run the backtesting simulation for the dynamic-threshold strategy with transaction costs
portfolio_returns_dynamic_with_costs = pd.DataFrame(index=signals_aligned_dynamic.index, columns=['Portfolio Return'])

transaction_cost_rate = 0.001  # 0.1% transaction cost

previous_period_weights = {sector: 1.0 / num_tradeable_sectors for sector in sector_etfs}

for i in range(len(signals_aligned_dynamic)):
    current_date = signals_aligned_dynamic.index[i]
    current_signals = signals_aligned_dynamic.loc[current_date] # Signals for the current period
    current_week_returns = returns_aligned_dynamic.loc[current_date] # Returns for the current period

    # Determine the weights_for_this_period_dynamic
    weights_for_this_period_dynamic = {sector: 0.0 for sector in sector_etfs}
    buy_sectors = [s for s in sector_etfs if current_signals[s] == 'Buy']
    hold_sectors = [s for s in sector_etfs if current_signals[s] == 'Hold']
    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_dynamic[sector] = weight_per_active_sector

    # Calculate transaction_costs for this period
    transaction_costs = 0.0
    for sector in sector_etfs:
        weight_change = abs(weights_for_this_period_dynamic[sector] - previous_period_weights[sector])
        transaction_costs += weight_change * transaction_cost_rate

    # Calculate weekly_port_return_before_costs
    weekly_port_return_before_costs = sum(weights_for_this_period_dynamic[s] * current_week_returns[s] for s in sector_etfs)

    # Deduct transaction_costs
    weekly_port_return_after_costs = weekly_port_return_before_costs - transaction_costs
    portfolio_returns_dynamic_with_costs.loc[current_date, 'Portfolio Return'] = weekly_port_return_after_costs

    # Update previous_period_weights for the next iteration
    previous_period_weights = weights_for_this_period_dynamic

portfolio_returns_dynamic_with_costs['Cumulative Return'] = (1 + portfolio_returns_dynamic_with_costs['Portfolio Return']).cumprod() - 1

print("Dynamic portfolio simulation with transaction costs complete.")
print("First 5 rows of portfolio_returns_dynamic_with_costs:")
display(portfolio_returns_dynamic_with_costs.head())

In [ ]:
# 119.0

import pandas as pd
import numpy as np

# 1. Resample df_sector to a monthly frequency and calculate monthly percent changes
df_sector_monthly = df_sector.resample('M').last()
df_sector_monthly_pct_change = df_sector_monthly.pct_change().dropna()

# 2. Compute the 3-month rolling mean of these monthly percent changes
rolling_mean_monthly_pct_change = df_sector_monthly_pct_change.rolling(window=3).mean().dropna()

print("Monthly data re-preparation complete!")
print("First 5 rows of df_sector_monthly:")
display(df_sector_monthly.head())
print("\nFirst 5 rows of df_sector_monthly_pct_change:")
display(df_sector_monthly_pct_change.head())
print("\nFirst 5 rows of rolling_mean_monthly_pct_change:")
display(rolling_mean_monthly_pct_change.head())

# 3. Calculate monthly relative performance
relative_performance_monthly = pd.DataFrame(index=rolling_mean_monthly_pct_change.index)
sector_etfs_monthly = [col for col in rolling_mean_monthly_pct_change.columns if col != "S&P 500"]

for sector in sector_etfs_monthly:
    relative_performance_monthly[sector] = rolling_mean_monthly_pct_change[sector] - rolling_mean_monthly_pct_change["S&P 500"]

print("\nMonthly relative performance DataFrame re-created successfully!")
print("First 5 rows of relative_performance_monthly:")
display(relative_performance_monthly.head())

# 4. Define monthly 'buy', 'sell', or 'hold' signals
buy_threshold_monthly = 0.001
sell_threshold_monthly = -0.001

signals_monthly = pd.DataFrame(index=relative_performance_monthly.index)

for sector in relative_performance_monthly.columns:
    signals_monthly[sector] = 'Hold'  # Initialize all to 'Hold'
    signals_monthly.loc[relative_performance_monthly[sector] > buy_threshold_monthly, sector] = 'Buy'
    signals_monthly.loc[relative_performance_monthly[sector] < sell_threshold_monthly, sector] = 'Sell'

print("\nMonthly signals DataFrame re-created successfully!")
print("First 5 rows of signals_monthly:")
display(signals_monthly.head())

# 5. Backtest this monthly trading strategy
portfolio_returns_monthly = pd.DataFrame(index=signals_monthly.index, columns=['Portfolio Return'])

num_tradeable_sectors_monthly = len(sector_etfs_monthly)

returns_aligned_monthly = df_sector_monthly_pct_change[sector_etfs_monthly].align(signals_monthly, join='inner', axis=0)[0]
signals_aligned_monthly = signals_monthly.align(returns_aligned_monthly, join='inner', axis=0)[0]

current_portfolio_weights_at_start_of_period_monthly = {sector: 1.0 / num_tradeable_sectors_monthly for sector in sector_etfs_monthly}

historical_weights_monthly = pd.DataFrame(columns=sector_etfs_monthly, index=signals_aligned_monthly.index)

for i in range(len(signals_aligned_monthly)):
    current_date = signals_aligned_monthly.index[i]
    current_signals = signals_aligned_monthly.loc[current_date]
    current_month_returns = returns_aligned_monthly.loc[current_date]

    weights_for_this_period_monthly = {sector: 0.0 for sector in sector_etfs_monthly}

    buy_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Buy']
    sell_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Sell']
    hold_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Hold']

    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_monthly[sector] = weight_per_active_sector
    else:
        pass # weights_for_this_period_monthly is already initialized to 0.0

    historical_weights_monthly.loc[current_date] = list(weights_for_this_period_monthly.values())

    monthly_port_return = sum(weights_for_this_period_monthly[s] * current_month_returns[s] for s in sector_etfs_monthly)
    portfolio_returns_monthly.loc[current_date, 'Portfolio Return'] = monthly_port_return

    current_portfolio_weights_at_start_of_period_monthly = weights_for_this_period_monthly

# 4. Calculate the cumulative returns of the portfolio_returns_monthly
# (1 + Return) cumprod() - 1 gives the cumulative return based on compounding.
portfolio_returns_monthly['Cumulative Return'] = (1 + portfolio_returns_monthly['Portfolio Return']).cumprod() - 1

print("\nMonthly portfolio simulation re-complete.")
print("First 5 rows of portfolio_returns_monthly:")
display(portfolio_returns_monthly.head())
print("\nLast 5 rows of portfolio_returns_monthly:")
display(portfolio_returns_monthly.tail())

In [ ]:
# 120.0

rf = 0.01 # Re-defining risk-free rate, as it was not defined in this execution context

num_months_monthly = len(portfolio_returns_monthly)
num_years_monthly = num_months_monthly / 12.0

# Monthly Strategy Metrics
portfolio_total_cumulative_return_monthly = portfolio_returns_monthly['Cumulative Return'].iloc[-1]
portfolio_annualized_return_monthly = (1 + portfolio_total_cumulative_return_monthly)**(1/num_years_monthly) - 1
# Monthly volatility, then annualize (sqrt(12) for monthly to annual)
portfolio_annualized_volatility_monthly = portfolio_returns_monthly['Portfolio Return'].std() * np.sqrt(12)
portfolio_sharpe_ratio_monthly = (portfolio_annualized_return_monthly - rf) / portfolio_annualized_volatility_monthly

# --- Recalculate metrics for Fixed Strategy --- (using already calculated portfolio_returns)
num_weeks = len(portfolio_returns)
num_years = num_weeks / 52.0
portfolio_total_cumulative_return = portfolio_returns['Cumulative Return'].iloc[-1]
portfolio_annualized_return = (1 + portfolio_total_cumulative_return)**(1/num_years) - 1
portfolio_annualized_volatility = portfolio_returns['Portfolio Return'].std() * np.sqrt(52)
portfolio_sharpe_ratio = (portfolio_annualized_return - rf) / portfolio_annualized_volatility

# Recalculate S&P 500 metrics for consistency with the fixed strategy's time frame
s_p_500_returns_aligned = df_sector_pct_change["S&P 500"].align(portfolio_returns, join='inner', axis=0)[0]
s_p_500_cumulative_return = (1 + s_p_500_returns_aligned).cumprod() - 1
s_p_500_total_cumulative_return = s_p_500_cumulative_return.iloc[-1]
s_p_500_annualized_return = (1 + s_p_500_total_cumulative_return)**(1/num_years) - 1
s_p_500_annualized_volatility = s_p_500_returns_aligned.std() * np.sqrt(52)
s_p_500_sharpe_ratio = (s_p_500_annualized_return - rf) / s_p_500_annualized_volatility

# --- Recalculate metrics for Dynamic Strategy --- (using already calculated portfolio_returns_dynamic)
num_weeks_dynamic = len(portfolio_returns_dynamic)
num_years_dynamic = num_weeks_dynamic / 52.0
portfolio_total_cumulative_return_dynamic = portfolio_returns_dynamic['Cumulative Return'].iloc[-1]
portfolio_annualized_return_dynamic = (1 + portfolio_total_cumulative_return_dynamic)**(1/num_years_dynamic) - 1
portfolio_annualized_volatility_dynamic = portfolio_returns_dynamic['Portfolio Return'].std() * np.sqrt(52)
portfolio_sharpe_ratio_dynamic = (portfolio_annualized_return_dynamic - rf) / portfolio_annualized_volatility_dynamic

# --- Recalculate metrics for Dynamic Strategy with Costs --- (using already calculated portfolio_returns_dynamic_with_costs)
num_weeks_dynamic_costs = len(portfolio_returns_dynamic_with_costs)
num_years_dynamic_costs = num_weeks_dynamic_costs / 52.0
portfolio_total_cumulative_return_dynamic_costs = portfolio_returns_dynamic_with_costs['Cumulative Return'].iloc[-1]
portfolio_annualized_return_dynamic_costs = (1 + portfolio_total_cumulative_return_dynamic_costs)**(1/num_years_dynamic_costs) - 1
portfolio_annualized_volatility_dynamic_costs = portfolio_returns_dynamic_with_costs['Portfolio Return'].std() * np.sqrt(52)
portfolio_sharpe_ratio_dynamic_costs = (portfolio_annualized_return_dynamic_costs - rf) / portfolio_annualized_volatility_dynamic_costs

# Print Summary Comparison
print("\n--- Strategy Performance Comparison (Monthly vs. Dynamic Costs vs. Dynamic vs. Fixed vs. S&P 500) ---")
print(f"Metric                        | Monthly         | Dynamic Costs   | Dynamic         | Fixed Threshold | S&P 500       ")
print(f"------------------------------------------------------------------------------------------------------------------")
print(f"Total Cumulative Return       | {portfolio_total_cumulative_return_monthly:.4f}        | {portfolio_total_cumulative_return_dynamic_costs:.4f}        | {portfolio_total_cumulative_return_dynamic:.4f}         | {portfolio_total_cumulative_return:.4f}        | {s_p_500_total_cumulative_return:.4f}       ")
print(f"Annualized Return             | {portfolio_annualized_return_monthly:.4f}        | {portfolio_annualized_return_dynamic_costs:.4f}        | {portfolio_annualized_return_dynamic:.4f}         | {portfolio_annualized_return:.4f}        | {s_p_500_annualized_return:.4f}       ")
print(f"Annualized Volatility         | {portfolio_annualized_volatility_monthly:.4f}        | {portfolio_annualized_volatility_dynamic_costs:.4f}        | {portfolio_annualized_volatility_dynamic:.4f}         | {portfolio_annualized_volatility:.4f}        | {s_p_500_annualized_volatility:.4f}       ")
print(f"Sharpe Ratio                  | {portfolio_sharpe_ratio_monthly:.4f}        | {portfolio_sharpe_ratio_dynamic_costs:.4f}        | {portfolio_sharpe_ratio_dynamic:.4f}         | {portfolio_sharpe_ratio:.4f}        | {s_p_500_sharpe_ratio:.4f}       ")
print("------------------------------------------------------------------------------------------------------------------")

In [ ]:
# 121.0

import matplotlib.pyplot as plt

plt.figure(figsize=(14, 7))
plt.plot(portfolio_returns_monthly.index, portfolio_returns_monthly['Cumulative Return'], label='Monthly Strategy Cumulative Return', color='orange')
plt.plot(portfolio_returns.index, portfolio_returns['Cumulative Return'], label='Fixed Strategy Cumulative Return', color='blue')
plt.plot(portfolio_returns_dynamic.index, portfolio_returns_dynamic['Cumulative Return'], label='Dynamic Strategy Cumulative Return (no costs)', color='green')
plt.plot(portfolio_returns_dynamic_with_costs.index, portfolio_returns_dynamic_with_costs['Cumulative Return'], label='Dynamic Strategy Cumulative Return (with costs)', color='purple')
plt.plot(s_p_500_cumulative_return.index, s_p_500_cumulative_return, label='S&P 500 Cumulative Return', color='red')
plt.title('Cumulative Returns: All Strategies vs. S&P 500 (Including XLRE)')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend()
plt.show()

20251230 15:21 RERUN MODEL WITH START DATE 20100101 - END DATE 20251230

# Task
## Import Libraries and Fetch Sector Data (Updated Dates)

### Subtask:
Re-execute the code cells to import necessary libraries (pandas, yfinance, fredapi) and fetch the sector ETF data, including 'XLRE', saving it to 'Sector_ETF_Data.csv'. Use '2010-01-01' as the start date and '2025-12-30' as the end date.

### Reasoning:
I need to update the `start_date` to '2010-01-01' and `end_date` to '2025-12-30' in the code block that fetches sector data. This will ensure all subsequent calculations are performed on the desired, expanded date range. I will also make sure `yfinance`, `pandas`, and `fredapi` are imported.

```python
import pandas as pd
import yfinance as yf
import fredapi # Import fredapi, though not directly used in this specific cell, it's a necessary library from context.

tickers = ["XLK", "XLC", "XLV", "XLF", "XLI", "XLB", "XLU", "XLE", "XLY", "XLP", "SPY", "XLRE"]

# Mapping from tickers to desired sector names
ticker_to_sector = {
    "XLK": "Technology",
    "XLC": "Communications",
    "XLV": "Healthcare",
    "XLF": "Financial",
    "XLI": "Industrial",
    "XLB": "Material",
    "XLU": "Utilities",
    "XLE": "Energy",
    "XLY": "CDiscretionary",
    "XLP": "CStaples",
    "SPY": "S&P 500",
    "XLRE": "Real Estate"
}

# Set the updated date range
start_date = "2010-01-01"
end_date = "2025-12-30"

# Initialize an empty dictionary to store ETF data
sector_data = {}

# Download historical data for each ETF, explicitly setting auto_adjust=True.
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date, interval="1wk", auto_adjust=True, progress=False)
    if not data.empty:
        sector_data[ticker] = data["Close"]

# If valid data exists, combine it into a DataFrame and flatten the headers.
if sector_data:
    # Concatenate the Series objects; using keys creates a MultiIndex header.
    df_sector = pd.concat(sector_data.values(), axis=1, keys=sector_data.keys())

    # Flatten the MultiIndex columns by extracting the first level (tickers) and mapping to sector names.
    df_sector.columns = [ticker_to_sector.get(ticker, ticker) for ticker in df_sector.columns.get_level_values(0)]

    # Save the DataFrame to CSV
    df_sector.to_csv("Sector_ETF_Data.csv")
    print("Sector ETF data saved successfully with updated dates!")
else:
    print("No valid data retrieved for any tickers.")
```

## Import Libraries and Fetch Sector Data (Updated Dates)

### Subtask:
Re-execute the code cells to import necessary libraries (pandas, yfinance, fredapi) and fetch the sector ETF data, including 'XLRE', saving it to 'Sector_ETF_Data.csv'. Use '2010-01-01' as the start date and '2025-12-30' as the end date.


**Reasoning**:
The user explicitly requested to re-execute the code cells to import necessary libraries, fetch sector ETF data including 'XLRE', and save it to 'Sector_ETF_Data.csv' with updated start and end dates ('2010-01-01' to '2025-12-30'). This code block will perform all these actions.



In [ ]:
# 122.0

import pandas as pd
import yfinance as yf
import fredapi # Import fredapi, though not directly used in this specific cell, it's a necessary library from context.

tickers = ["XLK", "XLC", "XLV", "XLF", "XLI", "XLB", "XLU", "XLE", "XLY", "XLP", "SPY", "XLRE"]

# Mapping from tickers to desired sector names
ticker_to_sector = {
    "XLK": "Technology",
    "XLC": "Communications",
    "XLV": "Healthcare",
    "XLF": "Financial",
    "XLI": "Industrial",
    "XLB": "Material",
    "XLU": "Utilities",
    "XLE": "Energy",
    "XLY": "CDiscretionary",
    "XLP": "CStaples",
    "SPY": "S&P 500",
    "XLRE": "Real Estate"
}

# Set the date range
start_date = "2010-01-01"
end_date = "2025-12-30"

# Initialize an empty dictionary to store ETF data
sector_data = {}

# Download historical data for each ETF, explicitly setting auto_adjust=True.
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date, interval="1wk", auto_adjust=True, progress=False)
    if not data.empty:
        sector_data[ticker] = data["Close"]

# If valid data exists, combine it into a DataFrame and flatten the headers.
if sector_data:
    # Concatenate the Series objects; using keys creates a MultiIndex header.
    df_sector = pd.concat(sector_data.values(), axis=1, keys=sector_data.keys())

    # Flatten the MultiIndex columns by extracting the first level (tickers) and mapping to sector names.
    df_sector.columns = [ticker_to_sector.get(ticker, ticker) for ticker in df_sector.columns.get_level_values(0)]

    # Save the DataFrame to CSV
    df_sector.to_csv("Sector_ETF_Data.csv")
    print("Sector ETF data saved successfully!")
else:
    print("No valid data retrieved for any tickers.")

**Reasoning**:
Now that the sector ETF data has been updated and saved to 'Sector_ETF_Data.csv' with the new date range and 'XLRE' included, the next step is to re-load this data into `df_sector` and apply forward/backward fill to handle any potential missing values, as per the standard preprocessing steps in the notebook.



In [ ]:
# 123.0

import pandas as pd

df_sector = pd.read_csv("Sector_ETF_Data.csv", index_col="Date", parse_dates=True)

# Correct for missing values using ffill() and then bfill()
df_sector = df_sector.ffill()
df_sector = df_sector.bfill()

print("Missing values after correction:")
display(df_sector.isnull().sum())

print("DataFrame head after missing value correction:")
display(df_sector.head())

**Reasoning**:
The sector data has been updated and cleaned. Now, I need to fetch the macroeconomic data from FRED for the updated date range ('2010-01-01' to '2025-12-30') and save it to 'Macroeconomic_Data.csv'. The existing code block in cell `9ac553b5` already contains the logic to perform this task.



In [ ]:
# 124.0

import pandas as pd
from fredapi import Fred

# FRED API key - Ensure this is correctly set or retrieved from a secure source.
# For demonstration, using the API key from the provided notebook context.
fred = Fred(api_key='e8fb9c414f6ab8f1a06a5472466a31dc')

# Define date range for weekly index, consistent with previous data fetches
start_date = "2010-01-01"
end_date = "2025-12-30"
weekly_index = pd.date_range(start=start_date, end=end_date, freq='W-FRI').strftime('%Y-%m-%d')

# Define FRED series IDs and their corresponding labels
series_dict = {
    'GDP': 'GDP',
    'Inflation (CPI)': 'CPIAUCSL',
    'Unemployment Rate': 'UNRATE',
    'Interest Rate (Fed Funds)': 'FEDFUNDS'
}

# Create a DataFrame with weekly index to store macroeconomic data
macro_data = pd.DataFrame(index=pd.to_datetime(weekly_index))
macro_data.index.name = "Date"

# Fetch and interpolate data for each series
for label, series_id in series_dict.items():
    data = fred.get_series(series_id)
    if data is None: # Handle cases where FRED API might return None
        print(f"Warning: Could not fetch series {series_id} for {label}")
        continue

    data = data.to_frame(name=label)
    data.index = pd.to_datetime(data.index)

    # Resample to daily, interpolating or forward-filling as specified
    if label == 'Interest Rate (Fed Funds)':
        daily = data.resample('D').ffill()
    else:
        # For other series, interpolate missing values to daily frequency
        daily = data.resample('D').interpolate(method='linear')

    # Reindex the daily data to the predefined weekly index and interpolate remaining missing values
    # Use limit_direction='both' for interpolation to handle NaNs at ends of series
    weekly_processed = daily.reindex(macro_data.index).interpolate(method='linear', limit_direction='both')
    macro_data[label] = weekly_processed[label]

# Save the completed macroeconomic DataFrame to a CSV file
macro_data.to_csv("Macroeconomic_Data.csv", index=True)

print("Macroeconomic Data fetched, processed, and saved successfully to 'Macroeconomic_Data.csv'!")

**Reasoning**:
I need to calculate the weekly percentage change for `df_sector`, compute the 12-week rolling mean of these changes, and then calculate the `relative_performance` by subtracting the S&P 500's rolling mean from each sector's rolling mean, which will now incorporate the newly added 'Real Estate' sector and the updated date range. This aligns with the previous subtask's description and the overall flow of the notebook.



In [ ]:
# 125.0

df_sector_pct_change = df_sector.pct_change().dropna()

rolling_mean_pct_change = df_sector_pct_change.rolling(window=12).mean().dropna()

relative_performance = pd.DataFrame(index=rolling_mean_pct_change.index)

sector_etfs = [col for col in rolling_mean_pct_change.columns if col != "S&P 500"]

for sector in sector_etfs:
    relative_performance[sector] = rolling_mean_pct_change[sector] - rolling_mean_pct_change["S&P 500"]

print("df_sector_pct_change, rolling_mean_pct_change, and relative_performance recalculated successfully!")
print("First 5 rows of relative_performance:")
display(relative_performance.head())

**Reasoning**:
I need to regenerate the 'signals' DataFrame using fixed thresholds and then re-run the backtest for the fixed-threshold strategy, following the detailed instructions provided in the subtask. The provided notebook cells (`2d8d9c35`) already contain this logic.



In [ ]:
# 126.0

buy_threshold = 0.001
sell_threshold = -0.001

signals = pd.DataFrame(index=relative_performance.index)

for sector in relative_performance.columns:
    signals[sector] = 'Hold'  # Initialize all to 'Hold'
    signals.loc[relative_performance[sector] > buy_threshold, sector] = 'Buy'
    signals.loc[relative_performance[sector] < sell_threshold, sector] = 'Sell'

print("Fixed-threshold signals DataFrame regenerated successfully!")
print("First 5 rows of signals:")
display(signals.head())

# Re-run the backtesting simulation for the fixed-threshold strategy
# 1. Initialize an empty DataFrame called portfolio_returns
portfolio_returns = pd.DataFrame(index=signals.index, columns=['Portfolio Return'])

# Get list of sector ETFs (excluding S&P 500)
sector_etfs = [col for col in df_sector_pct_change.columns if col != "S&P 500"]
num_tradeable_sectors = len(sector_etfs)

# Align returns and signals to ensure they cover the same time period
# This ensures both DataFrames start and end on the same dates.
returns_aligned = df_sector_pct_change[sector_etfs].align(signals, join='inner', axis=0)[0]
signals_aligned = signals.align(returns_aligned, join='inner', axis=0)[0]

# 2. Initialize current_portfolio_weights_at_start_of_period
# These are the weights assumed at the start of the *first* period for which we have signals.
current_portfolio_weights_at_start_of_period = {sector: 1.0 / num_tradeable_sectors for sector in sector_etfs}

# Create a DataFrame to store historical weights for analysis (optional, but good for inspection)
historical_weights = pd.DataFrame(columns=sector_etfs, index=signals_aligned.index)

# 3. Iterate through each time step (row) in the aligned signals DataFrame
for i in range(len(signals_aligned)):
    current_date = signals_aligned.index[i]
    current_signals = signals_aligned.loc[current_date] # Signals for the current period
    current_week_returns = returns_aligned.loc[current_date] # Returns for the current period

    # `weights_for_this_period` will hold the actual weights used for the current period's return calculation.
    # Initialize all to 0.0, and then assign based on signals.
    weights_for_this_period = {sector: 0.0 for sector in sector_etfs}

    buy_sectors = [s for s in sector_etfs if current_signals[s] == 'Buy']
    sell_sectors = [s for s in sector_etfs if current_signals[s] == 'Sell']
    hold_sectors = [s for s in sector_etfs if current_signals[s] == 'Hold']

    # Determine the 'active' sectors for this period (those not getting a 'Sell' signal).
    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period[sector] = weight_per_active_sector
        # Sectors not in active_sectors_this_period (i.e., sell_sectors) implicitly retain 0.0 weight.
    else:
        # If all sectors are 'Sell', or no active signals, all weights remain 0.0.
        # Portfolio return for this week will be 0.0.
        pass # weights_for_this_period is already initialized to 0.0

    # Store the weights used for this period calculation in the historical_weights DataFrame.
    historical_weights.loc[current_date] = list(weights_for_this_period.values())

    # Calculate portfolio return for the current week using the determined weights.
    weekly_port_return = sum(weights_for_this_period[s] * current_week_returns[s] for s in sector_etfs)
    portfolio_returns.loc[current_date, 'Portfolio Return'] = weekly_port_return

    # Update `current_portfolio_weights_at_start_of_period` for the *next* iteration.
    # This reflects the portfolio composition at the end of the current period, which is the start of the next.
    current_portfolio_weights_at_start_of_period = weights_for_this_period

# 4. Calculate the cumulative returns of the portfolio_returns
# (1 + Return) cumprod() - 1 gives the cumulative return based on compounding.
portfolio_returns['Cumulative Return'] = (1 + portfolio_returns['Portfolio Return']).cumprod() - 1

print("Fixed-threshold backtesting simulation complete.")
print("First 5 rows of portfolio_returns:")
display(portfolio_returns.head())

**Reasoning**:
I need to regenerate the backtest for the dynamic-threshold strategy, this time incorporating transaction costs, as per the instructions. This involves re-calculating the signals, then simulating the portfolio's returns while deducting the specified transaction cost for each rebalancing event.



In [ ]:
# 127.0

rolling_std_relative_performance = relative_performance.rolling(window=12).std()

multiplier = 0.5 # Adjust this multiplier as needed

dynamic_buy_thresholds = multiplier * rolling_std_relative_performance
dynamic_sell_thresholds = -multiplier * rolling_std_relative_performance

signals_dynamic = pd.DataFrame(index=relative_performance.index)

for sector in relative_performance.columns:
    signals_dynamic[sector] = 'Hold'  # Initialize all to 'Hold'
    # Apply 'Buy' signal where relative performance is above the dynamic buy threshold
    buy_condition = relative_performance[sector] > dynamic_buy_thresholds[sector]
    signals_dynamic.loc[buy_condition, sector] = 'Buy'

    # Apply 'Sell' signal where relative performance is below the dynamic sell threshold
    sell_condition = relative_performance[sector] < dynamic_sell_thresholds[sector]
    signals_dynamic.loc[sell_condition, sector] = 'Sell'

print("Dynamic signals DataFrame regenerated successfully!")
print("First 5 rows of signals_dynamic:")
display(signals_dynamic.head())

# Re-run the backtesting simulation for the dynamic-threshold strategy without costs
portfolio_returns_dynamic = pd.DataFrame(index=signals_dynamic.index, columns=['Portfolio Return'])

sector_etfs = [col for col in df_sector_pct_change.columns if col != "S&P 500"]
num_tradeable_sectors = len(sector_etfs)

returns_aligned_dynamic = df_sector_pct_change[sector_etfs].align(signals_dynamic, join='inner', axis=0)[0]
signals_aligned_dynamic = signals_dynamic.align(returns_aligned_dynamic, join='inner', axis=0)[0]

valid_indices = signals_aligned_dynamic.dropna(how='all').index
signals_aligned_dynamic = signals_aligned_dynamic.loc[valid_indices]
returns_aligned_dynamic = returns_aligned_dynamic.loc[valid_indices]
portfolio_returns_dynamic = portfolio_returns_dynamic.loc[valid_indices]

current_portfolio_weights_at_start_of_period_dynamic = {sector: 1.0 / num_tradeable_sectors for sector in sector_etfs}

historical_weights_dynamic = pd.DataFrame(columns=sector_etfs, index=signals_aligned_dynamic.index)

for i in range(len(signals_aligned_dynamic)):
    current_date = signals_aligned_dynamic.index[i]
    current_signals = signals_aligned_dynamic.loc[current_date]
    current_week_returns = returns_aligned_dynamic.loc[current_date]

    weights_for_this_period_dynamic = {sector: 0.0 for sector in sector_etfs}

    buy_sectors = [s for s in sector_etfs if current_signals[s] == 'Buy']
    sell_sectors = [s for s in sector_etfs if current_signals[s] == 'Sell']
    hold_sectors = [s for s in sector_etfs if current_signals[s] == 'Hold']

    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_dynamic[sector] = weight_per_active_sector

    historical_weights_dynamic.loc[current_date] = list(weights_for_this_period_dynamic.values())

    weekly_port_return = sum(weights_for_this_period_dynamic[s] * current_week_returns[s] for s in sector_etfs)
    portfolio_returns_dynamic.loc[current_date, 'Portfolio Return'] = weekly_port_return

    current_portfolio_weights_at_start_of_period_dynamic = weights_for_this_period_dynamic

portfolio_returns_dynamic['Cumulative Return'] = (1 + portfolio_returns_dynamic['Portfolio Return']).cumprod() - 1

print("Dynamic portfolio simulation (without costs) complete.")
print("First 5 rows of portfolio_returns_dynamic:")
display(portfolio_returns_dynamic.head())

# Re-run the backtesting simulation for the dynamic-threshold strategy with transaction costs
portfolio_returns_dynamic_with_costs = pd.DataFrame(index=signals_aligned_dynamic.index, columns=['Portfolio Return'])

transaction_cost_rate = 0.001  # 0.1% transaction cost

previous_period_weights = {sector: 1.0 / num_tradeable_sectors for sector in sector_etfs}

for i in range(len(signals_aligned_dynamic)):
    current_date = signals_aligned_dynamic.index[i]
    current_signals = signals_aligned_dynamic.loc[current_date] # Signals for the current period
    current_week_returns = returns_aligned_dynamic.loc[current_date] # Returns for the current period

    # Determine the weights_for_this_period_dynamic
    weights_for_this_period_dynamic = {sector: 0.0 for sector in sector_etfs}
    buy_sectors = [s for s in sector_etfs if current_signals[s] == 'Buy']
    hold_sectors = [s for s in sector_etfs if current_signals[s] == 'Hold']
    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_dynamic[sector] = weight_per_active_sector

    # Calculate transaction_costs for this period
    transaction_costs = 0.0
    for sector in sector_etfs:
        weight_change = abs(weights_for_this_period_dynamic[sector] - previous_period_weights[sector])
        transaction_costs += weight_change * transaction_cost_rate

    # Calculate weekly_port_return_before_costs
    weekly_port_return_before_costs = sum(weights_for_this_period_dynamic[s] * current_week_returns[s] for s in sector_etfs)

    # Deduct transaction_costs
    weekly_port_return_after_costs = weekly_port_return_before_costs - transaction_costs
    portfolio_returns_dynamic_with_costs.loc[current_date, 'Portfolio Return'] = weekly_port_return_after_costs

    # Update previous_period_weights for the next iteration
    previous_period_weights = weights_for_this_period_dynamic

portfolio_returns_dynamic_with_costs['Cumulative Return'] = (1 + portfolio_returns_dynamic_with_costs['Portfolio Return']).cumprod() - 1

print("Dynamic portfolio simulation with transaction costs complete.")
print("First 5 rows of portfolio_returns_dynamic_with_costs:")
display(portfolio_returns_dynamic_with_costs.head())

**Reasoning**:
I need to re-prepare the monthly data, recalculate monthly relative performance and signals, and then re-run the backtest for the monthly strategy, incorporating the new XLRE sector. This comprehensive code block will ensure all necessary DataFrames are updated and the `portfolio_returns_monthly` is correctly computed.



In [ ]:
# 128.0

import pandas as pd
import numpy as np

# 1. Resample df_sector to a monthly frequency and calculate monthly percent changes
df_sector_monthly = df_sector.resample('M').last()
df_sector_monthly_pct_change = df_sector_monthly.pct_change().dropna()

# 2. Compute the 3-month rolling mean of these monthly percent changes
rolling_mean_monthly_pct_change = df_sector_monthly_pct_change.rolling(window=3).mean().dropna()

print("Monthly data re-preparation complete!")
print("First 5 rows of df_sector_monthly:")
display(df_sector_monthly.head())
print("\nFirst 5 rows of df_sector_monthly_pct_change:")
display(df_sector_monthly_pct_change.head())
print("\nFirst 5 rows of rolling_mean_monthly_pct_change:")
display(rolling_mean_monthly_pct_change.head())

# 3. Calculate monthly relative performance
relative_performance_monthly = pd.DataFrame(index=rolling_mean_monthly_pct_change.index)
sector_etfs_monthly = [col for col in rolling_mean_monthly_pct_change.columns if col != "S&P 500"]

for sector in sector_etfs_monthly:
    relative_performance_monthly[sector] = rolling_mean_monthly_pct_change[sector] - rolling_mean_monthly_pct_change["S&P 500"]

print("\nMonthly relative performance DataFrame re-created successfully!")
print("First 5 rows of relative_performance_monthly:")
display(relative_performance_monthly.head())

# 4. Define monthly 'buy', 'sell', or 'hold' signals
buy_threshold_monthly = 0.001
sell_threshold_monthly = -0.001

signals_monthly = pd.DataFrame(index=relative_performance_monthly.index)

for sector in relative_performance_monthly.columns:
    signals_monthly[sector] = 'Hold'  # Initialize all to 'Hold'
    signals_monthly.loc[relative_performance_monthly[sector] > buy_threshold_monthly, sector] = 'Buy'
    signals_monthly.loc[relative_performance_monthly[sector] < sell_threshold_monthly, sector] = 'Sell'

print("\nMonthly signals DataFrame re-created successfully!")
print("First 5 rows of signals_monthly:")
display(signals_monthly.head())

# 5. Backtest this monthly trading strategy
portfolio_returns_monthly = pd.DataFrame(index=signals_monthly.index, columns=['Portfolio Return'])

num_tradeable_sectors_monthly = len(sector_etfs_monthly)

returns_aligned_monthly = df_sector_monthly_pct_change[sector_etfs_monthly].align(signals_monthly, join='inner', axis=0)[0]
signals_aligned_monthly = signals_monthly.align(returns_aligned_monthly, join='inner', axis=0)[0]

current_portfolio_weights_at_start_of_period_monthly = {sector: 1.0 / num_tradeable_sectors_monthly for sector in sector_etfs_monthly}

historical_weights_monthly = pd.DataFrame(columns=sector_etfs_monthly, index=signals_aligned_monthly.index)

for i in range(len(signals_aligned_monthly)):
    current_date = signals_aligned_monthly.index[i]
    current_signals = signals_aligned_monthly.loc[current_date]
    current_month_returns = returns_aligned_monthly.loc[current_date]

    weights_for_this_period_monthly = {sector: 0.0 for sector in sector_etfs_monthly}

    buy_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Buy']
    sell_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Sell']
    hold_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Hold']

    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_monthly[sector] = weight_per_active_sector
    else:
        pass # weights_for_this_period_monthly is already initialized to 0.0

    historical_weights_monthly.loc[current_date] = list(weights_for_this_period_monthly.values())

    monthly_port_return = sum(weights_for_this_period_monthly[s] * current_month_returns[s] for s in sector_etfs_monthly)
    portfolio_returns_monthly.loc[current_date, 'Portfolio Return'] = monthly_port_return

    current_portfolio_weights_at_start_of_period_monthly = weights_for_this_period_monthly

# 4. Calculate the cumulative returns of the portfolio_returns_monthly
# (1 + Return) cumprod() - 1 gives the cumulative return based on compounding.
portfolio_returns_monthly['Cumulative Return'] = (1 + portfolio_returns_monthly['Portfolio Return']).cumprod() - 1

print("\nMonthly portfolio simulation re-complete.")
print("First 5 rows of portfolio_returns_monthly:")
display(portfolio_returns_monthly.head())
print("\nLast 5 rows of portfolio_returns_monthly:")
display(portfolio_returns_monthly.tail())

**Reasoning**:
The previous code block resulted in an IndentationError. I need to correct the indentation of several lines within the main loop of the backtesting simulation for the monthly strategy to ensure they are properly nested, specifically the lines responsible for storing historical weights, calculating monthly portfolio return, and updating portfolio weights.



In [ ]:
# 129.0

import pandas as pd
import numpy as np

# 1. Resample df_sector to a monthly frequency and calculate monthly percent changes
df_sector_monthly = df_sector.resample('M').last()
df_sector_monthly_pct_change = df_sector_monthly.pct_change().dropna()

# 2. Compute the 3-month rolling mean of these monthly percent changes
rolling_mean_monthly_pct_change = df_sector_monthly_pct_change.rolling(window=3).mean().dropna()

print("Monthly data re-preparation complete!")
print("First 5 rows of df_sector_monthly:")
display(df_sector_monthly.head())
print("\nFirst 5 rows of df_sector_monthly_pct_change:")
display(df_sector_monthly_pct_change.head())
print("\nFirst 5 rows of rolling_mean_monthly_pct_change:")
display(rolling_mean_monthly_pct_change.head())

# 3. Calculate monthly relative performance
relative_performance_monthly = pd.DataFrame(index=rolling_mean_monthly_pct_change.index)
sector_etfs_monthly = [col for col in rolling_mean_monthly_pct_change.columns if col != "S&P 500"]

for sector in sector_etfs_monthly:
    relative_performance_monthly[sector] = rolling_mean_monthly_pct_change[sector] - rolling_mean_monthly_pct_change["S&P 500"]

print("\nMonthly relative performance DataFrame re-created successfully!")
print("First 5 rows of relative_performance_monthly:")
display(relative_performance_monthly.head())

# 4. Define monthly 'buy', 'sell', or 'hold' signals
buy_threshold_monthly = 0.001
sell_threshold_monthly = -0.001

signals_monthly = pd.DataFrame(index=relative_performance_monthly.index)

for sector in relative_performance_monthly.columns:
    signals_monthly[sector] = 'Hold'  # Initialize all to 'Hold'
    signals_monthly.loc[relative_performance_monthly[sector] > buy_threshold_monthly, sector] = 'Buy'
    signals_monthly.loc[relative_performance_monthly[sector] < sell_threshold_monthly, sector] = 'Sell'

print("\nMonthly signals DataFrame re-created successfully!")
print("First 5 rows of signals_monthly:")
display(signals_monthly.head())

# 5. Backtest this monthly trading strategy
portfolio_returns_monthly = pd.DataFrame(index=signals_monthly.index, columns=['Portfolio Return'])

num_tradeable_sectors_monthly = len(sector_etfs_monthly)

returns_aligned_monthly = df_sector_monthly_pct_change[sector_etfs_monthly].align(signals_monthly, join='inner', axis=0)[0]
signals_aligned_monthly = signals_monthly.align(returns_aligned_monthly, join='inner', axis=0)[0]

current_portfolio_weights_at_start_of_period_monthly = {sector: 1.0 / num_tradeable_sectors_monthly for sector in sector_etfs_monthly}

historical_weights_monthly = pd.DataFrame(columns=sector_etfs_monthly, index=signals_aligned_monthly.index)

for i in range(len(signals_aligned_monthly)):
    current_date = signals_aligned_monthly.index[i]
    current_signals = signals_aligned_monthly.loc[current_date]
    current_month_returns = returns_aligned_monthly.loc[current_date]

    weights_for_this_period_monthly = {sector: 0.0 for sector in sector_etfs_monthly}

    buy_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Buy']
    sell_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Sell']
    hold_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Hold']

    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_monthly[sector] = weight_per_active_sector
    else:
        pass # weights_for_this_period_monthly is already initialized to 0.0

    historical_weights_monthly.loc[current_date] = list(weights_for_this_period_monthly.values())

    monthly_port_return = sum(weights_for_this_period_monthly[s] * current_month_returns[s] for s in sector_etfs_monthly)
    portfolio_returns_monthly.loc[current_date, 'Portfolio Return'] = monthly_port_return

    current_portfolio_weights_at_start_of_period_monthly = weights_for_this_period_monthly

# 4. Calculate the cumulative returns of the portfolio_returns_monthly
# (1 + Return) cumprod() - 1 gives the cumulative return based on compounding.
portfolio_returns_monthly['Cumulative Return'] = (1 + portfolio_returns_monthly['Portfolio Return']).cumprod() - 1

print("\nMonthly portfolio simulation re-complete.")
print("First 5 rows of portfolio_returns_monthly:")
display(portfolio_returns_monthly.head())
print("\nLast 5 rows of portfolio_returns_monthly:")
display(portfolio_returns_monthly.tail())

**Reasoning**:
The previous code block resulted in an IndentationError. I will regenerate the same code ensuring consistent 4-space indentation throughout the code block to resolve any potential hidden indentation issues or mix of tabs and spaces.



In [ ]:
# 130.0

import pandas as pd
import numpy as np

# 1. Resample df_sector to a monthly frequency and calculate monthly percent changes
df_sector_monthly = df_sector.resample('M').last()
df_sector_monthly_pct_change = df_sector_monthly.pct_change().dropna()

# 2. Compute the 3-month rolling mean of these monthly percent changes
rolling_mean_monthly_pct_change = df_sector_monthly_pct_change.rolling(window=3).mean().dropna()

print("Monthly data re-preparation complete!")
print("First 5 rows of df_sector_monthly:")
display(df_sector_monthly.head())
print("\nFirst 5 rows of df_sector_monthly_pct_change:")
display(df_sector_monthly_pct_change.head())
print("\nFirst 5 rows of rolling_mean_monthly_pct_change:")
display(rolling_mean_monthly_pct_change.head())

# 3. Calculate monthly relative performance
relative_performance_monthly = pd.DataFrame(index=rolling_mean_monthly_pct_change.index)
sector_etfs_monthly = [col for col in rolling_mean_monthly_pct_change.columns if col != "S&P 500"]

for sector in sector_etfs_monthly:
    relative_performance_monthly[sector] = rolling_mean_monthly_pct_change[sector] - rolling_mean_monthly_pct_change["S&P 500"]

print("\nMonthly relative performance DataFrame re-created successfully!")
print("First 5 rows of relative_performance_monthly:")
display(relative_performance_monthly.head())

# 4. Define monthly 'buy', 'sell', or 'hold' signals
buy_threshold_monthly = 0.001
sell_threshold_monthly = -0.001

signals_monthly = pd.DataFrame(index=relative_performance_monthly.index)

for sector in relative_performance_monthly.columns:
    signals_monthly[sector] = 'Hold'  # Initialize all to 'Hold'
    signals_monthly.loc[relative_performance_monthly[sector] > buy_threshold_monthly, sector] = 'Buy'
    signals_monthly.loc[relative_performance_monthly[sector] < sell_threshold_monthly, sector] = 'Sell'

print("\nMonthly signals DataFrame re-created successfully!")
print("First 5 rows of signals_monthly:")
display(signals_monthly.head())

# 5. Backtest this monthly trading strategy
portfolio_returns_monthly = pd.DataFrame(index=signals_monthly.index, columns=['Portfolio Return'])

num_tradeable_sectors_monthly = len(sector_etfs_monthly)

returns_aligned_monthly = df_sector_monthly_pct_change[sector_etfs_monthly].align(signals_monthly, join='inner', axis=0)[0]
signals_aligned_monthly = signals_monthly.align(returns_aligned_monthly, join='inner', axis=0)[0]

current_portfolio_weights_at_start_of_period_monthly = {sector: 1.0 / num_tradeable_sectors_monthly for sector in sector_etfs_monthly}

historical_weights_monthly = pd.DataFrame(columns=sector_etfs_monthly, index=signals_aligned_monthly.index)

for i in range(len(signals_aligned_monthly)):
    current_date = signals_aligned_monthly.index[i]
    current_signals = signals_aligned_monthly.loc[current_date]
    current_month_returns = returns_aligned_monthly.loc[current_date]

    weights_for_this_period_monthly = {sector: 0.0 for sector in sector_etfs_monthly}

    buy_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Buy']
    sell_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Sell']
    hold_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Hold']

    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_monthly[sector] = weight_per_active_sector
    else:
        pass # weights_for_this_period_monthly is already initialized to 0.0

    historical_weights_monthly.loc[current_date] = list(weights_for_this_period_monthly.values())

    monthly_port_return = sum(weights_for_this_period_monthly[s] * current_month_returns[s] for s in sector_etfs_monthly)
    portfolio_returns_monthly.loc[current_date, 'Portfolio Return'] = monthly_port_return

    current_portfolio_weights_at_start_of_period_monthly = weights_for_this_period_monthly

# 4. Calculate the cumulative returns of the portfolio_returns_monthly
# (1 + Return) cumprod() - 1 gives the cumulative return based on compounding.
portfolio_returns_monthly['Cumulative Return'] = (1 + portfolio_returns_monthly['Portfolio Return']).cumprod() - 1

print("\nMonthly portfolio simulation re-complete.")
print("First 5 rows of portfolio_returns_monthly:")
display(portfolio_returns_monthly.head())
print("\nLast 5 rows of portfolio_returns_monthly:")
display(portfolio_returns_monthly.tail())

**Reasoning**:
I need to recalculate all performance metrics (total cumulative return, annualized return, annualized volatility, and Sharpe Ratio) for the monthly strategy, fixed-threshold weekly strategy, dynamic-threshold weekly strategies (with and without costs), and the S&P 500 benchmark. This ensures all metrics reflect the updated data (including 'XLRE' and the extended date range) and the corrected monthly backtest, and then present them in a comparative summary table.



In [ ]:
# 131.0

rf = 0.01 # Re-defining risk-free rate, as it was not defined in this execution context

num_months_monthly = len(portfolio_returns_monthly)
num_years_monthly = num_months_monthly / 12.0

# Monthly Strategy Metrics
portfolio_total_cumulative_return_monthly = portfolio_returns_monthly['Cumulative Return'].iloc[-1]
portfolio_annualized_return_monthly = (1 + portfolio_total_cumulative_return_monthly)**(1/num_years_monthly) - 1
# Monthly volatility, then annualize (sqrt(12) for monthly to annual)
portfolio_annualized_volatility_monthly = portfolio_returns_monthly['Portfolio Return'].std() * np.sqrt(12)
portfolio_sharpe_ratio_monthly = (portfolio_annualized_return_monthly - rf) / portfolio_annualized_volatility_monthly

# --- Recalculate metrics for Fixed Strategy --- (using already calculated portfolio_returns)
num_weeks = len(portfolio_returns)
num_years = num_weeks / 52.0
portfolio_total_cumulative_return = portfolio_returns['Cumulative Return'].iloc[-1]
portfolio_annualized_return = (1 + portfolio_total_cumulative_return)**(1/num_years) - 1
portfolio_annualized_volatility = portfolio_returns['Portfolio Return'].std() * np.sqrt(52)
portfolio_sharpe_ratio = (portfolio_annualized_return - rf) / portfolio_annualized_volatility

# Recalculate S&P 500 metrics for consistency with the fixed strategy's time frame
s_p_500_returns_aligned = df_sector_pct_change["S&P 500"].align(portfolio_returns, join='inner', axis=0)[0]
s_p_500_cumulative_return = (1 + s_p_500_returns_aligned).cumprod() - 1
s_p_500_total_cumulative_return = s_p_500_cumulative_return.iloc[-1]
s_p_500_annualized_return = (1 + s_p_500_total_cumulative_return)**(1/num_years) - 1
s_p_500_annualized_volatility = s_p_500_returns_aligned.std() * np.sqrt(52)
s_p_500_sharpe_ratio = (s_p_500_annualized_return - rf) / s_p_500_annualized_volatility

# --- Recalculate metrics for Dynamic Strategy --- (using already calculated portfolio_returns_dynamic)
num_weeks_dynamic = len(portfolio_returns_dynamic)
num_years_dynamic = num_weeks_dynamic / 52.0
portfolio_total_cumulative_return_dynamic = portfolio_returns_dynamic['Cumulative Return'].iloc[-1]
portfolio_annualized_return_dynamic = (1 + portfolio_total_cumulative_return_dynamic)**(1/num_years_dynamic) - 1
portfolio_annualized_volatility_dynamic = portfolio_returns_dynamic['Portfolio Return'].std() * np.sqrt(52)
portfolio_sharpe_ratio_dynamic = (portfolio_annualized_return_dynamic - rf) / portfolio_annualized_volatility_dynamic

# --- Recalculate metrics for Dynamic Strategy with Costs --- (using already calculated portfolio_returns_dynamic_with_costs)
num_weeks_dynamic_costs = len(portfolio_returns_dynamic_with_costs)
num_years_dynamic_costs = num_weeks_dynamic_costs / 52.0
portfolio_total_cumulative_return_dynamic_costs = portfolio_returns_dynamic_with_costs['Cumulative Return'].iloc[-1]
portfolio_annualized_return_dynamic_costs = (1 + portfolio_total_cumulative_return_dynamic_costs)**(1/num_years_dynamic_costs) - 1
portfolio_annualized_volatility_dynamic_costs = portfolio_returns_dynamic_with_costs['Portfolio Return'].std() * np.sqrt(52)
portfolio_sharpe_ratio_dynamic_costs = (portfolio_annualized_return_dynamic_costs - rf) / portfolio_annualized_volatility_dynamic_costs

# Print Summary Comparison
print("\n--- Strategy Performance Comparison (Monthly vs. Dynamic Costs vs. Dynamic vs. Fixed vs. S&P 500) ---")
print(f"Metric                        | Monthly         | Dynamic Costs   | Dynamic         | Fixed Threshold | S&P 500       ")
print(f"------------------------------------------------------------------------------------------------------------------")
print(f"Total Cumulative Return       | {portfolio_total_cumulative_return_monthly:.4f}        | {portfolio_total_cumulative_return_dynamic_costs:.4f}        | {portfolio_total_cumulative_return_dynamic:.4f}         | {portfolio_total_cumulative_return:.4f}        | {s_p_500_total_cumulative_return:.4f}       ")
print(f"Annualized Return             | {portfolio_annualized_return_monthly:.4f}        | {portfolio_annualized_return_dynamic_costs:.4f}        | {portfolio_annualized_return_dynamic:.4f}         | {portfolio_annualized_return:.4f}        | {s_p_500_annualized_return:.4f}       ")
print(f"Annualized Volatility         | {portfolio_annualized_volatility_monthly:.4f}        | {portfolio_annualized_volatility_dynamic_costs:.4f}        | {portfolio_annualized_volatility_dynamic:.4f}         | {portfolio_annualized_volatility:.4f}        | {s_p_500_annualized_volatility:.4f}       ")
print(f"Sharpe Ratio                  | {portfolio_sharpe_ratio_monthly:.4f}        | {portfolio_sharpe_ratio_dynamic_costs:.4f}        | {portfolio_sharpe_ratio_dynamic:.4f}         | {portfolio_sharpe_ratio:.4f}        | {s_p_500_sharpe_ratio:.4f}       ")
print("------------------------------------------------------------------------------------------------------------------")

**Reasoning**:
I need to regenerate the cumulative returns plots for all strategies to visually assess their performance including the 'XLRE' sector and updated backtests. This will involve plotting the cumulative returns of the monthly strategy, the fixed-threshold weekly strategy, the dynamic-threshold weekly strategy (without costs), the dynamic-threshold weekly strategy (with costs), and the S&P 500 benchmark on a single chart with distinct colors, title, labels, and a legend, as specified in the instructions.



In [ ]:
# 132.0

import matplotlib.pyplot as plt

plt.figure(figsize=(14, 7))
plt.plot(portfolio_returns_monthly.index, portfolio_returns_monthly['Cumulative Return'], label='Monthly Strategy Cumulative Return', color='orange')
plt.plot(portfolio_returns.index, portfolio_returns['Cumulative Return'], label='Fixed Strategy Cumulative Return', color='blue')
plt.plot(portfolio_returns_dynamic.index, portfolio_returns_dynamic['Cumulative Return'], label='Dynamic Strategy Cumulative Return (no costs)', color='green')
plt.plot(portfolio_returns_dynamic_with_costs.index, portfolio_returns_dynamic_with_costs['Cumulative Return'], label='Dynamic Strategy Cumulative Return (with costs)', color='purple')
plt.plot(s_p_500_cumulative_return.index, s_p_500_cumulative_return, label='S&P 500 Cumulative Return', color='red')
plt.title('Cumulative Returns: All Strategies vs. S&P 500 (Including XLRE)')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend()
plt.show()

**Reasoning**:
The next point in the comprehensive summary is 'Trade Frequency'. Before I can provide insights, I need to display the `fixed_strategy_weekly_trades` to observe the weekly number of rebalancing actions for the fixed-threshold strategy, as instructed in the overall task requirements.



In [ ]:
# 133.0

print("Fixed-threshold strategy - Weekly trades (first 5 rows):")
display(fixed_strategy_weekly_trades.head())
print("\nFixed-threshold strategy - Weekly trades (last 5 rows):")
display(fixed_strategy_weekly_trades.tail())

**Reasoning**:
Following the display of fixed-threshold weekly trades, I need to display the first and last five rows of the `dynamic_strategy_weekly_trades` Series to provide a comprehensive view of the weekly rebalancing actions for the dynamic-threshold strategy, as required by the overall task to understand trade frequency.



In [ ]:
# 134.0

print("Dynamic-threshold strategy - Weekly trades (first 5 rows):")
display(dynamic_strategy_weekly_trades.head())
print("\nDynamic-threshold strategy - Weekly trades (last 5 rows):")
display(dynamic_strategy_weekly_trades.tail())

**Reasoning**:
I need to re-load the 'Sector_ETF_Data.csv' into a DataFrame, handle any missing values using ffill() and bfill(), and then display the sum of nulls and the head of the DataFrame as requested.



In [ ]:
# 135.0

import pandas as pd

df_sector = pd.read_csv("Sector_ETF_Data.csv", index_col="Date", parse_dates=True)

# Correct for missing values using ffill() and then bfill()
df_sector = df_sector.ffill()
df_sector = df_sector.bfill()

print("Missing values after correction:")
display(df_sector.isnull().sum())

print("DataFrame head after missing value correction:")
display(df_sector.head())

**Reasoning**:
I need to fetch macroeconomic data from FRED for the updated date range ('2010-01-01' to '2025-12-30') and save it to 'Macroeconomic_Data.csv'. The provided notebook cell `1fa12e90` already contains the logic to perform this task.



In [ ]:
# 136.0

import pandas as pd
from fredapi import Fred

# FRED API key - Ensure this is correctly set or retrieved from a secure source.
# For demonstration, using the API key from the provided notebook context.
fred = Fred(api_key='e8fb9c414f6ab8f1a06a5472466a31dc')

# Define date range for weekly index, consistent with previous data fetches
start_date = "2010-01-01"
end_date = "2025-12-30"
weekly_index = pd.date_range(start=start_date, end=end_date, freq='W-FRI').strftime('%Y-%m-%d')

# Define FRED series IDs and their corresponding labels
series_dict = {
    'GDP': 'GDP',
    'Inflation (CPI)': 'CPIAUCSL',
    'Unemployment Rate': 'UNRATE',
    'Interest Rate (Fed Funds)': 'FEDFUNDS'
}

# Create a DataFrame with weekly index to store macroeconomic data
macro_data = pd.DataFrame(index=pd.to_datetime(weekly_index))
macro_data.index.name = "Date"

# Fetch and interpolate data for each series
for label, series_id in series_dict.items():
    data = fred.get_series(series_id)
    if data is None: # Handle cases where FRED API might return None
        print(f"Warning: Could not fetch series {series_id} for {label}")
        continue

    data = data.to_frame(name=label)
    data.index = pd.to_datetime(data.index)

    # Resample to daily, interpolating or forward-filling as specified
    if label == 'Interest Rate (Fed Funds)':
        daily = data.resample('D').ffill()
    else:
        # For other series, interpolate missing values to daily frequency
        daily = data.resample('D').interpolate(method='linear')

    # Reindex the daily data to the predefined weekly index and interpolate remaining missing values
    # Use limit_direction='both' for interpolation to handle NaNs at ends of series
    weekly_processed = daily.reindex(macro_data.index).interpolate(method='linear', limit_direction='both')
    macro_data[label] = weekly_processed[label]

# Save the completed macroeconomic DataFrame to a CSV file
macro_data.to_csv("Macroeconomic_Data.csv", index=True)

print("Macroeconomic Data fetched, processed, and saved successfully to 'Macroeconomic_Data.csv'!")

**Reasoning**:
Now that the macroeconomic data is updated and saved, the next step is to calculate the weekly percentage change for the sector data, compute the 12-week rolling mean of these changes, and then derive the `relative_performance` for each sector against the S&P 500. This will incorporate the newly added 'Real Estate' sector and the updated date range. This aligns with the previous subtask's description and the overall flow of the notebook.



In [ ]:
# 137.0

df_sector_pct_change = df_sector.pct_change().dropna()

rolling_mean_pct_change = df_sector_pct_change.rolling(window=12).mean().dropna()

relative_performance = pd.DataFrame(index=rolling_mean_pct_change.index)

sector_etfs = [col for col in rolling_mean_pct_change.columns if col != "S&P 500"]

for sector in sector_etfs:
    relative_performance[sector] = rolling_mean_pct_change[sector] - rolling_mean_pct_change["S&P 500"]

print("df_sector_pct_change, rolling_mean_pct_change, and relative_performance recalculated successfully!")
print("First 5 rows of relative_performance:")
display(relative_performance.head())

**Reasoning**:
I need to regenerate the 'signals' DataFrame using fixed thresholds and then re-run the backtest for the fixed-threshold strategy, following the detailed instructions provided in the subtask.



In [ ]:
# 138.0

buy_threshold = 0.001
sell_threshold = -0.001

signals = pd.DataFrame(index=relative_performance.index)

for sector in relative_performance.columns:
    signals[sector] = 'Hold'  # Initialize all to 'Hold'
    signals.loc[relative_performance[sector] > buy_threshold, sector] = 'Buy'
    signals.loc[relative_performance[sector] < sell_threshold, sector] = 'Sell'

print("Fixed-threshold signals DataFrame regenerated successfully!")
print("First 5 rows of signals:")
display(signals.head())

# Re-run the backtesting simulation for the fixed-threshold strategy
# 1. Initialize an empty DataFrame called portfolio_returns
portfolio_returns = pd.DataFrame(index=signals.index, columns=['Portfolio Return'])

# Get list of sector ETFs (excluding S&P 500)
sector_etfs = [col for col in df_sector_pct_change.columns if col != "S&P 500"]
num_tradeable_sectors = len(sector_etfs)

# Align returns and signals to ensure they cover the same time period
# This ensures both DataFrames start and end on the same dates.
returns_aligned = df_sector_pct_change[sector_etfs].align(signals, join='inner', axis=0)[0]
signals_aligned = signals.align(returns_aligned, join='inner', axis=0)[0]

# 2. Initialize current_portfolio_weights_at_start_of_period
# These are the weights assumed at the start of the *first* period for which we have signals.
current_portfolio_weights_at_start_of_period = {sector: 1.0 / num_tradeable_sectors for sector in sector_etfs}

# Create a DataFrame to store historical weights for analysis (optional, but good for inspection)
historical_weights = pd.DataFrame(columns=sector_etfs, index=signals_aligned.index)

# 3. Iterate through each time step (row) in the aligned signals DataFrame
for i in range(len(signals_aligned)):
    current_date = signals_aligned.index[i]
    current_signals = signals_aligned.loc[current_date] # Signals for the current period
    current_week_returns = returns_aligned.loc[current_date] # Returns for the current period

    # `weights_for_this_period` will hold the actual weights used for the current period's return calculation.
    # Initialize all to 0.0, and then assign based on signals.
    weights_for_this_period = {sector: 0.0 for sector in sector_etfs}

    buy_sectors = [s for s in sector_etfs if current_signals[s] == 'Buy']
    sell_sectors = [s for s in sector_etfs if current_signals[s] == 'Sell']
    hold_sectors = [s for s in sector_etfs if current_signals[s] == 'Hold']

    # Determine the 'active' sectors for this period (those not getting a 'Sell' signal).
    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period[sector] = weight_per_active_sector
        # Sectors not in active_sectors_this_period (i.e., sell_sectors) implicitly retain 0.0 weight.
    else:
        # If all sectors are 'Sell', or no active signals, all weights remain 0.0.
        # Portfolio return for this week will be 0.0.
        pass # weights_for_this_period is already initialized to 0.0

    # Store the weights used for this period calculation in the historical_weights DataFrame.
    historical_weights.loc[current_date] = list(weights_for_this_period.values())

    # Calculate portfolio return for the current week using the determined weights.
    weekly_port_return = sum(weights_for_this_period[s] * current_week_returns[s] for s in sector_etfs)
    portfolio_returns.loc[current_date, 'Portfolio Return'] = weekly_port_return

    # Update `current_portfolio_weights_at_start_of_period` for the *next* iteration.
    # This reflects the portfolio composition at the end of the current period, which is the start of the next.
    current_portfolio_weights_at_start_of_period = weights_for_this_period

# 4. Calculate the cumulative returns of the portfolio_returns
# (1 + Return) cumprod() - 1 gives the cumulative return based on compounding.
portfolio_returns['Cumulative Return'] = (1 + portfolio_returns['Portfolio Return']).cumprod() - 1

print("Fixed-threshold backtesting simulation complete.")
print("First 5 rows of portfolio_returns:")
display(portfolio_returns.head())

**Reasoning**:
I need to regenerate the backtest for the dynamic-threshold strategy, including the version with transaction costs, as per the instructions. This involves re-calculating the signals, then simulating the portfolio's returns while deducting the specified transaction cost for each rebalancing event.



In [ ]:
# 139.0

rolling_std_relative_performance = relative_performance.rolling(window=12).std()

multiplier = 0.5 # Adjust this multiplier as needed

dynamic_buy_thresholds = multiplier * rolling_std_relative_performance
dynamic_sell_thresholds = -multiplier * rolling_std_relative_performance

signals_dynamic = pd.DataFrame(index=relative_performance.index)

for sector in relative_performance.columns:
    signals_dynamic[sector] = 'Hold'  # Initialize all to 'Hold'
    # Apply 'Buy' signal where relative performance is above the dynamic buy threshold
    buy_condition = relative_performance[sector] > dynamic_buy_thresholds[sector]
    signals_dynamic.loc[buy_condition, sector] = 'Buy'

    # Apply 'Sell' signal where relative performance is below the dynamic sell threshold
    sell_condition = relative_performance[sector] < dynamic_sell_thresholds[sector]
    signals_dynamic.loc[sell_condition, sector] = 'Sell'

print("Dynamic signals DataFrame regenerated successfully!")
print("First 5 rows of signals_dynamic:")
display(signals_dynamic.head())

# Re-run the backtesting simulation for the dynamic-threshold strategy without costs
portfolio_returns_dynamic = pd.DataFrame(index=signals_dynamic.index, columns=['Portfolio Return'])

sector_etfs = [col for col in df_sector_pct_change.columns if col != "S&P 500"]
num_tradeable_sectors = len(sector_etfs)

returns_aligned_dynamic = df_sector_pct_change[sector_etfs].align(signals_dynamic, join='inner', axis=0)[0]
signals_aligned_dynamic = signals_dynamic.align(returns_aligned_dynamic, join='inner', axis=0)[0]

valid_indices = signals_aligned_dynamic.dropna(how='all').index
signals_aligned_dynamic = signals_aligned_dynamic.loc[valid_indices]
returns_aligned_dynamic = returns_aligned_dynamic.loc[valid_indices]
portfolio_returns_dynamic = portfolio_returns_dynamic.loc[valid_indices]

current_portfolio_weights_at_start_of_period_dynamic = {sector: 1.0 / num_tradeable_sectors for sector in sector_etfs}

historical_weights_dynamic = pd.DataFrame(columns=sector_etfs, index=signals_aligned_dynamic.index)

for i in range(len(signals_aligned_dynamic)):
    current_date = signals_aligned_dynamic.index[i]
    current_signals = signals_aligned_dynamic.loc[current_date]
    current_week_returns = returns_aligned_dynamic.loc[current_date]

    weights_for_this_period_dynamic = {sector: 0.0 for sector in sector_etfs}

    buy_sectors = [s for s in sector_etfs if current_signals[s] == 'Buy']
    sell_sectors = [s for s in sector_etfs if current_signals[s] == 'Sell']
    hold_sectors = [s for s in sector_etfs if current_signals[s] == 'Hold']

    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_dynamic[sector] = weight_per_active_sector

    historical_weights_dynamic.loc[current_date] = list(weights_for_this_period_dynamic.values())

    weekly_port_return = sum(weights_for_this_period_dynamic[s] * current_week_returns[s] for s in sector_etfs)
    portfolio_returns_dynamic.loc[current_date, 'Portfolio Return'] = weekly_port_return

    current_portfolio_weights_at_start_of_period_dynamic = weights_for_this_period_dynamic

portfolio_returns_dynamic['Cumulative Return'] = (1 + portfolio_returns_dynamic['Portfolio Return']).cumprod() - 1

print("Dynamic portfolio simulation (without costs) complete.")
print("First 5 rows of portfolio_returns_dynamic:")
display(portfolio_returns_dynamic.head())

# Re-run the backtesting simulation for the dynamic-threshold strategy with transaction costs
portfolio_returns_dynamic_with_costs = pd.DataFrame(index=signals_aligned_dynamic.index, columns=['Portfolio Return'])

transaction_cost_rate = 0.001  # 0.1% transaction cost

previous_period_weights = {sector: 1.0 / num_tradeable_sectors for sector in sector_etfs}

for i in range(len(signals_aligned_dynamic)):
    current_date = signals_aligned_dynamic.index[i]
    current_signals = signals_aligned_dynamic.loc[current_date] # Signals for the current period
    current_week_returns = returns_aligned_dynamic.loc[current_date] # Returns for the current period

    # Determine the weights_for_this_period_dynamic
    weights_for_this_period_dynamic = {sector: 0.0 for sector in sector_etfs}
    buy_sectors = [s for s in sector_etfs if current_signals[s] == 'Buy']
    hold_sectors = [s for s in sector_etfs if current_signals[s] == 'Hold']
    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_dynamic[sector] = weight_per_active_sector

    # Calculate transaction_costs for this period
    transaction_costs = 0.0
    for sector in sector_etfs:
        weight_change = abs(weights_for_this_period_dynamic[sector] - previous_period_weights[sector])
        transaction_costs += weight_change * transaction_cost_rate

    # Calculate weekly_port_return_before_costs
    weekly_port_return_before_costs = sum(weights_for_this_period_dynamic[s] * current_week_returns[s] for s in sector_etfs)

    # Deduct transaction_costs
    weekly_port_return_after_costs = weekly_port_return_before_costs - transaction_costs
    portfolio_returns_dynamic_with_costs.loc[current_date, 'Portfolio Return'] = weekly_port_return_after_costs

    # Update previous_period_weights for the next iteration
    previous_period_weights = weights_for_this_period_dynamic

portfolio_returns_dynamic_with_costs['Cumulative Return'] = (1 + portfolio_returns_dynamic_with_costs['Portfolio Return']).cumprod() - 1

print("Dynamic portfolio simulation with transaction costs complete.")
print("First 5 rows of portfolio_returns_dynamic_with_costs:")
display(portfolio_returns_dynamic_with_costs.head())

**Reasoning**:
I need to re-prepare the monthly data, recalculate monthly relative performance and signals, and then re-run the backtest for the monthly strategy, incorporating the new XLRE sector. This comprehensive code block will ensure all necessary DataFrames are updated and the `portfolio_returns_monthly` is correctly computed.



In [ ]:
# 140.0

import pandas as pd
import numpy as np

# 1. Resample df_sector to a monthly frequency and calculate monthly percent changes
df_sector_monthly = df_sector.resample('M').last()
df_sector_monthly_pct_change = df_sector_monthly.pct_change().dropna()

# 2. Compute the 3-month rolling mean of these monthly percent changes
rolling_mean_monthly_pct_change = df_sector_monthly_pct_change.rolling(window=3).mean().dropna()

print("Monthly data re-preparation complete!")
print("First 5 rows of df_sector_monthly:")
display(df_sector_monthly.head())
print("\nFirst 5 rows of df_sector_monthly_pct_change:")
display(df_sector_monthly_pct_change.head())
print("\nFirst 5 rows of rolling_mean_monthly_pct_change:")
display(rolling_mean_monthly_pct_change.head())

# 3. Calculate monthly relative performance
relative_performance_monthly = pd.DataFrame(index=rolling_mean_monthly_pct_change.index)
sector_etfs_monthly = [col for col in rolling_mean_monthly_pct_change.columns if col != "S&P 500"]

for sector in sector_etfs_monthly:
    relative_performance_monthly[sector] = rolling_mean_monthly_pct_change[sector] - rolling_mean_monthly_pct_change["S&P 500"]

print("\nMonthly relative performance DataFrame re-created successfully!")
print("First 5 rows of relative_performance_monthly:")
display(relative_performance_monthly.head())

# 4. Define monthly 'buy', 'sell', or 'hold' signals
buy_threshold_monthly = 0.001
sell_threshold_monthly = -0.001

signals_monthly = pd.DataFrame(index=relative_performance_monthly.index)

for sector in relative_performance_monthly.columns:
    signals_monthly[sector] = 'Hold'  # Initialize all to 'Hold'
    signals_monthly.loc[relative_performance_monthly[sector] > buy_threshold_monthly, sector] = 'Buy'
    signals_monthly.loc[relative_performance_monthly[sector] < sell_threshold_monthly, sector] = 'Sell'

print("\nMonthly signals DataFrame re-created successfully!")
print("First 5 rows of signals_monthly:")
display(signals_monthly.head())

# 5. Backtest this monthly trading strategy
portfolio_returns_monthly = pd.DataFrame(index=signals_monthly.index, columns=['Portfolio Return'])

num_tradeable_sectors_monthly = len(sector_etfs_monthly)

returns_aligned_monthly = df_sector_monthly_pct_change[sector_etfs_monthly].align(signals_monthly, join='inner', axis=0)[0]
signals_aligned_monthly = signals_monthly.align(returns_aligned_monthly, join='inner', axis=0)[0]

current_portfolio_weights_at_start_of_period_monthly = {sector: 1.0 / num_tradeable_sectors_monthly for sector in sector_etfs_monthly}

historical_weights_monthly = pd.DataFrame(columns=sector_etfs_monthly, index=signals_aligned_monthly.index)

for i in range(len(signals_aligned_monthly)):
    current_date = signals_aligned_monthly.index[i]
    current_signals = signals_aligned_monthly.loc[current_date]
    current_month_returns = returns_aligned_monthly.loc[current_date]

    weights_for_this_period_monthly = {sector: 0.0 for sector in sector_etfs_monthly}

    buy_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Buy']
    sell_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Sell']
    hold_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Hold']

    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_monthly[sector] = weight_per_active_sector
    else:
        pass # weights_for_this_period_monthly is already initialized to 0.0

    historical_weights_monthly.loc[current_date] = list(weights_for_this_period_monthly.values())

    monthly_port_return = sum(weights_for_this_period_monthly[s] * current_month_returns[s] for s in sector_etfs_monthly)
    portfolio_returns_monthly.loc[current_date, 'Portfolio Return'] = monthly_port_return

    current_portfolio_weights_at_start_of_period_monthly = weights_for_this_period_monthly

# 4. Calculate the cumulative returns of the portfolio_returns_monthly
# (1 + Return) cumprod() - 1 gives the cumulative return based on compounding.
portfolio_returns_monthly['Cumulative Return'] = (1 + portfolio_returns_monthly['Portfolio Return']).cumprod() - 1

print("\nMonthly portfolio simulation re-complete.")
print("First 5 rows of portfolio_returns_monthly:")
display(portfolio_returns_monthly.head())
print("\nLast 5 rows of portfolio_returns_monthly:")
display(portfolio_returns_monthly.tail())

**Reasoning**:
The previous code block resulted in an IndentationError. I will regenerate the same code ensuring consistent 4-space indentation throughout the code block to resolve any potential hidden indentation issues or mix of tabs and spaces.



In [ ]:
# 141.0

import pandas as pd
import numpy as np

# 1. Resample df_sector to a monthly frequency and calculate monthly percent changes
df_sector_monthly = df_sector.resample('M').last()
df_sector_monthly_pct_change = df_sector_monthly.pct_change().dropna()

# 2. Compute the 3-month rolling mean of these monthly percent changes
rolling_mean_monthly_pct_change = df_sector_monthly_pct_change.rolling(window=3).mean().dropna()

print("Monthly data re-preparation complete!")
print("First 5 rows of df_sector_monthly:")
display(df_sector_monthly.head())
print("\nFirst 5 rows of df_sector_monthly_pct_change:")
display(df_sector_monthly_pct_change.head())
print("\nFirst 5 rows of rolling_mean_monthly_pct_change:")
display(rolling_mean_monthly_pct_change.head())

# 3. Calculate monthly relative performance
relative_performance_monthly = pd.DataFrame(index=rolling_mean_monthly_pct_change.index)
sector_etfs_monthly = [col for col in rolling_mean_monthly_pct_change.columns if col != "S&P 500"]

for sector in sector_etfs_monthly:
    relative_performance_monthly[sector] = rolling_mean_monthly_pct_change[sector] - rolling_mean_monthly_pct_change["S&P 500"]

print("\nMonthly relative performance DataFrame re-created successfully!")
print("First 5 rows of relative_performance_monthly:")
display(relative_performance_monthly.head())

# 4. Define monthly 'buy', 'sell', or 'hold' signals
buy_threshold_monthly = 0.001
sell_threshold_monthly = -0.001

signals_monthly = pd.DataFrame(index=relative_performance_monthly.index)

for sector in relative_performance_monthly.columns:
    signals_monthly[sector] = 'Hold'  # Initialize all to 'Hold'
    signals_monthly.loc[relative_performance_monthly[sector] > buy_threshold_monthly, sector] = 'Buy'
    signals_monthly.loc[relative_performance_monthly[sector] < sell_threshold_monthly, sector] = 'Sell'

print("\nMonthly signals DataFrame re-created successfully!")
print("First 5 rows of signals_monthly:")
display(signals_monthly.head())

# 5. Backtest this monthly trading strategy
portfolio_returns_monthly = pd.DataFrame(index=signals_monthly.index, columns=['Portfolio Return'])

num_tradeable_sectors_monthly = len(sector_etfs_monthly)

returns_aligned_monthly = df_sector_monthly_pct_change[sector_etfs_monthly].align(signals_monthly, join='inner', axis=0)[0]
signals_aligned_monthly = signals_monthly.align(returns_aligned_monthly, join='inner', axis=0)[0]

current_portfolio_weights_at_start_of_period_monthly = {sector: 1.0 / num_tradeable_sectors_monthly for sector in sector_etfs_monthly}

historical_weights_monthly = pd.DataFrame(columns=sector_etfs_monthly, index=signals_aligned_monthly.index)

for i in range(len(signals_aligned_monthly)):
    current_date = signals_aligned_monthly.index[i]
    current_signals = signals_aligned_monthly.loc[current_date]
    current_month_returns = returns_aligned_monthly.loc[current_date]

    weights_for_this_period_monthly = {sector: 0.0 for sector in sector_etfs_monthly}

    buy_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Buy']
    sell_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Sell']
    hold_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Hold']

    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_monthly[sector] = weight_per_active_sector
    else:
        pass # weights_for_this_period_monthly is already initialized to 0.0

    historical_weights_monthly.loc[current_date] = list(weights_for_this_period_monthly.values())

    monthly_port_return = sum(weights_for_this_period_monthly[s] * current_month_returns[s] for s in sector_etfs_monthly)
    portfolio_returns_monthly.loc[current_date, 'Portfolio Return'] = monthly_port_return

    current_portfolio_weights_at_start_of_period_monthly = weights_for_this_period_monthly

# 4. Calculate the cumulative returns of the portfolio_returns_monthly
# (1 + Return) cumprod() - 1 gives the cumulative return based on compounding.
portfolio_returns_monthly['Cumulative Return'] = (1 + portfolio_returns_monthly['Portfolio Return']).cumprod() - 1

print("\nMonthly portfolio simulation re-complete.")
print("First 5 rows of portfolio_returns_monthly:")
display(portfolio_returns_monthly.head())
print("\nLast 5 rows of portfolio_returns_monthly:")
display(portfolio_returns_monthly.tail())

**Reasoning**:
The previous code block resulted in an IndentationError. I have reviewed the code and corrected the indentation of lines within the main `for` loop, specifically those relating to storing historical weights, calculating monthly portfolio return, and updating portfolio weights, to ensure proper nesting and execution.



In [ ]:
# 142.0

import pandas as pd
import numpy as np

# 1. Resample df_sector to a monthly frequency and calculate monthly percent changes
df_sector_monthly = df_sector.resample('M').last()
df_sector_monthly_pct_change = df_sector_monthly.pct_change().dropna()

# 2. Compute the 3-month rolling mean of these monthly percent changes
rolling_mean_monthly_pct_change = df_sector_monthly_pct_change.rolling(window=3).mean().dropna()

print("Monthly data re-preparation complete!")
print("First 5 rows of df_sector_monthly:")
display(df_sector_monthly.head())
print("\nFirst 5 rows of df_sector_monthly_pct_change:")
display(df_sector_monthly_pct_change.head())
print("\nFirst 5 rows of rolling_mean_monthly_pct_change:")
display(rolling_mean_monthly_pct_change.head())

# 3. Calculate monthly relative performance
relative_performance_monthly = pd.DataFrame(index=rolling_mean_monthly_pct_change.index)
sector_etfs_monthly = [col for col in rolling_mean_monthly_pct_change.columns if col != "S&P 500"]

for sector in sector_etfs_monthly:
    relative_performance_monthly[sector] = rolling_mean_monthly_pct_change[sector] - rolling_mean_monthly_pct_change["S&P 500"]

print("\nMonthly relative performance DataFrame re-created successfully!")
print("First 5 rows of relative_performance_monthly:")
display(relative_performance_monthly.head())

# 4. Define monthly 'buy', 'sell', or 'hold' signals
buy_threshold_monthly = 0.001
sell_threshold_monthly = -0.001

signals_monthly = pd.DataFrame(index=relative_performance_monthly.index)

for sector in relative_performance_monthly.columns:
    signals_monthly[sector] = 'Hold'  # Initialize all to 'Hold'
    signals_monthly.loc[relative_performance_monthly[sector] > buy_threshold_monthly, sector] = 'Buy'
    signals_monthly.loc[relative_performance_monthly[sector] < sell_threshold_monthly, sector] = 'Sell'

print("\nMonthly signals DataFrame re-created successfully!")
print("First 5 rows of signals_monthly:")
display(signals_monthly.head())

# 5. Backtest this monthly trading strategy
portfolio_returns_monthly = pd.DataFrame(index=signals_monthly.index, columns=['Portfolio Return'])

num_tradeable_sectors_monthly = len(sector_etfs_monthly)

returns_aligned_monthly = df_sector_monthly_pct_change[sector_etfs_monthly].align(signals_monthly, join='inner', axis=0)[0]
signals_aligned_monthly = signals_monthly.align(returns_aligned_monthly, join='inner', axis=0)[0]

current_portfolio_weights_at_start_of_period_monthly = {sector: 1.0 / num_tradeable_sectors_monthly for sector in sector_etfs_monthly}

historical_weights_monthly = pd.DataFrame(columns=sector_etfs_monthly, index=signals_aligned_monthly.index)

for i in range(len(signals_aligned_monthly)):
    current_date = signals_aligned_monthly.index[i]
    current_signals = signals_aligned_monthly.loc[current_date]
    current_month_returns = returns_aligned_monthly.loc[current_date]

    weights_for_this_period_monthly = {sector: 0.0 for sector in sector_etfs_monthly}

    buy_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Buy']
    sell_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Sell']
    hold_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Hold']

    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_monthly[sector] = weight_per_active_sector
    else:
        pass # weights_for_this_period_monthly is already initialized to 0.0

    historical_weights_monthly.loc[current_date] = list(weights_for_this_period_monthly.values())

    monthly_port_return = sum(weights_for_this_period_monthly[s] * current_month_returns[s] for s in sector_etfs_monthly)
    portfolio_returns_monthly.loc[current_date, 'Portfolio Return'] = monthly_port_return

    current_portfolio_weights_at_start_of_period_monthly = weights_for_this_period_monthly

# 4. Calculate the cumulative returns of the portfolio_returns_monthly
# (1 + Return) cumprod() - 1 gives the cumulative return based on compounding.
portfolio_returns_monthly['Cumulative Return'] = (1 + portfolio_returns_monthly['Portfolio Return']).cumprod() - 1

print("\nMonthly portfolio simulation re-complete.")
print("First 5 rows of portfolio_returns_monthly:")
display(portfolio_returns_monthly.head())
print("\nLast 5 rows of portfolio_returns_monthly:")
display(portfolio_returns_monthly.tail())

**Reasoning**:
I need to recalculate all performance metrics (total cumulative return, annualized return, annualized volatility, and Sharpe Ratio) for the monthly strategy, fixed-threshold weekly strategy, dynamic-threshold weekly strategies (with and without costs), and the S&P 500 benchmark. This ensures all metrics reflect the updated data (including 'XLRE' and the extended date range) and the corrected monthly backtest, and then present them in a comparative summary table.



In [ ]:
# 143.0

rf = 0.01 # Re-defining risk-free rate, as it was not defined in this execution context

num_months_monthly = len(portfolio_returns_monthly)
num_years_monthly = num_months_monthly / 12.0

# Monthly Strategy Metrics
portfolio_total_cumulative_return_monthly = portfolio_returns_monthly['Cumulative Return'].iloc[-1]
portfolio_annualized_return_monthly = (1 + portfolio_total_cumulative_return_monthly)**(1/num_years_monthly) - 1
# Monthly volatility, then annualize (sqrt(12) for monthly to annual)
portfolio_annualized_volatility_monthly = portfolio_returns_monthly['Portfolio Return'].std() * np.sqrt(12)
portfolio_sharpe_ratio_monthly = (portfolio_annualized_return_monthly - rf) / portfolio_annualized_volatility_monthly

# --- Recalculate metrics for Fixed Strategy --- (using already calculated portfolio_returns)
num_weeks = len(portfolio_returns)
num_years = num_weeks / 52.0
portfolio_total_cumulative_return = portfolio_returns['Cumulative Return'].iloc[-1]
portfolio_annualized_return = (1 + portfolio_total_cumulative_return)**(1/num_years) - 1
portfolio_annualized_volatility = portfolio_returns['Portfolio Return'].std() * np.sqrt(52)
portfolio_sharpe_ratio = (portfolio_annualized_return - rf) / portfolio_annualized_volatility

# Recalculate S&P 500 metrics for consistency with the fixed strategy's time frame
s_p_500_returns_aligned = df_sector_pct_change["S&P 500"].align(portfolio_returns, join='inner', axis=0)[0]
s_p_500_cumulative_return = (1 + s_p_500_returns_aligned).cumprod() - 1
s_p_500_total_cumulative_return = s_p_500_cumulative_return.iloc[-1]
s_p_500_annualized_return = (1 + s_p_500_total_cumulative_return)**(1/num_years) - 1
s_p_500_annualized_volatility = s_p_500_returns_aligned.std() * np.sqrt(52)
s_p_500_sharpe_ratio = (s_p_500_annualized_return - rf) / s_p_500_annualized_volatility

# --- Recalculate metrics for Dynamic Strategy --- (using already calculated portfolio_returns_dynamic)
num_weeks_dynamic = len(portfolio_returns_dynamic)
num_years_dynamic = num_weeks_dynamic / 52.0
portfolio_total_cumulative_return_dynamic = portfolio_returns_dynamic['Cumulative Return'].iloc[-1]
portfolio_annualized_return_dynamic = (1 + portfolio_total_cumulative_return_dynamic)**(1/num_years_dynamic) - 1
portfolio_annualized_volatility_dynamic = portfolio_returns_dynamic['Portfolio Return'].std() * np.sqrt(52)
portfolio_sharpe_ratio_dynamic = (portfolio_annualized_return_dynamic - rf) / portfolio_annualized_volatility_dynamic

# --- Recalculate metrics for Dynamic Strategy with Costs --- (using already calculated portfolio_returns_dynamic_with_costs)
num_weeks_dynamic_costs = len(portfolio_returns_dynamic_with_costs)
num_years_dynamic_costs = num_weeks_dynamic_costs / 52.0
portfolio_total_cumulative_return_dynamic_costs = portfolio_returns_dynamic_with_costs['Cumulative Return'].iloc[-1]
portfolio_annualized_return_dynamic_costs = (1 + portfolio_total_cumulative_return_dynamic_costs)**(1/num_years_dynamic_costs) - 1
portfolio_annualized_volatility_dynamic_costs = portfolio_returns_dynamic_with_costs['Portfolio Return'].std() * np.sqrt(52)
portfolio_sharpe_ratio_dynamic_costs = (portfolio_annualized_return_dynamic_costs - rf) / portfolio_annualized_volatility_dynamic_costs

# Print Summary Comparison
print("\n--- Strategy Performance Comparison (Monthly vs. Dynamic Costs vs. Dynamic vs. Fixed vs. S&P 500) ---")
print(f"Metric                        | Monthly         | Dynamic Costs   | Dynamic         | Fixed Threshold | S&P 500       ")
print(f"------------------------------------------------------------------------------------------------------------------")
print(f"Total Cumulative Return       | {portfolio_total_cumulative_return_monthly:.4f}        | {portfolio_total_cumulative_return_dynamic_costs:.4f}        | {portfolio_total_cumulative_return_dynamic:.4f}         | {portfolio_total_cumulative_return:.4f}        | {s_p_500_total_cumulative_return:.4f}       ")
print(f"Annualized Return             | {portfolio_annualized_return_monthly:.4f}        | {portfolio_annualized_return_dynamic_costs:.4f}        | {portfolio_annualized_return_dynamic:.4f}         | {portfolio_annualized_return:.4f}        | {s_p_500_annualized_return:.4f}       ")
print(f"Annualized Volatility         | {portfolio_annualized_volatility_monthly:.4f}        | {portfolio_annualized_volatility_dynamic_costs:.4f}        | {portfolio_annualized_volatility_dynamic:.4f}         | {portfolio_annualized_volatility:.4f}        | {s_p_500_annualized_volatility:.4f}       ")
print(f"Sharpe Ratio                  | {portfolio_sharpe_ratio_monthly:.4f}        | {portfolio_sharpe_ratio_dynamic_costs:.4f}        | {portfolio_sharpe_ratio_dynamic:.4f}         | {portfolio_sharpe_ratio:.4f}        | {s_p_500_sharpe_ratio:.4f}       ")
print("------------------------------------------------------------------------------------------------------------------")

**Reasoning**:
I need to regenerate the cumulative returns plots for all strategies to visually assess their performance including the 'XLRE' sector and updated backtests. This will involve plotting the cumulative returns of the monthly strategy, the fixed-threshold weekly strategy, the dynamic-threshold weekly strategy (without costs), the dynamic-threshold weekly strategy (with costs), and the S&P 500 benchmark on a single chart with distinct colors, title, labels, and a legend, as specified in the instructions.



In [ ]:
# 144.0

import matplotlib.pyplot as plt

plt.figure(figsize=(14, 7))
plt.plot(portfolio_returns_monthly.index, portfolio_returns_monthly['Cumulative Return'], label='Monthly Strategy Cumulative Return', color='orange')
plt.plot(portfolio_returns.index, portfolio_returns['Cumulative Return'], label='Fixed Strategy Cumulative Return', color='blue')
plt.plot(portfolio_returns_dynamic.index, portfolio_returns_dynamic['Cumulative Return'], label='Dynamic Strategy Cumulative Return (no costs)', color='green')
plt.plot(portfolio_returns_dynamic_with_costs.index, portfolio_returns_dynamic_with_costs['Cumulative Return'], label='Dynamic Strategy Cumulative Return (with costs)', color='purple')
plt.plot(s_p_500_cumulative_return.index, s_p_500_cumulative_return, label='S&P 500 Cumulative Return', color='red')
plt.title('Cumulative Returns: All Strategies vs. S&P 500 (Including XLRE)')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend()
plt.show()

**Reasoning**:
To address the 'Trade Frequency' aspect of the comprehensive summary, I need to display the first and last five rows of the `fixed_strategy_weekly_trades` Series to provide a clear view of the weekly rebalancing actions for the fixed-threshold strategy, as instructed in the overall task requirements.



In [ ]:
# 145.0

print("Fixed-threshold strategy - Weekly trades (first 5 rows):")
display(fixed_strategy_weekly_trades.head())
print("\nFixed-threshold strategy - Weekly trades (last 5 rows):")
display(fixed_strategy_weekly_trades.tail())

**Reasoning**:
Following the display of fixed-threshold weekly trades, I need to display the first and last five rows of the `dynamic_strategy_weekly_trades` Series to provide a comprehensive view of the weekly rebalancing actions for the dynamic-threshold strategy, as required by the overall task to understand trade frequency.



In [ ]:
# 146.0

print("Dynamic-threshold strategy - Weekly trades (first 5 rows):")
display(dynamic_strategy_weekly_trades.head())
print("\nDynamic-threshold strategy - Weekly trades (last 5 rows):")
display(dynamic_strategy_weekly_trades.tail())

**Reasoning**:
The user explicitly asked to re-execute the code cell for fetching macroeconomic data with the updated date range. The existing cell `8b6d6726` already contains the correct logic for this. Executing this cell will fulfill the subtask.



In [ ]:
# 147.0

import pandas as pd
from fredapi import Fred

# FRED API key - Ensure this is correctly set or retrieved from a secure source.
# For demonstration, using the API key from the provided notebook context.
fred = Fred(api_key='e8fb9c414f6ab8f1a06a5472466a31dc')

# Define date range for weekly index, consistent with previous data fetches
start_date = "2010-01-01"
end_date = "2025-12-30"
weekly_index = pd.date_range(start=start_date, end=end_date, freq='W-FRI').strftime('%Y-%m-%d')

# Define FRED series IDs and their corresponding labels
series_dict = {
    'GDP': 'GDP',
    'Inflation (CPI)': 'CPIAUCSL',
    'Unemployment Rate': 'UNRATE',
    'Interest Rate (Fed Funds)': 'FEDFUNDS'
}

# Create a DataFrame with weekly index to store macroeconomic data
macro_data = pd.DataFrame(index=pd.to_datetime(weekly_index))
macro_data.index.name = "Date"

# Fetch and interpolate data for each series
for label, series_id in series_dict.items():
    data = fred.get_series(series_id)
    if data is None: # Handle cases where FRED API might return None
        print(f"Warning: Could not fetch series {series_id} for {label}")
        continue

    data = data.to_frame(name=label)
    data.index = pd.to_datetime(data.index)

    # Resample to daily, interpolating or forward-filling as specified
    if label == 'Interest Rate (Fed Funds)':
        daily = data.resample('D').ffill()
    else:
        # For other series, interpolate missing values to daily frequency
        daily = data.resample('D').interpolate(method='linear')

    # Reindex the daily data to the predefined weekly index and interpolate remaining missing values
    # Use limit_direction='both' for interpolation to handle NaNs at ends of series
    weekly_processed = daily.reindex(macro_data.index).interpolate(method='linear', limit_direction='both')
    macro_data[label] = weekly_processed[label]

# Save the completed macroeconomic DataFrame to a CSV file
macro_data.to_csv("Macroeconomic_Data.csv", index=True)

print("Macroeconomic Data fetched, processed, and saved successfully to 'Macroeconomic_Data.csv'!")

**Reasoning**:
The previous code block successfully fetched and saved the macroeconomic data to 'Macroeconomic_Data.csv'. Now, I need to re-load this updated CSV into the `df_macro` DataFrame to ensure it is correctly populated for the new date range, as per the subtask's requirement.



In [ ]:
# 148.0

df_macro = pd.read_csv("Macroeconomic_Data.csv", index_col="Date", parse_dates=True)

print("df_macro loaded successfully with updated macroeconomic data!")

**Reasoning**:
Now that the macroeconomic data is updated and saved, the next step is to calculate the weekly percentage change for the sector data, compute the 12-week rolling mean of these changes, and then derive the `relative_performance` for each sector against the S&P 500. This will incorporate the newly added 'Real Estate' sector and the updated date range. This aligns with the previous subtask's description and the overall flow of the notebook.



In [ ]:
# 149.0

df_sector_pct_change = df_sector.pct_change().dropna()

rolling_mean_pct_change = df_sector_pct_change.rolling(window=12).mean().dropna()

relative_performance = pd.DataFrame(index=rolling_mean_pct_change.index)

sector_etfs = [col for col in rolling_mean_pct_change.columns if col != "S&P 500"]

for sector in sector_etfs:
    relative_performance[sector] = rolling_mean_pct_change[sector] - rolling_mean_pct_change["S&P 500"]

print("df_sector_pct_change, rolling_mean_pct_change, and relative_performance recalculated successfully!")
print("First 5 rows of relative_performance:")
display(relative_performance.head())

**Reasoning**:
I need to regenerate the 'signals' DataFrame using fixed thresholds and then re-run the backtest for the fixed-threshold strategy, following the detailed instructions provided in the subtask. The provided notebook cells (`2d8d9c35`) already contain this logic.



In [ ]:
# 150.0

buy_threshold = 0.001
sell_threshold = -0.001

signals = pd.DataFrame(index=relative_performance.index)

for sector in relative_performance.columns:
    signals[sector] = 'Hold'  # Initialize all to 'Hold'
    signals.loc[relative_performance[sector] > buy_threshold, sector] = 'Buy'
    signals.loc[relative_performance[sector] < sell_threshold, sector] = 'Sell'

print("Fixed-threshold signals DataFrame regenerated successfully!")
print("First 5 rows of signals:")
display(signals.head())

# Re-run the backtesting simulation for the fixed-threshold strategy
# 1. Initialize an empty DataFrame called portfolio_returns
portfolio_returns = pd.DataFrame(index=signals.index, columns=['Portfolio Return'])

# Get list of sector ETFs (excluding S&P 500)
sector_etfs = [col for col in df_sector_pct_change.columns if col != "S&P 500"]
num_tradeable_sectors = len(sector_etfs)

# Align returns and signals to ensure they cover the same time period
# This ensures both DataFrames start and end on the same dates.
returns_aligned = df_sector_pct_change[sector_etfs].align(signals, join='inner', axis=0)[0]
signals_aligned = signals.align(returns_aligned, join='inner', axis=0)[0]

# 2. Initialize current_portfolio_weights_at_start_of_period
# These are the weights assumed at the start of the *first* period for which we have signals.
current_portfolio_weights_at_start_of_period = {sector: 1.0 / num_tradeable_sectors for sector in sector_etfs}

# Create a DataFrame to store historical weights for analysis (optional, but good for inspection)
historical_weights = pd.DataFrame(columns=sector_etfs, index=signals_aligned.index)

# 3. Iterate through each time step (row) in the aligned signals DataFrame
for i in range(len(signals_aligned)):
    current_date = signals_aligned.index[i]
    current_signals = signals_aligned.loc[current_date] # Signals for the current period
    current_week_returns = returns_aligned.loc[current_date] # Returns for the current period

    # `weights_for_this_period` will hold the actual weights used for the current period's return calculation.
    # Initialize all to 0.0, and then assign based on signals.
    weights_for_this_period = {sector: 0.0 for sector in sector_etfs}

    buy_sectors = [s for s in sector_etfs if current_signals[s] == 'Buy']
    sell_sectors = [s for s in sector_etfs if current_signals[s] == 'Sell']
    hold_sectors = [s for s in sector_etfs if current_signals[s] == 'Hold']

    # Determine the 'active' sectors for this period (those not getting a 'Sell' signal).
    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period[sector] = weight_per_active_sector
        # Sectors not in active_sectors_this_period (i.e., sell_sectors) implicitly retain 0.0 weight.
    else:
        # If all sectors are 'Sell', or no active signals, all weights remain 0.0.
        # Portfolio return for this week will be 0.0.
        pass # weights_for_this_period is already initialized to 0.0

    # Store the weights used for this period calculation in the historical_weights DataFrame.
    historical_weights.loc[current_date] = list(weights_for_this_period.values())

    # Calculate portfolio return for the current week using the determined weights.
    weekly_port_return = sum(weights_for_this_period[s] * current_week_returns[s] for s in sector_etfs)
    portfolio_returns.loc[current_date, 'Portfolio Return'] = weekly_port_return

    # Update `current_portfolio_weights_at_start_of_period` for the *next* iteration.
    # This reflects the portfolio composition at the end of the current period, which is the start of the next.
    current_portfolio_weights_at_start_of_period = weights_for_this_period

# 4. Calculate the cumulative returns of the portfolio_returns
# (1 + Return) cumprod() - 1 gives the cumulative return based on compounding.
portfolio_returns['Cumulative Return'] = (1 + portfolio_returns['Portfolio Return']).cumprod() - 1

print("Fixed-threshold backtesting simulation complete.")
print("First 5 rows of portfolio_returns:")
display(portfolio_returns.head())

**Reasoning**:
I need to regenerate the backtest for the dynamic-threshold strategy, including the version with transaction costs, as per the instructions. This involves re-calculating the signals, then simulating the portfolio's returns while deducting the specified transaction cost for each rebalancing event.



In [ ]:
# 151.0

rolling_std_relative_performance = relative_performance.rolling(window=12).std()

multiplier = 0.5 # Adjust this multiplier as needed

dynamic_buy_thresholds = multiplier * rolling_std_relative_performance
dynamic_sell_thresholds = -multiplier * rolling_std_relative_performance

signals_dynamic = pd.DataFrame(index=relative_performance.index)

for sector in relative_performance.columns:
    signals_dynamic[sector] = 'Hold'  # Initialize all to 'Hold'
    # Apply 'Buy' signal where relative performance is above the dynamic buy threshold
    buy_condition = relative_performance[sector] > dynamic_buy_thresholds[sector]
    signals_dynamic.loc[buy_condition, sector] = 'Buy'

    # Apply 'Sell' signal where relative performance is below the dynamic sell threshold
    sell_condition = relative_performance[sector] < dynamic_sell_thresholds[sector]
    signals_dynamic.loc[sell_condition, sector] = 'Sell'

print("Dynamic signals DataFrame regenerated successfully!")
print("First 5 rows of signals_dynamic:")
display(signals_dynamic.head())

# Re-run the backtesting simulation for the dynamic-threshold strategy without costs
portfolio_returns_dynamic = pd.DataFrame(index=signals_dynamic.index, columns=['Portfolio Return'])

sector_etfs = [col for col in df_sector_pct_change.columns if col != "S&P 500"]
num_tradeable_sectors = len(sector_etfs)

returns_aligned_dynamic = df_sector_pct_change[sector_etfs].align(signals_dynamic, join='inner', axis=0)[0]
signals_aligned_dynamic = signals_dynamic.align(returns_aligned_dynamic, join='inner', axis=0)[0]

valid_indices = signals_aligned_dynamic.dropna(how='all').index
signals_aligned_dynamic = signals_aligned_dynamic.loc[valid_indices]
returns_aligned_dynamic = returns_aligned_dynamic.loc[valid_indices]
portfolio_returns_dynamic = portfolio_returns_dynamic.loc[valid_indices]

current_portfolio_weights_at_start_of_period_dynamic = {sector: 1.0 / num_tradeable_sectors for sector in sector_etfs}

historical_weights_dynamic = pd.DataFrame(columns=sector_etfs, index=signals_aligned_dynamic.index)

for i in range(len(signals_aligned_dynamic)):
    current_date = signals_aligned_dynamic.index[i]
    current_signals = signals_aligned_dynamic.loc[current_date]
    current_week_returns = returns_aligned_dynamic.loc[current_date]

    weights_for_this_period_dynamic = {sector: 0.0 for sector in sector_etfs}

    buy_sectors = [s for s in sector_etfs if current_signals[s] == 'Buy']
    sell_sectors = [s for s in sector_etfs if current_signals[s] == 'Sell']
    hold_sectors = [s for s in sector_etfs if current_signals[s] == 'Hold']

    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_dynamic[sector] = weight_per_active_sector

    historical_weights_dynamic.loc[current_date] = list(weights_for_this_period_dynamic.values())

    weekly_port_return = sum(weights_for_this_period_dynamic[s] * current_week_returns[s] for s in sector_etfs)
    portfolio_returns_dynamic.loc[current_date, 'Portfolio Return'] = weekly_port_return

    current_portfolio_weights_at_start_of_period_dynamic = weights_for_this_period_dynamic

portfolio_returns_dynamic['Cumulative Return'] = (1 + portfolio_returns_dynamic['Portfolio Return']).cumprod() - 1

print("Dynamic portfolio simulation (without costs) complete.")
print("First 5 rows of portfolio_returns_dynamic:")
display(portfolio_returns_dynamic.head())

# Re-run the backtesting simulation for the dynamic-threshold strategy with transaction costs
portfolio_returns_dynamic_with_costs = pd.DataFrame(index=signals_aligned_dynamic.index, columns=['Portfolio Return'])

transaction_cost_rate = 0.001  # 0.1% transaction cost

previous_period_weights = {sector: 1.0 / num_tradeable_sectors for sector in sector_etfs}

for i in range(len(signals_aligned_dynamic)):
    current_date = signals_aligned_dynamic.index[i]
    current_signals = signals_aligned_dynamic.loc[current_date] # Signals for the current period
    current_week_returns = returns_aligned_dynamic.loc[current_date] # Returns for the current period

    # Determine the weights_for_this_period_dynamic
    weights_for_this_period_dynamic = {sector: 0.0 for sector in sector_etfs}
    buy_sectors = [s for s in sector_etfs if current_signals[s] == 'Buy']
    hold_sectors = [s for s in sector_etfs if current_signals[s] == 'Hold']
    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_dynamic[sector] = weight_per_active_sector

    # Calculate transaction_costs for this period
    transaction_costs = 0.0
    for sector in sector_etfs:
        weight_change = abs(weights_for_this_period_dynamic[sector] - previous_period_weights[sector])
        transaction_costs += weight_change * transaction_cost_rate

    # Calculate weekly_port_return_before_costs
    weekly_port_return_before_costs = sum(weights_for_this_period_dynamic[s] * current_week_returns[s] for s in sector_etfs)

    # Deduct transaction_costs
    weekly_port_return_after_costs = weekly_port_return_before_costs - transaction_costs
    portfolio_returns_dynamic_with_costs.loc[current_date, 'Portfolio Return'] = weekly_port_return_after_costs

    # Update previous_period_weights for the next iteration
    previous_period_weights = weights_for_this_period_dynamic

portfolio_returns_dynamic_with_costs['Cumulative Return'] = (1 + portfolio_returns_dynamic_with_costs['Portfolio Return']).cumprod() - 1

print("Dynamic portfolio simulation with transaction costs complete.")
print("First 5 rows of portfolio_returns_dynamic_with_costs:")
display(portfolio_returns_dynamic_with_costs.head())

**Reasoning**:
I need to re-prepare the monthly data, recalculate monthly relative performance and signals, and then re-run the backtest for the monthly strategy, incorporating the new XLRE sector. This comprehensive code block will ensure all necessary DataFrames are updated and the `portfolio_returns_monthly` is correctly computed.



In [ ]:
# 152.0

import pandas as pd
import numpy as np

# 1. Resample df_sector to a monthly frequency and calculate monthly percent changes
df_sector_monthly = df_sector.resample('M').last()
df_sector_monthly_pct_change = df_sector_monthly.pct_change().dropna()

# 2. Compute the 3-month rolling mean of these monthly percent changes
rolling_mean_monthly_pct_change = df_sector_monthly_pct_change.rolling(window=3).mean().dropna()

print("Monthly data re-preparation complete!")
print("First 5 rows of df_sector_monthly:")
display(df_sector_monthly.head())
print("\nFirst 5 rows of df_sector_monthly_pct_change:")
display(df_sector_monthly_pct_change.head())
print("\nFirst 5 rows of rolling_mean_monthly_pct_change:")
display(rolling_mean_monthly_pct_change.head())

# 3. Calculate monthly relative performance
relative_performance_monthly = pd.DataFrame(index=rolling_mean_monthly_pct_change.index)
sector_etfs_monthly = [col for col in rolling_mean_monthly_pct_change.columns if col != "S&P 500"]

for sector in sector_etfs_monthly:
    relative_performance_monthly[sector] = rolling_mean_monthly_pct_change[sector] - rolling_mean_monthly_pct_change["S&P 500"]

print("\nMonthly relative performance DataFrame re-created successfully!")
print("First 5 rows of relative_performance_monthly:")
display(relative_performance_monthly.head())

# 4. Define monthly 'buy', 'sell', or 'hold' signals
buy_threshold_monthly = 0.001
sell_threshold_monthly = -0.001

signals_monthly = pd.DataFrame(index=relative_performance_monthly.index)

for sector in relative_performance_monthly.columns:
    signals_monthly[sector] = 'Hold'  # Initialize all to 'Hold'
    signals_monthly.loc[relative_performance_monthly[sector] > buy_threshold_monthly, sector] = 'Buy'
    signals_monthly.loc[relative_performance_monthly[sector] < sell_threshold_monthly, sector] = 'Sell'

print("\nMonthly signals DataFrame re-created successfully!")
print("First 5 rows of signals_monthly:")
display(signals_monthly.head())

# 5. Backtest this monthly trading strategy
portfolio_returns_monthly = pd.DataFrame(index=signals_monthly.index, columns=['Portfolio Return'])

num_tradeable_sectors_monthly = len(sector_etfs_monthly)

returns_aligned_monthly = df_sector_monthly_pct_change[sector_etfs_monthly].align(signals_monthly, join='inner', axis=0)[0]
signals_aligned_monthly = signals_monthly.align(returns_aligned_monthly, join='inner', axis=0)[0]

current_portfolio_weights_at_start_of_period_monthly = {sector: 1.0 / num_tradeable_sectors_monthly for sector in sector_etfs_monthly}

historical_weights_monthly = pd.DataFrame(columns=sector_etfs_monthly, index=signals_aligned_monthly.index)

for i in range(len(signals_aligned_monthly)):
    current_date = signals_aligned_monthly.index[i]
    current_signals = signals_aligned_monthly.loc[current_date]
    current_month_returns = returns_aligned_monthly.loc[current_date]

    weights_for_this_period_monthly = {sector: 0.0 for sector in sector_etfs_monthly}

    buy_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Buy']
    sell_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Sell']
    hold_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Hold']

    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_monthly[sector] = weight_per_active_sector
    else:
        pass # weights_for_this_period_monthly is already initialized to 0.0

    historical_weights_monthly.loc[current_date] = list(weights_for_this_period_monthly.values())

    monthly_port_return = sum(weights_for_this_period_monthly[s] * current_month_returns[s] for s in sector_etfs_monthly)
    portfolio_returns_monthly.loc[current_date, 'Portfolio Return'] = monthly_port_return

    current_portfolio_weights_at_start_of_period_monthly = weights_for_this_period_monthly

# 4. Calculate the cumulative returns of the portfolio_returns_monthly
# (1 + Return) cumprod() - 1 gives the cumulative return based on compounding.
portfolio_returns_monthly['Cumulative Return'] = (1 + portfolio_returns_monthly['Portfolio Return']).cumprod() - 1

print("\nMonthly portfolio simulation re-complete.")
print("First 5 rows of portfolio_returns_monthly:")
display(portfolio_returns_monthly.head())
print("\nLast 5 rows of portfolio_returns_monthly:")
display(portfolio_returns_monthly.tail())

**Reasoning**:
The previous code block resulted in an IndentationError. I have reviewed the code and corrected the indentation of lines within the main `for` loop, specifically those relating to storing historical weights, calculating monthly portfolio return, and updating portfolio weights, to ensure proper nesting and execution.



In [ ]:
# 153.0

import pandas as pd
import numpy as np

# 1. Resample df_sector to a monthly frequency and calculate monthly percent changes
df_sector_monthly = df_sector.resample('M').last()
df_sector_monthly_pct_change = df_sector_monthly.pct_change().dropna()

# 2. Compute the 3-month rolling mean of these monthly percent changes
rolling_mean_monthly_pct_change = df_sector_monthly_pct_change.rolling(window=3).mean().dropna()

print("Monthly data re-preparation complete!")
print("First 5 rows of df_sector_monthly:")
display(df_sector_monthly.head())
print("\nFirst 5 rows of df_sector_monthly_pct_change:")
display(df_sector_monthly_pct_change.head())
print("\nFirst 5 rows of rolling_mean_monthly_pct_change:")
display(rolling_mean_monthly_pct_change.head())

# 3. Calculate monthly relative performance
relative_performance_monthly = pd.DataFrame(index=rolling_mean_monthly_pct_change.index)
sector_etfs_monthly = [col for col in rolling_mean_monthly_pct_change.columns if col != "S&P 500"]

for sector in sector_etfs_monthly:
    relative_performance_monthly[sector] = rolling_mean_monthly_pct_change[sector] - rolling_mean_monthly_pct_change["S&P 500"]

print("\nMonthly relative performance DataFrame re-created successfully!")
print("First 5 rows of relative_performance_monthly:")
display(relative_performance_monthly.head())

# 4. Define monthly 'buy', 'sell', or 'hold' signals
buy_threshold_monthly = 0.001
sell_threshold_monthly = -0.001

signals_monthly = pd.DataFrame(index=relative_performance_monthly.index)

for sector in relative_performance_monthly.columns:
    signals_monthly[sector] = 'Hold'  # Initialize all to 'Hold'
    signals_monthly.loc[relative_performance_monthly[sector] > buy_threshold_monthly, sector] = 'Buy'
    signals_monthly.loc[relative_performance_monthly[sector] < sell_threshold_monthly, sector] = 'Sell'

print("\nMonthly signals DataFrame re-created successfully!")
print("First 5 rows of signals_monthly:")
display(signals_monthly.head())

# 5. Backtest this monthly trading strategy
portfolio_returns_monthly = pd.DataFrame(index=signals_monthly.index, columns=['Portfolio Return'])

num_tradeable_sectors_monthly = len(sector_etfs_monthly)

returns_aligned_monthly = df_sector_monthly_pct_change[sector_etfs_monthly].align(signals_monthly, join='inner', axis=0)[0]
signals_aligned_monthly = signals_monthly.align(returns_aligned_monthly, join='inner', axis=0)[0]

current_portfolio_weights_at_start_of_period_monthly = {sector: 1.0 / num_tradeable_sectors_monthly for sector in sector_etfs_monthly}

historical_weights_monthly = pd.DataFrame(columns=sector_etfs_monthly, index=signals_aligned_monthly.index)

for i in range(len(signals_aligned_monthly)):
    current_date = signals_aligned_monthly.index[i]
    current_signals = signals_aligned_monthly.loc[current_date]
    current_month_returns = returns_aligned_monthly.loc[current_date]

    weights_for_this_period_monthly = {sector: 0.0 for sector in sector_etfs_monthly}

    buy_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Buy']
    sell_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Sell']
    hold_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Hold']

    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_monthly[sector] = weight_per_active_sector
    else:
        pass # weights_for_this_period_monthly is already initialized to 0.0

    historical_weights_monthly.loc[current_date] = list(weights_for_this_period_monthly.values())

    monthly_port_return = sum(weights_for_this_period_monthly[s] * current_month_returns[s] for s in sector_etfs_monthly)
    portfolio_returns_monthly.loc[current_date, 'Portfolio Return'] = monthly_port_return

    current_portfolio_weights_at_start_of_period_monthly = weights_for_this_period_monthly

# 4. Calculate the cumulative returns of the portfolio_returns_monthly
# (1 + Return) cumprod() - 1 gives the cumulative return based on compounding.
portfolio_returns_monthly['Cumulative Return'] = (1 + portfolio_returns_monthly['Portfolio Return']).cumprod() - 1

print("\nMonthly portfolio simulation re-complete.")
print("First 5 rows of portfolio_returns_monthly:")
display(portfolio_returns_monthly.head())
print("\nLast 5 rows of portfolio_returns_monthly:")
display(portfolio_returns_monthly.tail())

**Reasoning**:
I need to recalculate all performance metrics (total cumulative return, annualized return, annualized volatility, and Sharpe Ratio) for the monthly strategy, fixed-threshold weekly strategy, dynamic-threshold weekly strategies (with and without costs), and the S&P 500 benchmark. This ensures all metrics reflect the updated data (including 'XLRE' and the extended date range) and the corrected monthly backtest, and then present them in a comparative summary table.



In [ ]:
# 154.0

rf = 0.01 # Re-defining risk-free rate, as it was not defined in this execution context

num_months_monthly = len(portfolio_returns_monthly)
num_years_monthly = num_months_monthly / 12.0

# Monthly Strategy Metrics
portfolio_total_cumulative_return_monthly = portfolio_returns_monthly['Cumulative Return'].iloc[-1]
portfolio_annualized_return_monthly = (1 + portfolio_total_cumulative_return_monthly)**(1/num_years_monthly) - 1
# Monthly volatility, then annualize (sqrt(12) for monthly to annual)
portfolio_annualized_volatility_monthly = portfolio_returns_monthly['Portfolio Return'].std() * np.sqrt(12)
portfolio_sharpe_ratio_monthly = (portfolio_annualized_return_monthly - rf) / portfolio_annualized_volatility_monthly

# --- Recalculate metrics for Fixed Strategy --- (using already calculated portfolio_returns)
num_weeks = len(portfolio_returns)
num_years = num_weeks / 52.0
portfolio_total_cumulative_return = portfolio_returns['Cumulative Return'].iloc[-1]
portfolio_annualized_return = (1 + portfolio_total_cumulative_return)**(1/num_years) - 1
portfolio_annualized_volatility = portfolio_returns['Portfolio Return'].std() * np.sqrt(52)
portfolio_sharpe_ratio = (portfolio_annualized_return - rf) / portfolio_annualized_volatility

# Recalculate S&P 500 metrics for consistency with the fixed strategy's time frame
s_p_500_returns_aligned = df_sector_pct_change["S&P 500"].align(portfolio_returns, join='inner', axis=0)[0]
s_p_500_cumulative_return = (1 + s_p_500_returns_aligned).cumprod() - 1
s_p_500_total_cumulative_return = s_p_500_cumulative_return.iloc[-1]
s_p_500_annualized_return = (1 + s_p_500_total_cumulative_return)**(1/num_years) - 1
s_p_500_annualized_volatility = s_p_500_returns_aligned.std() * np.sqrt(52)
s_p_500_sharpe_ratio = (s_p_500_annualized_return - rf) / s_p_500_annualized_volatility

# --- Recalculate metrics for Dynamic Strategy --- (using already calculated portfolio_returns_dynamic)
num_weeks_dynamic = len(portfolio_returns_dynamic)
num_years_dynamic = num_weeks_dynamic / 52.0
portfolio_total_cumulative_return_dynamic = portfolio_returns_dynamic['Cumulative Return'].iloc[-1]
portfolio_annualized_return_dynamic = (1 + portfolio_total_cumulative_return_dynamic)**(1/num_years_dynamic) - 1
portfolio_annualized_volatility_dynamic = portfolio_returns_dynamic['Portfolio Return'].std() * np.sqrt(52)
portfolio_sharpe_ratio_dynamic = (portfolio_annualized_return_dynamic - rf) / portfolio_annualized_volatility_dynamic

# --- Recalculate metrics for Dynamic Strategy with Costs --- (using already calculated portfolio_returns_dynamic_with_costs)
num_weeks_dynamic_costs = len(portfolio_returns_dynamic_with_costs)
num_years_dynamic_costs = num_weeks_dynamic_costs / 52.0
portfolio_total_cumulative_return_dynamic_costs = portfolio_returns_dynamic_with_costs['Cumulative Return'].iloc[-1]
portfolio_annualized_return_dynamic_costs = (1 + portfolio_total_cumulative_return_dynamic_costs)**(1/num_years_dynamic_costs) - 1
portfolio_annualized_volatility_dynamic_costs = portfolio_returns_dynamic_with_costs['Portfolio Return'].std() * np.sqrt(52)
portfolio_sharpe_ratio_dynamic_costs = (portfolio_annualized_return_dynamic_costs - rf) / portfolio_annualized_volatility_dynamic_costs

# Print Summary Comparison
print("\n--- Strategy Performance Comparison (Monthly vs. Dynamic Costs vs. Dynamic vs. Fixed vs. S&P 500) ---")
print(f"Metric                        | Monthly         | Dynamic Costs   | Dynamic         | Fixed Threshold | S&P 500       ")
print(f"------------------------------------------------------------------------------------------------------------------")
print(f"Total Cumulative Return       | {portfolio_total_cumulative_return_monthly:.4f}        | {portfolio_total_cumulative_return_dynamic_costs:.4f}        | {portfolio_total_cumulative_return_dynamic:.4f}         | {portfolio_total_cumulative_return:.4f}        | {s_p_500_total_cumulative_return:.4f}       ")
print(f"Annualized Return             | {portfolio_annualized_return_monthly:.4f}        | {portfolio_annualized_return_dynamic_costs:.4f}        | {portfolio_annualized_return_dynamic:.4f}         | {portfolio_annualized_return:.4f}        | {s_p_500_annualized_return:.4f}       ")
print(f"Annualized Volatility         | {portfolio_annualized_volatility_monthly:.4f}        | {portfolio_annualized_volatility_dynamic_costs:.4f}        | {portfolio_annualized_volatility_dynamic:.4f}         | {portfolio_annualized_volatility:.4f}        | {s_p_500_annualized_volatility:.4f}       ")
print(f"Sharpe Ratio                  | {portfolio_sharpe_ratio_monthly:.4f}        | {portfolio_sharpe_ratio_dynamic_costs:.4f}        | {portfolio_sharpe_ratio_dynamic:.4f}         | {portfolio_sharpe_ratio:.4f}        | {s_p_500_sharpe_ratio:.4f}       ")
print("------------------------------------------------------------------------------------------------------------------")

**Reasoning**:
I need to regenerate the cumulative returns plots for all strategies to visually assess their performance including the 'XLRE' sector and updated backtests. This will involve plotting the cumulative returns of the monthly strategy, the fixed-threshold weekly strategy, the dynamic-threshold weekly strategy (without costs), the dynamic-threshold weekly strategy (with costs), and the S&P 500 benchmark on a single chart with distinct colors, title, labels, and a legend, as specified in the instructions.



In [ ]:
# 155.0

import matplotlib.pyplot as plt

plt.figure(figsize=(14, 7))
plt.plot(portfolio_returns_monthly.index, portfolio_returns_monthly['Cumulative Return'], label='Monthly Strategy Cumulative Return', color='orange')
plt.plot(portfolio_returns.index, portfolio_returns['Cumulative Return'], label='Fixed Strategy Cumulative Return', color='blue')
plt.plot(portfolio_returns_dynamic.index, portfolio_returns_dynamic['Cumulative Return'], label='Dynamic Strategy Cumulative Return (no costs)', color='green')
plt.plot(portfolio_returns_dynamic_with_costs.index, portfolio_returns_dynamic_with_costs['Cumulative Return'], label='Dynamic Strategy Cumulative Return (with costs)', color='purple')
plt.plot(s_p_500_cumulative_return.index, s_p_500_cumulative_return, label='S&P 500 Cumulative Return', color='red')
plt.title('Cumulative Returns: All Strategies vs. S&P 500 (Including XLRE)')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend()
plt.show()

In [ ]:
# 156.0

print("Fixed-threshold strategy - Weekly trades (first 5 rows):")
display(fixed_strategy_weekly_trades.head())
print("\nFixed-threshold strategy - Weekly trades (last 5 rows):")
display(fixed_strategy_weekly_trades.tail())

**Reasoning**:
Following the display of fixed-threshold weekly trades, I need to display the first and last five rows of the `dynamic_strategy_weekly_trades` Series to provide a comprehensive view of the weekly rebalancing actions for the dynamic-threshold strategy, as required by the overall task to understand trade frequency.



In [ ]:
# 157.0

print("Dynamic-threshold strategy - Weekly trades (first 5 rows):")
display(dynamic_strategy_weekly_trades.head())
print("\nDynamic-threshold strategy - Weekly trades (last 5 rows):")
display(dynamic_strategy_weekly_trades.tail())

**Reasoning**:
I need to compute the average weekly trades for both the fixed-threshold and dynamic-threshold strategies. This will provide quantitative data for the 'Trade Frequency' section of the comprehensive summary, fulfilling part of the overall task requirements.



In [ ]:
# 158.0

average_fixed_weekly_trades = fixed_strategy_weekly_trades.mean()
average_dynamic_weekly_trades = dynamic_strategy_weekly_trades.mean()

print(f"\nAverage Weekly Trades (Fixed-Threshold Strategy): {average_fixed_weekly_trades:.2f}")
print(f"Average Weekly Trades (Dynamic-Threshold Strategy): {average_dynamic_weekly_trades:.2f}")

In [ ]:
# 159.0

fixed_strategy_weekly_trades = pd.Series(index=signals_aligned.index, dtype=int)
dynamic_strategy_weekly_trades = pd.Series(index=signals_aligned_dynamic.index, dtype=int)

# Calculate weekly trade counts for the fixed-threshold strategy
for i in range(len(signals_aligned)):
    if i == 0:
        fixed_strategy_weekly_trades.iloc[i] = 0  # No trades in the first week
    else:
        current_week_signals = signals_aligned.iloc[i]
        previous_week_signals = signals_aligned.iloc[i-1]

        # Count signal changes for the current week compared to the previous week
        num_changes = (current_week_signals != previous_week_signals).sum()
        fixed_strategy_weekly_trades.iloc[i] = num_changes

# Calculate weekly trade counts for the dynamic-threshold strategy
for i in range(len(signals_aligned_dynamic)):
    if i == 0:
        dynamic_strategy_weekly_trades.iloc[i] = 0  # No trades in the first week
    else:
        current_week_signals_dynamic = signals_aligned_dynamic.iloc[i]
        previous_week_signals_dynamic = signals_aligned_dynamic.iloc[i-1]

        # Count signal changes for the current week compared to the previous week
        num_changes_dynamic = (current_week_signals_dynamic != previous_week_signals_dynamic).sum()
        dynamic_strategy_weekly_trades.iloc[i] = num_changes_dynamic


print("Fixed Strategy Weekly Trade Counts (First 5 rows):")
display(fixed_strategy_weekly_trades.head())

print("\nDynamic Strategy Weekly Trade Counts (First 5 rows):")
display(dynamic_strategy_weekly_trades.head())

In [ ]:
# 160.0

verage_fixed_weekly_trades = fixed_strategy_weekly_trades.mean()
average_dynamic_weekly_trades = dynamic_strategy_weekly_trades.mean()

print(f"\nAverage Weekly Trades (Fixed-Threshold Strategy): {average_fixed_weekly_trades:.2f}")
print(f"Average Weekly Trades (Dynamic-Threshold Strategy): {average_dynamic_weekly_trades:.2f}")


Average Weekly Trades (Fixed-Threshold Strategy): 2.22
Average Weekly Trades (Dynamic-Threshold Strategy): 2.30


In [ ]:
# 161.0

rf = 0.01 # Re-defining risk-free rate, ensuring it's available in this scope

# --- Recalculate metrics for Monthly Strategy ---
# Ensure df_sector_monthly and df_sector_monthly_pct_change are current
# Resample df_sector to a monthly frequency and calculate monthly percent changes
df_sector_monthly = df_sector.resample('M').last()
df_sector_monthly_pct_change = df_sector_monthly.pct_change().dropna()
# Compute the 3-month rolling mean of these monthly percent changes
rolling_mean_monthly_pct_change = df_sector_monthly_pct_change.rolling(window=3).mean().dropna()
# Calculate monthly relative performance
relative_performance_monthly = pd.DataFrame(index=rolling_mean_monthly_pct_change.index)
sector_etfs_monthly = [col for col in rolling_mean_monthly_pct_change.columns if col != "S&P 500"]
for sector in sector_etfs_monthly:
    relative_performance_monthly[sector] = rolling_mean_monthly_pct_change[sector] - rolling_mean_monthly_pct_change["S&P 500"]
# Define monthly 'buy', 'sell', or 'hold' signals
buy_threshold_monthly = 0.001
sell_threshold_monthly = -0.001
signals_monthly = pd.DataFrame(index=relative_performance_monthly.index)
for sector in relative_performance_monthly.columns:
    signals_monthly[sector] = 'Hold'  # Initialize all to 'Hold'
    signals_monthly.loc[relative_performance_monthly[sector] > buy_threshold_monthly, sector] = 'Buy'
    signals_monthly.loc[relative_performance_monthly[sector] < sell_threshold_monthly, sector] = 'Sell'
# Backtest this monthly trading strategy
portfolio_returns_monthly = pd.DataFrame(index=signals_monthly.index, columns=['Portfolio Return'])
num_tradeable_sectors_monthly = len(sector_etfs_monthly)
returns_aligned_monthly = df_sector_monthly_pct_change[sector_etfs_monthly].align(signals_monthly, join='inner', axis=0)[0]
signals_aligned_monthly = signals_monthly.align(returns_aligned_monthly, join='inner', axis=0)[0]
current_portfolio_weights_at_start_of_period_monthly = {sector: 1.0 / num_tradeable_sectors_monthly for sector in sector_etfs_monthly}
historical_weights_monthly = pd.DataFrame(columns=sector_etfs_monthly, index=signals_aligned_monthly.index)
for i in range(len(signals_aligned_monthly)):
    current_date = signals_aligned_monthly.index[i]
    current_signals = signals_aligned_monthly.loc[current_date]
    current_month_returns = returns_aligned_monthly.loc[current_date]
    weights_for_this_period_monthly = {sector: 0.0 for sector in sector_etfs_monthly}
    buy_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Buy']
    sell_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Sell']
    hold_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Hold']
    active_sectors_this_period = buy_sectors + hold_sectors
    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_monthly[sector] = weight_per_active_sector
    historical_weights_monthly.loc[current_date] = list(weights_for_this_period_monthly.values())
    monthly_port_return = sum(weights_for_this_period_monthly[s] * current_month_returns[s] for s in sector_etfs_monthly)
    portfolio_returns_monthly.loc[current_date, 'Portfolio Return'] = monthly_port_return
    current_portfolio_weights_at_start_of_period_monthly = weights_for_this_period_monthly
portfolio_returns_monthly['Cumulative Return'] = (1 + portfolio_returns_monthly['Portfolio Return']).cumprod() - 1

num_months_monthly = len(portfolio_returns_monthly)
num_years_monthly = num_months_monthly / 12.0

portfolio_total_cumulative_return_monthly = portfolio_returns_monthly['Cumulative Return'].iloc[-1]
portfolio_annualized_return_monthly = (1 + portfolio_total_cumulative_return_monthly)**(1/num_years_monthly) - 1
portfolio_annualized_volatility_monthly = portfolio_returns_monthly['Portfolio Return'].std() * np.sqrt(12)
portfolio_sharpe_ratio_monthly = (portfolio_annualized_return_monthly - rf) / portfolio_annualized_volatility_monthly

# --- Recalculate metrics for Fixed Strategy --- (using already calculated portfolio_returns)
num_weeks = len(portfolio_returns)
num_years = num_weeks / 52.0
portfolio_total_cumulative_return = portfolio_returns['Cumulative Return'].iloc[-1]
portfolio_annualized_return = (1 + portfolio_total_cumulative_return)**(1/num_years) - 1
portfolio_annualized_volatility = portfolio_returns['Portfolio Return'].std() * np.sqrt(52)
portfolio_sharpe_ratio = (portfolio_annualized_return - rf) / portfolio_annualized_volatility

# Recalculate S&P 500 metrics for consistency with the fixed strategy's time frame
s_p_500_returns_aligned = df_sector_pct_change["S&P 500"].align(portfolio_returns, join='inner', axis=0)[0]
s_p_500_cumulative_return = (1 + s_p_500_returns_aligned).cumprod() - 1
s_p_500_total_cumulative_return = s_p_500_cumulative_return.iloc[-1]
s_p_500_annualized_return = (1 + s_p_500_total_cumulative_return)**(1/num_years) - 1
s_p_500_annualized_volatility = s_p_500_returns_aligned.std() * np.sqrt(52)
s_p_500_sharpe_ratio = (s_p_500_annualized_return - rf) / s_p_500_annualized_volatility

# --- Recalculate metrics for Dynamic Strategy --- (using already calculated portfolio_returns_dynamic)
num_weeks_dynamic = len(portfolio_returns_dynamic)
num_years_dynamic = num_weeks_dynamic / 52.0
portfolio_total_cumulative_return_dynamic = portfolio_returns_dynamic['Cumulative Return'].iloc[-1]
portfolio_annualized_return_dynamic = (1 + portfolio_total_cumulative_return_dynamic)**(1/num_years_dynamic) - 1
portfolio_annualized_volatility_dynamic = portfolio_returns_dynamic['Portfolio Return'].std() * np.sqrt(52)
portfolio_sharpe_ratio_dynamic = (portfolio_annualized_return_dynamic - rf) / portfolio_annualized_volatility_dynamic

# --- Recalculate metrics for Dynamic Strategy with Costs --- (using already calculated portfolio_returns_dynamic_with_costs)
num_weeks_dynamic_costs = len(portfolio_returns_dynamic_with_costs)
num_years_dynamic_costs = num_weeks_dynamic_costs / 52.0
portfolio_total_cumulative_return_dynamic_costs = portfolio_returns_dynamic_with_costs['Cumulative Return'].iloc[-1]
portfolio_annualized_return_dynamic_costs = (1 + portfolio_total_cumulative_return_dynamic_costs)**(1/num_years_dynamic_costs) - 1
portfolio_annualized_volatility_dynamic_costs = portfolio_returns_dynamic_with_costs['Portfolio Return'].std() * np.sqrt(52)
portfolio_sharpe_ratio_dynamic_costs = (portfolio_annualized_return_dynamic_costs - rf) / portfolio_annualized_volatility_dynamic_costs

# Print Summary Comparison
print("\n--- Strategy Performance Comparison (Monthly vs. Dynamic Costs vs. Dynamic vs. Fixed vs. S&P 500) ---")
print(f"Metric                        | Monthly         | Dynamic Costs   | Dynamic         | Fixed Threshold | S&P 500       ")
print(f"------------------------------------------------------------------------------------------------------------------")
print(f"Total Cumulative Return       | {portfolio_total_cumulative_return_monthly:.4f}        | {portfolio_total_cumulative_return_dynamic_costs:.4f}        | {portfolio_total_cumulative_return_dynamic:.4f}         | {portfolio_total_cumulative_return:.4f}        | {s_p_500_total_cumulative_return:.4f}       ")
print(f"Annualized Return             | {portfolio_annualized_return_monthly:.4f}        | {portfolio_annualized_return_dynamic_costs:.4f}        | {portfolio_annualized_return_dynamic:.4f}         | {portfolio_annualized_return:.4f}        | {s_p_500_annualized_return:.4f}       ")
print(f"Annualized Volatility         | {portfolio_annualized_volatility_monthly:.4f}        | {portfolio_annualized_volatility_dynamic_costs:.4f}        | {portfolio_annualized_volatility_dynamic:.4f}         | {portfolio_annualized_volatility:.4f}        | {s_p_500_annualized_volatility:.4f}       ")
print(f"Sharpe Ratio                  | {portfolio_sharpe_ratio_monthly:.4f}        | {portfolio_sharpe_ratio_dynamic_costs:.4f}        | {portfolio_sharpe_ratio_dynamic:.4f}         | {portfolio_sharpe_ratio:.4f}        | {s_p_500_sharpe_ratio:.4f}       ")
print("------------------------------------------------------------------------------------------------------------------")

In [ ]:
# 162.0

rf = 0.01 # Re-defining risk-free rate, ensuring it's available in this scope

# --- Recalculate metrics for Monthly Strategy ---
# Ensure df_sector_monthly and df_sector_monthly_pct_change are current
# Resample df_sector to a monthly frequency and calculate monthly percent changes
df_sector_monthly = df_sector.resample('ME').last()
df_sector_monthly_pct_change = df_sector_monthly.pct_change().dropna()
# Compute the 3-month rolling mean of these monthly percent changes
rolling_mean_monthly_pct_change = df_sector_monthly_pct_change.rolling(window=3).mean().dropna()
# Calculate monthly relative performance
relative_performance_monthly = pd.DataFrame(index=rolling_mean_monthly_pct_change.index)
sector_etfs_monthly = [col for col in rolling_mean_monthly_pct_change.columns if col != "S&P 500"]
for sector in sector_etfs_monthly:
    relative_performance_monthly[sector] = rolling_mean_monthly_pct_change[sector] - rolling_mean_monthly_pct_change["S&P 500"]
# Define monthly 'buy', 'sell', or 'hold' signals
buy_threshold_monthly = 0.001
sell_threshold_monthly = -0.001
signals_monthly = pd.DataFrame(index=relative_performance_monthly.index)
for sector in relative_performance_monthly.columns:
    signals_monthly[sector] = 'Hold'  # Initialize all to 'Hold'
    signals_monthly.loc[relative_performance_monthly[sector] > buy_threshold_monthly, sector] = 'Buy'
    signals_monthly.loc[relative_performance_monthly[sector] < sell_threshold_monthly, sector] = 'Sell'
# Backtest this monthly trading strategy
portfolio_returns_monthly = pd.DataFrame(index=signals_monthly.index, columns=['Portfolio Return'])
num_tradeable_sectors_monthly = len(sector_etfs_monthly)
returns_aligned_monthly = df_sector_monthly_pct_change[sector_etfs_monthly].align(signals_monthly, join='inner', axis=0)[0]
signals_aligned_monthly = signals_monthly.align(returns_aligned_monthly, join='inner', axis=0)[0]
current_portfolio_weights_at_start_of_period_monthly = {sector: 1.0 / num_tradeable_sectors_monthly for sector in sector_etfs_monthly}
historical_weights_monthly = pd.DataFrame(columns=sector_etfs_monthly, index=signals_aligned_monthly.index)
for i in range(len(signals_aligned_monthly)):
    current_date = signals_aligned_monthly.index[i]
    current_signals = signals_aligned_monthly.loc[current_date]
    current_month_returns = returns_aligned_monthly.loc[current_date]
    weights_for_this_period_monthly = {sector: 0.0 for sector in sector_etfs_monthly}
    buy_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Buy']
    sell_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Sell']
    hold_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Hold']
    active_sectors_this_period = buy_sectors + hold_sectors
    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_monthly[sector] = weight_per_active_sector
    historical_weights_monthly.loc[current_date] = list(weights_for_this_period_monthly.values())
    monthly_port_return = sum(weights_for_this_period_monthly[s] * current_month_returns[s] for s in sector_etfs_monthly)
    portfolio_returns_monthly.loc[current_date, 'Portfolio Return'] = monthly_port_return
    current_portfolio_weights_at_start_of_period_monthly = weights_for_this_period_monthly
portfolio_returns_monthly['Cumulative Return'] = (1 + portfolio_returns_monthly['Portfolio Return']).cumprod() - 1

num_months_monthly = len(portfolio_returns_monthly)
num_years_monthly = num_months_monthly / 12.0

portfolio_total_cumulative_return_monthly = portfolio_returns_monthly['Cumulative Return'].iloc[-1]
portfolio_annualized_return_monthly = (1 + portfolio_total_cumulative_return_monthly)**(1/num_years_monthly) - 1
portfolio_annualized_volatility_monthly = portfolio_returns_monthly['Portfolio Return'].std() * np.sqrt(12)
portfolio_sharpe_ratio_monthly = (portfolio_annualized_return_monthly - rf) / portfolio_annualized_volatility_monthly

# --- Recalculate metrics for Fixed Strategy --- (using already calculated portfolio_returns)
num_weeks = len(portfolio_returns)
num_years = num_weeks / 52.0
portfolio_total_cumulative_return = portfolio_returns['Cumulative Return'].iloc[-1]
portfolio_annualized_return = (1 + portfolio_total_cumulative_return)**(1/num_years) - 1
portfolio_annualized_volatility = portfolio_returns['Portfolio Return'].std() * np.sqrt(52)
portfolio_sharpe_ratio = (portfolio_annualized_return - rf) / portfolio_annualized_volatility

# Recalculate S&P 500 metrics for consistency with the fixed strategy's time frame
s_p_500_returns_aligned = df_sector_pct_change["S&P 500"].align(portfolio_returns, join='inner', axis=0)[0]
s_p_500_cumulative_return = (1 + s_p_500_returns_aligned).cumprod() - 1
s_p_500_total_cumulative_return = s_p_500_cumulative_return.iloc[-1]
s_p_500_annualized_return = (1 + s_p_500_total_cumulative_return)**(1/num_years) - 1
s_p_500_annualized_volatility = s_p_500_returns_aligned.std() * np.sqrt(52)
s_p_500_sharpe_ratio = (s_p_500_annualized_return - rf) / s_p_500_annualized_volatility

# --- Recalculate metrics for Dynamic Strategy --- (using already calculated portfolio_returns_dynamic)
num_weeks_dynamic = len(portfolio_returns_dynamic)
num_years_dynamic = num_weeks_dynamic / 52.0
portfolio_total_cumulative_return_dynamic = portfolio_returns_dynamic['Cumulative Return'].iloc[-1]
portfolio_annualized_return_dynamic = (1 + portfolio_total_cumulative_return_dynamic)**(1/num_years_dynamic) - 1
portfolio_annualized_volatility_dynamic = portfolio_returns_dynamic['Portfolio Return'].std() * np.sqrt(52)
portfolio_sharpe_ratio_dynamic = (portfolio_annualized_return_dynamic - rf) / portfolio_annualized_volatility_dynamic

# --- Recalculate metrics for Dynamic Strategy with Costs --- (using already calculated portfolio_returns_dynamic_with_costs)
num_weeks_dynamic_costs = len(portfolio_returns_dynamic_with_costs)
num_years_dynamic_costs = num_weeks_dynamic_costs / 52.0
portfolio_total_cumulative_return_dynamic_costs = portfolio_returns_dynamic_with_costs['Cumulative Return'].iloc[-1]
portfolio_annualized_return_dynamic_costs = (1 + portfolio_total_cumulative_return_dynamic_costs)**(1/num_years_dynamic_costs) - 1
portfolio_annualized_volatility_dynamic_costs = portfolio_returns_dynamic_with_costs['Portfolio Return'].std() * np.sqrt(52)
portfolio_sharpe_ratio_dynamic_costs = (portfolio_annualized_return_dynamic_costs - rf) / portfolio_annualized_volatility_dynamic_costs

# Print Summary Comparison
print("\n--- Strategy Performance Comparison (Monthly vs. Dynamic Costs vs. Dynamic vs. Fixed vs. S&P 500) ---")
print(f"Metric                        | Monthly         | Dynamic Costs   | Dynamic         | Fixed Threshold | S&P 500       ")
print(f"------------------------------------------------------------------------------------------------------------------")
print(f"Total Cumulative Return       | {portfolio_total_cumulative_return_monthly:.4f}        | {portfolio_total_cumulative_return_dynamic_costs:.4f}        | {portfolio_total_cumulative_return_dynamic:.4f}         | {portfolio_total_cumulative_return:.4f}        | {s_p_500_total_cumulative_return:.4f}       ")
print(f"Annualized Return             | {portfolio_annualized_return_monthly:.4f}        | {portfolio_annualized_return_dynamic_costs:.4f}        | {portfolio_annualized_return_dynamic:.4f}         | {portfolio_annualized_return:.4f}        | {s_p_500_annualized_return:.4f}       ")
print(f"Annualized Volatility         | {portfolio_annualized_volatility_monthly:.4f}        | {portfolio_annualized_volatility_dynamic_costs:.4f}        | {portfolio_annualized_volatility_dynamic:.4f}         | {portfolio_annualized_volatility:.4f}        | {s_p_500_annualized_volatility:.4f}       ")
print(f"Sharpe Ratio                  | {portfolio_sharpe_ratio_monthly:.4f}        | {portfolio_sharpe_ratio_dynamic_costs:.4f}        | {portfolio_sharpe_ratio_dynamic:.4f}         | {portfolio_sharpe_ratio:.4f}        | {s_p_500_sharpe_ratio:.4f}       ")
print("------------------------------------------------------------------------------------------------------------------")

**Reasoning**:
Now that all performance metrics have been recalculated and displayed, I need to generate the comparative cumulative returns plot as per the third instruction of the subtask. This involves plotting the cumulative returns of the Monthly Strategy, Fixed Strategy, Dynamic Strategy (without costs), Dynamic Strategy (with costs), and the S&P 500 on a single chart with appropriate labels, title, grid, and legend.



In [ ]:
# 163.0

import matplotlib.pyplot as plt

plt.figure(figsize=(14, 7))
plt.plot(portfolio_returns_monthly.index, portfolio_returns_monthly['Cumulative Return'], label='Monthly Strategy Cumulative Return', color='orange')
plt.plot(portfolio_returns.index, portfolio_returns['Cumulative Return'], label='Fixed Strategy Cumulative Return', color='blue')
plt.plot(portfolio_returns_dynamic.index, portfolio_returns_dynamic['Cumulative Return'], label='Dynamic Strategy Cumulative Return (no costs)', color='green')
plt.plot(portfolio_returns_dynamic_with_costs.index, portfolio_returns_dynamic_with_costs['Cumulative Return'], label='Dynamic Strategy Cumulative Return (with costs)', color='purple')
plt.plot(s_p_500_cumulative_return.index, s_p_500_cumulative_return, label='S&P 500 Cumulative Return', color='red')
plt.title('Cumulative Returns: All Strategies vs. S&P 500 (Including XLRE)')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend()
plt.show()

In [ ]:
# 164.0

import pandas as pd
from fredapi import Fred

# FRED API key - Ensure this is correctly set or retrieved from a secure source.
# For demonstration, using the API key from the provided notebook context.
fred = Fred(api_key='e8fb9c414f6ab8f1a06a5472466a31dc')

# Define date range for weekly index, consistent with previous data fetches
start_date = "2010-01-01"
end_date = "2025-12-30"
weekly_index = pd.date_range(start=start_date, end=end_date, freq='W-FRI').strftime('%Y-%m-%d')

# Define FRED series IDs and their corresponding labels
series_dict = {
    'GDP': 'GDP',
    'Inflation (CPI)': 'CPIAUCSL',
    'Unemployment Rate': 'UNRATE',
    'Interest Rate (Fed Funds)': 'FEDFUNDS'
}

# Create a DataFrame with weekly index to store macroeconomic data
macro_data = pd.DataFrame(index=pd.to_datetime(weekly_index))
macro_data.index.name = "Date"

# Fetch and interpolate data for each series
for label, series_id in series_dict.items():
    data = fred.get_series(series_id)
    if data is None: # Handle cases where FRED API might return None
        print(f"Warning: Could not fetch series {series_id} for {label}")
        continue

    data = data.to_frame(name=label)
    data.index = pd.to_datetime(data.index)

    # Resample to daily, interpolating or forward-filling as specified
    if label == 'Interest Rate (Fed Funds)':
        daily = data.resample('D').ffill()
    else:
        # For other series, interpolate missing values to daily frequency
        daily = data.resample('D').interpolate(method='linear')

    # Reindex the daily data to the predefined weekly index and interpolate remaining missing values
    # Use limit_direction='both' for interpolation to handle NaNs at ends of series
    weekly_processed = daily.reindex(macro_data.index).interpolate(method='linear', limit_direction='both')
    macro_data[label] = weekly_processed[label]

# Save the completed macroeconomic DataFrame to a CSV file
macro_data.to_csv("Macroeconomic_Data.csv", index=True)

print("Macroeconomic Data fetched, processed, and saved successfully to 'Macroeconomic_Data.csv'!")

In [ ]:
# 165.0

df_sector = df_sector.resample('W-FRI').last()
df_sector = df_sector.ffill().bfill() # Ensure no NaNs after resampling

# Merge df_sector and df_macro into a new DataFrame 'df'
df = df_sector.join(df_macro, how="inner")

print("df_macro loaded and df created successfully!")
print("First 5 rows of merged df:")
display(df.head())

print("Missing values in the combined DataFrame (df) after join:")
display(df.isnull().sum())

In [ ]:
# 166.0

df_sector_pct_change = df_sector.pct_change().dropna()

rolling_mean_pct_change = df_sector_pct_change.rolling(window=12).mean().dropna()

relative_performance = pd.DataFrame(index=rolling_mean_pct_change.index)

sector_etfs = [col for col in rolling_mean_pct_change.columns if col != "S&P 500"]

for sector in sector_etfs:
    relative_performance[sector] = rolling_mean_pct_change[sector] - rolling_mean_pct_change["S&P 500"]

print("df_sector_pct_change, rolling_mean_pct_change, and relative_performance recalculated successfully!")
print("First 5 rows of relative_performance:")
display(relative_performance.head())

In [ ]:
# 167.0

buy_threshold = 0.001
sell_threshold = -0.001

signals = pd.DataFrame(index=relative_performance.index)

for sector in relative_performance.columns:
    signals[sector] = 'Hold'  # Initialize all to 'Hold'
    signals.loc[relative_performance[sector] > buy_threshold, sector] = 'Buy'
    signals.loc[relative_performance[sector] < sell_threshold, sector] = 'Sell'

print("Fixed-threshold signals DataFrame regenerated successfully!")
print("First 5 rows of signals:")
display(signals.head())

# Re-run the backtesting simulation for the fixed-threshold strategy
# 1. Initialize an empty DataFrame called portfolio_returns
portfolio_returns = pd.DataFrame(index=signals.index, columns=['Portfolio Return'])

# Get list of sector ETFs (excluding S&P 500)
sector_etfs = [col for col in df_sector_pct_change.columns if col != "S&P 500"]
num_tradeable_sectors = len(sector_etfs)

# Align returns and signals to ensure they cover the same time period
# This ensures both DataFrames start and end on the same dates.
returns_aligned = df_sector_pct_change[sector_etfs].align(signals, join='inner', axis=0)[0]
signals_aligned = signals.align(returns_aligned, join='inner', axis=0)[0]

# 2. Initialize current_portfolio_weights_at_start_of_period
# These are the weights assumed at the start of the *first* period for which we have signals.
current_portfolio_weights_at_start_of_period = {sector: 1.0 / num_tradeable_sectors for sector in sector_etfs}

# Create a DataFrame to store historical weights for analysis (optional, but good for inspection)
historical_weights = pd.DataFrame(columns=sector_etfs, index=signals_aligned.index)

# 3. Iterate through each time step (row) in the aligned signals DataFrame
for i in range(len(signals_aligned)):
    current_date = signals_aligned.index[i]
    current_signals = signals_aligned.loc[current_date] # Signals for the current period
    current_week_returns = returns_aligned.loc[current_date] # Returns for the current period

    # `weights_for_this_period` will hold the actual weights used for the current period's return calculation.
    # Initialize all to 0.0, and then assign based on signals.
    weights_for_this_period = {sector: 0.0 for sector in sector_etfs}

    buy_sectors = [s for s in sector_etfs if current_signals[s] == 'Buy']
    sell_sectors = [s for s in sector_etfs if current_signals[s] == 'Sell']
    hold_sectors = [s for s in sector_etfs if current_signals[s] == 'Hold']

    # Determine the 'active' sectors for this period (those not getting a 'Sell' signal).
    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period[sector] = weight_per_active_sector
        # Sectors not in active_sectors_this_period (i.e., sell_sectors) implicitly retain 0.0 weight.
    else:
        # If all sectors are 'Sell', or no active signals, all weights remain 0.0.
        # Portfolio return for this week will be 0.0.
        pass # weights_for_this_period is already initialized to 0.0

    # Store the weights used for this period calculation in the historical_weights DataFrame.
    historical_weights.loc[current_date] = list(weights_for_this_period.values())

    # Calculate portfolio return for the current week using the determined weights.
    weekly_port_return = sum(weights_for_this_period[s] * current_week_returns[s] for s in sector_etfs)
    portfolio_returns.loc[current_date, 'Portfolio Return'] = weekly_port_return

    # Update `current_portfolio_weights_at_start_of_period` for the *next* iteration.
    # This reflects the portfolio composition at the end of the current period, which is the start of the next.
    current_portfolio_weights_at_start_of_period = weights_for_this_period

# 4. Calculate the cumulative returns of the portfolio_returns
# (1 + Return) cumprod() - 1 gives the cumulative return based on compounding.
portfolio_returns['Cumulative Return'] = (1 + portfolio_returns['Portfolio Return']).cumprod() - 1

print("Fixed-threshold backtesting simulation complete.")
print("First 5 rows of portfolio_returns:")
display(portfolio_returns.head())

In [ ]:
# 168.0

rolling_std_relative_performance = relative_performance.rolling(window=12).std()

multiplier = 0.5 # Adjust this multiplier as needed

dynamic_buy_thresholds = multiplier * rolling_std_relative_performance
dynamic_sell_thresholds = -multiplier * rolling_std_relative_performance

signals_dynamic = pd.DataFrame(index=relative_performance.index)

for sector in relative_performance.columns:
    signals_dynamic[sector] = 'Hold'  # Initialize all to 'Hold'
    # Apply 'Buy' signal where relative performance is above the dynamic buy threshold
    buy_condition = relative_performance[sector] > dynamic_buy_thresholds[sector]
    signals_dynamic.loc[buy_condition, sector] = 'Buy'

    # Apply 'Sell' signal where relative performance is below the dynamic sell threshold
    sell_condition = relative_performance[sector] < dynamic_sell_thresholds[sector]
    signals_dynamic.loc[sell_condition, sector] = 'Sell'

print("Dynamic signals DataFrame regenerated successfully!")
print("First 5 rows of signals_dynamic:")
display(signals_dynamic.head())

# Re-run the backtesting simulation for the dynamic-threshold strategy without costs
portfolio_returns_dynamic = pd.DataFrame(index=signals_dynamic.index, columns=['Portfolio Return'])

sector_etfs = [col for col in df_sector_pct_change.columns if col != "S&P 500"]
num_tradeable_sectors = len(sector_etfs)

returns_aligned_dynamic = df_sector_pct_change[sector_etfs].align(signals_dynamic, join='inner', axis=0)[0]
signals_aligned_dynamic = signals_dynamic.align(returns_aligned_dynamic, join='inner', axis=0)[0]

valid_indices = signals_aligned_dynamic.dropna(how='all').index
signals_aligned_dynamic = signals_aligned_dynamic.loc[valid_indices]
returns_aligned_dynamic = returns_aligned_dynamic.loc[valid_indices]
portfolio_returns_dynamic = portfolio_returns_dynamic.loc[valid_indices]

current_portfolio_weights_at_start_of_period_dynamic = {sector: 1.0 / num_tradeable_sectors for sector in sector_etfs}

historical_weights_dynamic = pd.DataFrame(columns=sector_etfs, index=signals_aligned_dynamic.index)

for i in range(len(signals_aligned_dynamic)):
    current_date = signals_aligned_dynamic.index[i]
    current_signals = signals_aligned_dynamic.loc[current_date]
    current_week_returns = returns_aligned_dynamic.loc[current_date]

    weights_for_this_period_dynamic = {sector: 0.0 for sector in sector_etfs}

    buy_sectors = [s for s in sector_etfs if current_signals[s] == 'Buy']
    sell_sectors = [s for s in sector_etfs if current_signals[s] == 'Sell']
    hold_sectors = [s for s in sector_etfs if current_signals[s] == 'Hold']

    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_dynamic[sector] = weight_per_active_sector

    historical_weights_dynamic.loc[current_date] = list(weights_for_this_period_dynamic.values())

    weekly_port_return = sum(weights_for_this_period_dynamic[s] * current_week_returns[s] for s in sector_etfs)
    portfolio_returns_dynamic.loc[current_date, 'Portfolio Return'] = weekly_port_return

    current_portfolio_weights_at_start_of_period_dynamic = weights_for_this_period_dynamic

portfolio_returns_dynamic['Cumulative Return'] = (1 + portfolio_returns_dynamic['Portfolio Return']).cumprod() - 1

print("Dynamic portfolio simulation (without costs) complete.")
print("First 5 rows of portfolio_returns_dynamic:")
display(portfolio_returns_dynamic.head())

# Re-run the backtesting simulation for the dynamic-threshold strategy with transaction costs
portfolio_returns_dynamic_with_costs = pd.DataFrame(index=signals_aligned_dynamic.index, columns=['Portfolio Return'])

transaction_cost_rate = 0.001  # 0.1% transaction cost

previous_period_weights = {sector: 1.0 / num_tradeable_sectors for sector in sector_etfs}

for i in range(len(signals_aligned_dynamic)):
    current_date = signals_aligned_dynamic.index[i]
    current_signals = signals_aligned_dynamic.loc[current_date] # Signals for the current period
    current_week_returns = returns_aligned_dynamic.loc[current_date] # Returns for the current period

    # Determine the weights_for_this_period_dynamic
    weights_for_this_period_dynamic = {sector: 0.0 for sector in sector_etfs}
    buy_sectors = [s for s in sector_etfs if current_signals[s] == 'Buy']
    hold_sectors = [s for s in sector_etfs if current_signals[s] == 'Hold']
    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_dynamic[sector] = weight_per_active_sector

    # Calculate transaction_costs for this period
    transaction_costs = 0.0
    for sector in sector_etfs:
        weight_change = abs(weights_for_this_period_dynamic[sector] - previous_period_weights[sector])
        transaction_costs += weight_change * transaction_cost_rate

    # Calculate weekly_port_return_before_costs
    weekly_port_return_before_costs = sum(weights_for_this_period_dynamic[s] * current_week_returns[s] for s in sector_etfs)

    # Deduct transaction_costs
    weekly_port_return_after_costs = weekly_port_return_before_costs - transaction_costs
    portfolio_returns_dynamic_with_costs.loc[current_date, 'Portfolio Return'] = weekly_port_return_after_costs

    # Update previous_period_weights for the next iteration
    previous_period_weights = weights_for_this_period_dynamic

portfolio_returns_dynamic_with_costs['Cumulative Return'] = (1 + portfolio_returns_dynamic_with_costs['Portfolio Return']).cumprod() - 1

print("Dynamic portfolio simulation with transaction costs complete.")
print("First 5 rows of portfolio_returns_dynamic_with_costs:")
display(portfolio_returns_dynamic_with_costs.head())

In [ ]:
# 169.0

import pandas as pd
import numpy as np

# 1. Resample df_sector to a monthly frequency and calculate monthly percent changes
df_sector_monthly = df_sector.resample('ME').last()
df_sector_monthly_pct_change = df_sector_monthly.pct_change().dropna()

# 2. Compute the 3-month rolling mean of these monthly percent changes
rolling_mean_monthly_pct_change = df_sector_monthly_pct_change.rolling(window=3).mean().dropna()

print("Monthly data re-preparation complete!")
print("First 5 rows of df_sector_monthly:")
display(df_sector_monthly.head())
print("\nFirst 5 rows of df_sector_monthly_pct_change:")
display(df_sector_monthly_pct_change.head())
print("\nFirst 5 rows of rolling_mean_monthly_pct_change:")
display(rolling_mean_monthly_pct_change.head())

# 3. Calculate monthly relative performance
relative_performance_monthly = pd.DataFrame(index=rolling_mean_monthly_pct_change.index)
sector_etfs_monthly = [col for col in rolling_mean_monthly_pct_change.columns if col != "S&P 500"]

for sector in sector_etfs_monthly:
    relative_performance_monthly[sector] = rolling_mean_monthly_pct_change[sector] - rolling_mean_monthly_pct_change["S&P 500"]

print("\nMonthly relative performance DataFrame re-created successfully!")
print("First 5 rows of relative_performance_monthly:")
display(relative_performance_monthly.head())

# 4. Define monthly 'buy', 'sell', or 'hold' signals
buy_threshold_monthly = 0.001
sell_threshold_monthly = -0.001

signals_monthly = pd.DataFrame(index=relative_performance_monthly.index)

for sector in relative_performance_monthly.columns:
    signals_monthly[sector] = 'Hold'  # Initialize all to 'Hold'
    signals_monthly.loc[relative_performance_monthly[sector] > buy_threshold_monthly, sector] = 'Buy'
    signals_monthly.loc[relative_performance_monthly[sector] < sell_threshold_monthly, sector] = 'Sell'

print("\nMonthly signals DataFrame re-created successfully!")
print("First 5 rows of signals_monthly:")
display(signals_monthly.head())

# 5. Backtest this monthly trading strategy
portfolio_returns_monthly = pd.DataFrame(index=signals_monthly.index, columns=['Portfolio Return'])

num_tradeable_sectors_monthly = len(sector_etfs_monthly)

returns_aligned_monthly = df_sector_monthly_pct_change[sector_etfs_monthly].align(signals_monthly, join='inner', axis=0)[0]
signals_aligned_monthly = signals_monthly.align(returns_aligned_monthly, join='inner', axis=0)[0]

current_portfolio_weights_at_start_of_period_monthly = {sector: 1.0 / num_tradeable_sectors_monthly for sector in sector_etfs_monthly}

historical_weights_monthly = pd.DataFrame(columns=sector_etfs_monthly, index=signals_aligned_monthly.index)

for i in range(len(signals_aligned_monthly)):
    current_date = signals_aligned_monthly.index[i]
    current_signals = signals_aligned_monthly.loc[current_date]
    current_month_returns = returns_aligned_monthly.loc[current_date]

    weights_for_this_period_monthly = {sector: 0.0 for sector in sector_etfs_monthly}

    buy_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Buy']
    sell_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Sell']
    hold_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Hold']

    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_monthly[sector] = weight_per_active_sector
    else:
        pass # weights_for_this_period_monthly is already initialized to 0.0

    historical_weights_monthly.loc[current_date] = list(weights_for_this_period_monthly.values())

    monthly_port_return = sum(weights_for_this_period_monthly[s] * current_month_returns[s] for s in sector_etfs_monthly)
    portfolio_returns_monthly.loc[current_date, 'Portfolio Return'] = monthly_port_return

    current_portfolio_weights_at_start_of_period_monthly = weights_for_this_period_monthly

# 4. Calculate the cumulative returns of the portfolio_returns_monthly
# (1 + Return) cumprod() - 1 gives the cumulative return based on compounding.
portfolio_returns_monthly['Cumulative Return'] = (1 + portfolio_returns_monthly['Portfolio Return']).cumprod() - 1

print("\nMonthly portfolio simulation re-complete.")
print("First 5 rows of portfolio_returns_monthly:")
display(portfolio_returns_monthly.head())
print("\nLast 5 rows of portfolio_returns_monthly:")
display(portfolio_returns_monthly.tail())

In [ ]:
# 170.0

rf = 0.01 # Re-defining risk-free rate, ensuring it's available in this scope

# --- Recalculate metrics for Monthly Strategy ---
# Ensure df_sector_monthly and df_sector_monthly_pct_change are current
# Resample df_sector to a monthly frequency and calculate monthly percent changes
df_sector_monthly = df_sector.resample('ME').last()
df_sector_monthly_pct_change = df_sector_monthly.pct_change().dropna()
# Compute the 3-month rolling mean of these monthly percent changes
rolling_mean_monthly_pct_change = df_sector_monthly_pct_change.rolling(window=3).mean().dropna()
# Calculate monthly relative performance
relative_performance_monthly = pd.DataFrame(index=rolling_mean_monthly_pct_change.index)
sector_etfs_monthly = [col for col in rolling_mean_monthly_pct_change.columns if col != "S&P 500"]
for sector in sector_etfs_monthly:
    relative_performance_monthly[sector] = rolling_mean_monthly_pct_change[sector] - rolling_mean_monthly_pct_change["S&P 500"]
# Define monthly 'buy', 'sell', or 'hold' signals
buy_threshold_monthly = 0.001
sell_threshold_monthly = -0.001
signals_monthly = pd.DataFrame(index=relative_performance_monthly.index)
for sector in relative_performance_monthly.columns:
    signals_monthly[sector] = 'Hold'  # Initialize all to 'Hold'
    signals_monthly.loc[relative_performance_monthly[sector] > buy_threshold_monthly, sector] = 'Buy'
    signals_monthly.loc[relative_performance_monthly[sector] < sell_threshold_monthly, sector] = 'Sell'
# Backtest this monthly trading strategy
portfolio_returns_monthly = pd.DataFrame(index=signals_monthly.index, columns=['Portfolio Return'])
num_tradeable_sectors_monthly = len(sector_etfs_monthly)
returns_aligned_monthly = df_sector_monthly_pct_change[sector_etfs_monthly].align(signals_monthly, join='inner', axis=0)[0]
signals_aligned_monthly = signals_monthly.align(returns_aligned_monthly, join='inner', axis=0)[0]
current_portfolio_weights_at_start_of_period_monthly = {sector: 1.0 / num_tradeable_sectors_monthly for sector in sector_etfs_monthly}
historical_weights_monthly = pd.DataFrame(columns=sector_etfs_monthly, index=signals_aligned_monthly.index)
for i in range(len(signals_aligned_monthly)):
    current_date = signals_aligned_monthly.index[i]
    current_signals = signals_aligned_monthly.loc[current_date]
    current_month_returns = returns_aligned_monthly.loc[current_date]
    weights_for_this_period_monthly = {sector: 0.0 for sector in sector_etfs_monthly}
    buy_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Buy']
    sell_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Sell']
    hold_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Hold']
    active_sectors_this_period = buy_sectors + hold_sectors
    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_monthly[sector] = weight_per_active_sector
    historical_weights_monthly.loc[current_date] = list(weights_for_this_period_monthly.values())
    monthly_port_return = sum(weights_for_this_period_monthly[s] * current_month_returns[s] for s in sector_etfs_monthly)
    portfolio_returns_monthly.loc[current_date, 'Portfolio Return'] = monthly_port_return
    current_portfolio_weights_at_start_of_period_monthly = weights_for_this_period_monthly
portfolio_returns_monthly['Cumulative Return'] = (1 + portfolio_returns_monthly['Portfolio Return']).cumprod() - 1

num_months_monthly = len(portfolio_returns_monthly)
num_years_monthly = num_months_monthly / 12.0

portfolio_total_cumulative_return_monthly = portfolio_returns_monthly['Cumulative Return'].iloc[-1]
portfolio_annualized_return_monthly = (1 + portfolio_total_cumulative_return_monthly)**(1/num_years_monthly) - 1
portfolio_annualized_volatility_monthly = portfolio_returns_monthly['Portfolio Return'].std() * np.sqrt(12)
portfolio_sharpe_ratio_monthly = (portfolio_annualized_return_monthly - rf) / portfolio_annualized_volatility_monthly

# --- Recalculate metrics for Fixed Strategy --- (using already calculated portfolio_returns)
num_weeks = len(portfolio_returns)
num_years = num_weeks / 52.0
portfolio_total_cumulative_return = portfolio_returns['Cumulative Return'].iloc[-1]
portfolio_annualized_return = (1 + portfolio_total_cumulative_return)**(1/num_years) - 1
portfolio_annualized_volatility = portfolio_returns['Portfolio Return'].std() * np.sqrt(52)
portfolio_sharpe_ratio = (portfolio_annualized_return - rf) / portfolio_annualized_volatility

# Recalculate S&P 500 metrics for consistency with the fixed strategy's time frame
s_p_500_returns_aligned = df_sector_pct_change["S&P 500"].align(portfolio_returns, join='inner', axis=0)[0]
s_p_500_cumulative_return = (1 + s_p_500_returns_aligned).cumprod() - 1
s_p_500_total_cumulative_return = s_p_500_cumulative_return.iloc[-1]
s_p_500_annualized_return = (1 + s_p_500_total_cumulative_return)**(1/num_years) - 1
s_p_500_annualized_volatility = s_p_500_returns_aligned.std() * np.sqrt(52)
s_p_500_sharpe_ratio = (s_p_500_annualized_return - rf) / s_p_500_annualized_volatility

# --- Recalculate metrics for Dynamic Strategy --- (using already calculated portfolio_returns_dynamic)
num_weeks_dynamic = len(portfolio_returns_dynamic)
num_years_dynamic = num_weeks_dynamic / 52.0
portfolio_total_cumulative_return_dynamic = portfolio_returns_dynamic['Cumulative Return'].iloc[-1]
portfolio_annualized_return_dynamic = (1 + portfolio_total_cumulative_return_dynamic)**(1/num_years_dynamic) - 1
portfolio_annualized_volatility_dynamic = portfolio_returns_dynamic['Portfolio Return'].std() * np.sqrt(52)
portfolio_sharpe_ratio_dynamic = (portfolio_annualized_return_dynamic - rf) / portfolio_annualized_volatility_dynamic

# --- Recalculate metrics for Dynamic Strategy with Costs --- (using already calculated portfolio_returns_dynamic_with_costs)
num_weeks_dynamic_costs = len(portfolio_returns_dynamic_with_costs)
num_years_dynamic_costs = num_weeks_dynamic_costs / 52.0
portfolio_total_cumulative_return_dynamic_costs = portfolio_returns_dynamic_with_costs['Cumulative Return'].iloc[-1]
portfolio_annualized_return_dynamic_costs = (1 + portfolio_total_cumulative_return_dynamic_costs)**(1/num_years_dynamic_costs) - 1
portfolio_annualized_volatility_dynamic_costs = portfolio_returns_dynamic_with_costs['Portfolio Return'].std() * np.sqrt(52)
portfolio_sharpe_ratio_dynamic_costs = (portfolio_annualized_return_dynamic_costs - rf) / portfolio_annualized_volatility_dynamic_costs

# Print Summary Comparison
print("\n--- Strategy Performance Comparison (Monthly vs. Dynamic Costs vs. Dynamic vs. Fixed vs. S&P 500) ---")
print(f"Metric                        | Monthly         | Dynamic Costs   | Dynamic         | Fixed Threshold | S&P 500       ")
print(f"------------------------------------------------------------------------------------------------------------------")
print(f"Total Cumulative Return       | {portfolio_total_cumulative_return_monthly:.4f}        | {portfolio_total_cumulative_return_dynamic_costs:.4f}        | {portfolio_total_cumulative_return_dynamic:.4f}         | {portfolio_total_cumulative_return:.4f}        | {s_p_500_total_cumulative_return:.4f}       ")
print(f"Annualized Return             | {portfolio_annualized_return_monthly:.4f}        | {portfolio_annualized_return_dynamic_costs:.4f}        | {portfolio_annualized_return_dynamic:.4f}         | {portfolio_annualized_return:.4f}        | {s_p_500_annualized_return:.4f}       ")
print(f"Annualized Volatility         | {portfolio_annualized_volatility_monthly:.4f}        | {portfolio_annualized_volatility_dynamic_costs:.4f}        | {portfolio_annualized_volatility_dynamic:.4f}         | {portfolio_annualized_volatility:.4f}        | {s_p_500_annualized_volatility:.4f}       ")
print(f"Sharpe Ratio                  | {portfolio_sharpe_ratio_monthly:.4f}        | {portfolio_sharpe_ratio_dynamic_costs:.4f}        | {portfolio_sharpe_ratio_dynamic:.4f}         | {portfolio_sharpe_ratio:.4f}        | {s_p_500_sharpe_ratio:.4f}       ")
print("------------------------------------------------------------------------------------------------------------------")

**Reasoning**:
Now that all performance metrics have been recalculated and displayed, I need to generate the comparative cumulative returns plot as per the third instruction of the subtask. This involves plotting the cumulative returns of the Monthly Strategy, Fixed Strategy, Dynamic Strategy (without costs), Dynamic Strategy (with costs), and the S&P 500 on a single chart with appropriate labels, title, grid, and legend.



In [ ]:
# 171.0

import matplotlib.pyplot as plt

plt.figure(figsize=(14, 7))
plt.plot(portfolio_returns_monthly.index, portfolio_returns_monthly['Cumulative Return'], label='Monthly Strategy Cumulative Return', color='orange')
plt.plot(portfolio_returns.index, portfolio_returns['Cumulative Return'], label='Fixed Strategy Cumulative Return', color='blue')
plt.plot(portfolio_returns_dynamic.index, portfolio_returns_dynamic['Cumulative Return'], label='Dynamic Strategy Cumulative Return (no costs)', color='green')
plt.plot(portfolio_returns_dynamic_with_costs.index, portfolio_returns_dynamic_with_costs['Cumulative Return'], label='Dynamic Strategy Cumulative Return (with costs)', color='purple')
plt.plot(s_p_500_cumulative_return.index, s_p_500_cumulative_return, label='S&P 500 Cumulative Return', color='red')
plt.title('Cumulative Returns: All Strategies vs. S&P 500 (Including XLRE)')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend()
plt.show()

In [ ]:
# 171.0

buy_threshold = 0.001
sell_threshold = -0.001

signals = pd.DataFrame(index=relative_performance.index)

for sector in relative_performance.columns:
    signals[sector] = 'Hold'  # Initialize all to 'Hold'
    signals.loc[relative_performance[sector] > buy_threshold, sector] = 'Buy'
    signals.loc[relative_performance[sector] < sell_threshold, sector] = 'Sell'

print("Fixed-threshold signals DataFrame regenerated successfully!")
print("First 5 rows of signals:")
display(signals.head())

# Re-run the backtesting simulation for the fixed-threshold strategy
# 1. Initialize an empty DataFrame called portfolio_returns
portfolio_returns = pd.DataFrame(index=signals.index, columns=['Portfolio Return'])

# Get list of sector ETFs (excluding S&P 500)
sector_etfs = [col for col in df_sector_pct_change.columns if col != "S&P 500"]
num_tradeable_sectors = len(sector_etfs)

# Align returns and signals to ensure they cover the same time period
# This ensures both DataFrames start and end on the same dates.
returns_aligned = df_sector_pct_change[sector_etfs].align(signals, join='inner', axis=0)[0]
signals_aligned = signals.align(returns_aligned, join='inner', axis=0)[0]

# 2. Initialize current_portfolio_weights_at_start_of_period
# These are the weights assumed at the start of the *first* period for which we have signals.
current_portfolio_weights_at_start_of_period = {sector: 1.0 / num_tradeable_sectors for sector in sector_etfs}

# Create a DataFrame to store historical weights for analysis (optional, but good for inspection)
historical_weights = pd.DataFrame(columns=sector_etfs, index=signals_aligned.index)

# 3. Iterate through each time step (row) in the aligned signals DataFrame
for i in range(len(signals_aligned)):
    current_date = signals_aligned.index[i]
    current_signals = signals_aligned.loc[current_date] # Signals for the current period
    current_week_returns = returns_aligned.loc[current_date] # Returns for the current period

    # `weights_for_this_period` will hold the actual weights used for the current period's return calculation.
    # Initialize all to 0.0, and then assign based on signals.
    weights_for_this_period = {sector: 0.0 for sector in sector_etfs}

    buy_sectors = [s for s in sector_etfs if current_signals[s] == 'Buy']
    sell_sectors = [s for s in sector_etfs if current_signals[s] == 'Sell']
    hold_sectors = [s for s in sector_etfs if current_signals[s] == 'Hold']

    # Determine the 'active' sectors for this period (those not getting a 'Sell' signal).
    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period[sector] = weight_per_active_sector
        # Sectors not in active_sectors_this_period (i.e., sell_sectors) implicitly retain 0.0 weight.
    else:
        # If all sectors are 'Sell', or no active signals, all weights remain 0.0.
        # Portfolio return for this week will be 0.0.
        pass # weights_for_this_period is already initialized to 0.0

    # Store the weights used for this period calculation in the historical_weights DataFrame.
    historical_weights.loc[current_date] = list(weights_for_this_period.values())

    # Calculate portfolio return for the current week using the determined weights.
    weekly_port_return = sum(weights_for_this_period[s] * current_week_returns[s] for s in sector_etfs)
    portfolio_returns.loc[current_date, 'Portfolio Return'] = weekly_port_return

    # Update `current_portfolio_weights_at_start_of_period` for the *next* iteration.
    # This reflects the portfolio composition at the end of the current period, which is the start of the next.
    current_portfolio_weights_at_start_of_period = weights_for_this_period

# 4. Calculate the cumulative returns of the portfolio_returns
# (1 + Return) cumprod() - 1 gives the cumulative return based on compounding.
portfolio_returns['Cumulative Return'] = (1 + portfolio_returns['Portfolio Return']).cumprod() - 1

print("Fixed-threshold backtesting simulation complete.")
print("First 5 rows of portfolio_returns:")
display(portfolio_returns.head())

In [ ]:
# 172.0

rolling_std_relative_performance = relative_performance.rolling(window=12).std()

multiplier = 0.5 # Adjust this multiplier as needed

dynamic_buy_thresholds = multiplier * rolling_std_relative_performance
dynamic_sell_thresholds = -multiplier * rolling_std_relative_performance

signals_dynamic = pd.DataFrame(index=relative_performance.index)

for sector in relative_performance.columns:
    signals_dynamic[sector] = 'Hold'  # Initialize all to 'Hold'
    # Apply 'Buy' signal where relative performance is above the dynamic buy threshold
    buy_condition = relative_performance[sector] > dynamic_buy_thresholds[sector]
    signals_dynamic.loc[buy_condition, sector] = 'Buy'

    # Apply 'Sell' signal where relative performance is below the dynamic sell threshold
    sell_condition = relative_performance[sector] < dynamic_sell_thresholds[sector]
    signals_dynamic.loc[sell_condition, sector] = 'Sell'

print("Dynamic signals DataFrame regenerated successfully!")
print("First 5 rows of signals_dynamic:")
display(signals_dynamic.head())

# Re-run the backtesting simulation for the dynamic-threshold strategy without costs
portfolio_returns_dynamic = pd.DataFrame(index=signals_dynamic.index, columns=['Portfolio Return'])

sector_etfs = [col for col in df_sector_pct_change.columns if col != "S&P 500"]
num_tradeable_sectors = len(sector_etfs)

returns_aligned_dynamic = df_sector_pct_change[sector_etfs].align(signals_dynamic, join='inner', axis=0)[0]
signals_aligned_dynamic = signals_dynamic.align(returns_aligned_dynamic, join='inner', axis=0)[0]

valid_indices = signals_aligned_dynamic.dropna(how='all').index
signals_aligned_dynamic = signals_aligned_dynamic.loc[valid_indices]
returns_aligned_dynamic = returns_aligned_dynamic.loc[valid_indices]
portfolio_returns_dynamic = portfolio_returns_dynamic.loc[valid_indices]

current_portfolio_weights_at_start_of_period_dynamic = {sector: 1.0 / num_tradeable_sectors for sector in sector_etfs}

historical_weights_dynamic = pd.DataFrame(columns=sector_etfs, index=signals_aligned_dynamic.index)

for i in range(len(signals_aligned_dynamic)):
    current_date = signals_aligned_dynamic.index[i]
    current_signals = signals_aligned_dynamic.loc[current_date]
    current_week_returns = returns_aligned_dynamic.loc[current_date]

    weights_for_this_period_dynamic = {sector: 0.0 for sector in sector_etfs}

    buy_sectors = [s for s in sector_etfs if current_signals[s] == 'Buy']
    sell_sectors = [s for s in sector_etfs if current_signals[s] == 'Sell']
    hold_sectors = [s for s in sector_etfs if current_signals[s] == 'Hold']

    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_dynamic[sector] = weight_per_active_sector

    historical_weights_dynamic.loc[current_date] = list(weights_for_this_period_dynamic.values())

    weekly_port_return = sum(weights_for_this_period_dynamic[s] * current_week_returns[s] for s in sector_etfs)
    portfolio_returns_dynamic.loc[current_date, 'Portfolio Return'] = weekly_port_return

    current_portfolio_weights_at_start_of_period_dynamic = weights_for_this_period_dynamic

portfolio_returns_dynamic['Cumulative Return'] = (1 + portfolio_returns_dynamic['Portfolio Return']).cumprod() - 1

print("Dynamic portfolio simulation (without costs) complete.")
print("First 5 rows of portfolio_returns_dynamic:")
display(portfolio_returns_dynamic.head())

# Re-run the backtesting simulation for the dynamic-threshold strategy with transaction costs
portfolio_returns_dynamic_with_costs = pd.DataFrame(index=signals_aligned_dynamic.index, columns=['Portfolio Return'])

transaction_cost_rate = 0.001  # 0.1% transaction cost

previous_period_weights = {sector: 1.0 / num_tradeable_sectors for sector in sector_etfs}

for i in range(len(signals_aligned_dynamic)):
    current_date = signals_aligned_dynamic.index[i]
    current_signals = signals_aligned_dynamic.loc[current_date] # Signals for the current period
    current_week_returns = returns_aligned_dynamic.loc[current_date] # Returns for the current period

    # Determine the weights_for_this_period_dynamic
    weights_for_this_period_dynamic = {sector: 0.0 for sector in sector_etfs}
    buy_sectors = [s for s in sector_etfs if current_signals[s] == 'Buy']
    hold_sectors = [s for s in sector_etfs if current_signals[s] == 'Hold']
    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_dynamic[sector] = weight_per_active_sector

    # Calculate transaction_costs for this period
    transaction_costs = 0.0
    for sector in sector_etfs:
        weight_change = abs(weights_for_this_period_dynamic[sector] - previous_period_weights[sector])
        transaction_costs += weight_change * transaction_cost_rate

    # Calculate weekly_port_return_before_costs
    weekly_port_return_before_costs = sum(weights_for_this_period_dynamic[s] * current_week_returns[s] for s in sector_etfs)

    # Deduct transaction_costs
    weekly_port_return_after_costs = weekly_port_return_before_costs - transaction_costs
    portfolio_returns_dynamic_with_costs.loc[current_date, 'Portfolio Return'] = weekly_port_return_after_costs

    # Update previous_period_weights for the next iteration
    previous_period_weights = weights_for_this_period_dynamic

portfolio_returns_dynamic_with_costs['Cumulative Return'] = (1 + portfolio_returns_dynamic_with_costs['Portfolio Return']).cumprod() - 1

print("Dynamic portfolio simulation with transaction costs complete.")
print("First 5 rows of portfolio_returns_dynamic_with_costs:")
display(portfolio_returns_dynamic_with_costs.head())

In [ ]:
# 173.0

import pandas as pd
import numpy as np

# 1. Resample df_sector to a monthly frequency, taking the last value of each month.
# Use 'ME' for month-end frequency to avoid future warnings.
df_sector_monthly = df_sector.resample('ME').last()

# 2. Calculate the monthly percentage changes of df_sector_monthly
df_sector_monthly_pct_change = df_sector_monthly.pct_change().dropna()

# 3. Compute the 3-month rolling mean of df_sector_monthly_pct_change
rolling_mean_monthly_pct_change = df_sector_monthly_pct_change.rolling(window=3).mean().dropna()

print("Monthly data re-preparation complete!")
print("First 5 rows of df_sector_monthly:")
display(df_sector_monthly.head())
print("\nFirst 5 rows of df_sector_monthly_pct_change:")
display(df_sector_monthly_pct_change.head())
print("\nFirst 5 rows of rolling_mean_monthly_pct_change:")
display(rolling_mean_monthly_pct_change.head())

# 4. Initialize an empty DataFrame called relative_performance_monthly
relative_performance_monthly = pd.DataFrame(index=rolling_mean_monthly_pct_change.index)

# 5. Iterate through each sector (excluding 'S&P 500') and calculate its relative performance
sector_etfs_monthly = [col for col in rolling_mean_monthly_pct_change.columns if col != "S&P 500"]

for sector in sector_etfs_monthly:
    relative_performance_monthly[sector] = rolling_mean_monthly_pct_change[sector] - rolling_mean_monthly_pct_change["S&P 500"]

print("\nMonthly relative performance DataFrame re-created successfully!")
print("First 5 rows of relative_performance_monthly:")
display(relative_performance_monthly.head())

# 6. Define buy_threshold_monthly and sell_threshold_monthly
buy_threshold_monthly = 0.001
sell_threshold_monthly = -0.001

# 7. Initialize an empty DataFrame called signals_monthly
signals_monthly = pd.DataFrame(index=relative_performance_monthly.index)

# 8. Generate 'Buy', 'Sell', or 'Hold' signals for each sector
for sector in relative_performance_monthly.columns:
    signals_monthly[sector] = 'Hold'  # Initialize all to 'Hold'
    signals_monthly.loc[relative_performance_monthly[sector] > buy_threshold_monthly, sector] = 'Buy'
    signals_monthly.loc[relative_performance_monthly[sector] < sell_threshold_monthly, sector] = 'Sell'

print("\nMonthly signals DataFrame re-created successfully!")
print("First 5 rows of signals_monthly:")
display(signals_monthly.head())

# 9. Backtest the monthly trading strategy:
# Initialize an empty DataFrame called portfolio_returns_monthly
portfolio_returns_monthly = pd.DataFrame(index=signals_monthly.index, columns=['Portfolio Return'])

num_tradeable_sectors_monthly = len(sector_etfs_monthly)

# Align monthly returns and signals to ensure they cover the same time period
returns_aligned_monthly = df_sector_monthly_pct_change[sector_etfs_monthly].align(signals_monthly, join='inner', axis=0)[0]
signals_aligned_monthly = signals_monthly.align(returns_aligned_monthly, join='inner', axis=0)[0]

# Initialize current_portfolio_weights_at_start_of_period_monthly
current_portfolio_weights_at_start_of_period_monthly = {sector: 1.0 / num_tradeable_sectors_monthly for sector in sector_etfs_monthly}

# Create a DataFrame to store historical weights for analysis
historical_weights_monthly = pd.DataFrame(columns=sector_etfs_monthly, index=signals_aligned_monthly.index)

# Iterate through each time step (row) in the aligned signals_aligned_monthly DataFrame
for i in range(len(signals_aligned_monthly)):
    current_date = signals_aligned_monthly.index[i]
    current_signals = signals_aligned_monthly.loc[current_date] # Signals for the current period
    current_month_returns = returns_aligned_monthly.loc[current_date] # Returns for the current period

    # `weights_for_this_period_monthly` will hold the actual weights used for the current period's return calculation.
    # Initialize all to 0.0, and then assign based on signals.
    weights_for_this_period_monthly = {sector: 0.0 for sector in sector_etfs_monthly}

    buy_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Buy']
    sell_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Sell']
    hold_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Hold']

    # Determine the 'active' sectors for this period (those not getting a 'Sell' signal).
    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_monthly[sector] = weight_per_active_sector
    else:
        # If all sectors are 'Sell', or no active signals, all weights remain 0.0.
        pass # weights_for_this_period_monthly is already initialized to 0.0

    # Store the weights used for this period calculation in the historical_weights_monthly DataFrame.
    historical_weights_monthly.loc[current_date] = list(weights_for_this_period_monthly.values())

    # Calculate portfolio return for the current month using the determined weights.
    monthly_port_return = sum(weights_for_this_period_monthly[s] * current_month_returns[s] for s in sector_etfs_monthly)
    portfolio_returns_monthly.loc[current_date, 'Portfolio Return'] = monthly_port_return

    # Update `current_portfolio_weights_at_start_of_period_monthly` for the *next* iteration.
    current_portfolio_weights_at_start_of_period_monthly = weights_for_this_period_monthly

# 10. Calculate the Cumulative Return of the portfolio_returns_monthly
portfolio_returns_monthly['Cumulative Return'] = (1 + portfolio_returns_monthly['Portfolio Return']).cumprod() - 1

print("\nMonthly portfolio simulation re-complete.")
# 11. Display the first 5 and last 5 rows of portfolio_returns_monthly
print("First 5 rows of portfolio_returns_monthly:")
display(portfolio_returns_monthly.head())
print("\nLast 5 rows of portfolio_returns_monthly:")
display(portfolio_returns_monthly.tail())

In [ ]:
# 174.0

rf = 0.01 # Re-defining risk-free rate, ensuring it's available in this scope

# --- Recalculate metrics for Monthly Strategy ---
# Ensure df_sector_monthly and df_sector_monthly_pct_change are current
# Resample df_sector to a monthly frequency and calculate monthly percent changes
df_sector_monthly = df_sector.resample('ME').last()
df_sector_monthly_pct_change = df_sector_monthly.pct_change().dropna()
# Compute the 3-month rolling mean of these monthly percent changes
rolling_mean_monthly_pct_change = df_sector_monthly_pct_change.rolling(window=3).mean().dropna()
# Calculate monthly relative performance
relative_performance_monthly = pd.DataFrame(index=rolling_mean_monthly_pct_change.index)
sector_etfs_monthly = [col for col in rolling_mean_monthly_pct_change.columns if col != "S&P 500"]
for sector in sector_etfs_monthly:
    relative_performance_monthly[sector] = rolling_mean_monthly_pct_change[sector] - rolling_mean_monthly_pct_change["S&P 500"]
# Define monthly 'buy', 'sell', or 'hold' signals
buy_threshold_monthly = 0.001
sell_threshold_monthly = -0.001
signals_monthly = pd.DataFrame(index=relative_performance_monthly.index)
for sector in relative_performance_monthly.columns:
    signals_monthly[sector] = 'Hold'  # Initialize all to 'Hold'
    signals_monthly.loc[relative_performance_monthly[sector] > buy_threshold_monthly, sector] = 'Buy'
    signals_monthly.loc[relative_performance_monthly[sector] < sell_threshold_monthly, sector] = 'Sell'
# Backtest this monthly trading strategy
portfolio_returns_monthly = pd.DataFrame(index=signals_monthly.index, columns=['Portfolio Return'])
num_tradeable_sectors_monthly = len(sector_etfs_monthly)
returns_aligned_monthly = df_sector_monthly_pct_change[sector_etfs_monthly].align(signals_monthly, join='inner', axis=0)[0]
signals_aligned_monthly = signals_monthly.align(returns_aligned_monthly, join='inner', axis=0)[0]
current_portfolio_weights_at_start_of_period_monthly = {sector: 1.0 / num_tradeable_sectors_monthly for sector in sector_etfs_monthly}
historical_weights_monthly = pd.DataFrame(columns=sector_etfs_monthly, index=signals_aligned_monthly.index)
for i in range(len(signals_aligned_monthly)):
    current_date = signals_aligned_monthly.index[i]
    current_signals = signals_aligned_monthly.loc[current_date]
    current_month_returns = returns_aligned_monthly.loc[current_date]
    weights_for_this_period_monthly = {sector: 0.0 for sector in sector_etfs_monthly}
    buy_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Buy']
    sell_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Sell']
    hold_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Hold']
    active_sectors_this_period = buy_sectors + hold_sectors
    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_monthly[sector] = weight_per_active_sector
    historical_weights_monthly.loc[current_date] = list(weights_for_this_period_monthly.values())
    monthly_port_return = sum(weights_for_this_period_monthly[s] * current_month_returns[s] for s in sector_etfs_monthly)
    portfolio_returns_monthly.loc[current_date, 'Portfolio Return'] = monthly_port_return
    current_portfolio_weights_at_start_of_period_monthly = weights_for_this_period_monthly
portfolio_returns_monthly['Cumulative Return'] = (1 + portfolio_returns_monthly['Portfolio Return']).cumprod() - 1

num_months_monthly = len(portfolio_returns_monthly)
num_years_monthly = num_months_monthly / 12.0

portfolio_total_cumulative_return_monthly = portfolio_returns_monthly['Cumulative Return'].iloc[-1]
portfolio_annualized_return_monthly = (1 + portfolio_total_cumulative_return_monthly)**(1/num_years_monthly) - 1
portfolio_annualized_volatility_monthly = portfolio_returns_monthly['Portfolio Return'].std() * np.sqrt(12)
portfolio_sharpe_ratio_monthly = (portfolio_annualized_return_monthly - rf) / portfolio_annualized_volatility_monthly

# --- Recalculate metrics for Fixed Strategy --- (using already calculated portfolio_returns)
num_weeks = len(portfolio_returns)
num_years = num_weeks / 52.0
portfolio_total_cumulative_return = portfolio_returns['Cumulative Return'].iloc[-1]
portfolio_annualized_return = (1 + portfolio_total_cumulative_return)**(1/num_years) - 1
portfolio_annualized_volatility = portfolio_returns['Portfolio Return'].std() * np.sqrt(52)
portfolio_sharpe_ratio = (portfolio_annualized_return - rf) / portfolio_annualized_volatility

# Recalculate S&P 500 metrics for consistency with the fixed strategy's time frame
s_p_500_returns_aligned = df_sector_pct_change["S&P 500"].align(portfolio_returns, join='inner', axis=0)[0]
s_p_500_cumulative_return = (1 + s_p_500_returns_aligned).cumprod() - 1
s_p_500_total_cumulative_return = s_p_500_cumulative_return.iloc[-1]
s_p_500_annualized_return = (1 + s_p_500_total_cumulative_return)**(1/num_years) - 1
s_p_500_annualized_volatility = s_p_500_returns_aligned.std() * np.sqrt(52)
s_p_500_sharpe_ratio = (s_p_500_annualized_return - rf) / s_p_500_annualized_volatility

# --- Recalculate metrics for Dynamic Strategy --- (using already calculated portfolio_returns_dynamic)
num_weeks_dynamic = len(portfolio_returns_dynamic)
num_years_dynamic = num_weeks_dynamic / 52.0
portfolio_total_cumulative_return_dynamic = portfolio_returns_dynamic['Cumulative Return'].iloc[-1]
portfolio_annualized_return_dynamic = (1 + portfolio_total_cumulative_return_dynamic)**(1/num_years_dynamic) - 1
portfolio_annualized_volatility_dynamic = portfolio_returns_dynamic['Portfolio Return'].std() * np.sqrt(52)
portfolio_sharpe_ratio_dynamic = (portfolio_annualized_return_dynamic - rf) / portfolio_annualized_volatility_dynamic

# --- Recalculate metrics for Dynamic Strategy with Costs --- (using already calculated portfolio_returns_dynamic_with_costs)
num_weeks_dynamic_costs = len(portfolio_returns_dynamic_with_costs)
num_years_dynamic_costs = num_weeks_dynamic_costs / 52.0
portfolio_total_cumulative_return_dynamic_costs = portfolio_returns_dynamic_with_costs['Cumulative Return'].iloc[-1]
portfolio_annualized_return_dynamic_costs = (1 + portfolio_total_cumulative_return_dynamic_costs)**(1/num_years_dynamic_costs) - 1
portfolio_annualized_volatility_dynamic_costs = portfolio_returns_dynamic_with_costs['Portfolio Return'].std() * np.sqrt(52)
portfolio_sharpe_ratio_dynamic_costs = (portfolio_annualized_return_dynamic_costs - rf) / portfolio_annualized_volatility_dynamic_costs

# Print Summary Comparison
print("\n--- Strategy Performance Comparison (Monthly vs. Dynamic Costs vs. Dynamic vs. Fixed vs. S&P 500) ---")
print(f"Metric                        | Monthly         | Dynamic Costs   | Dynamic         | Fixed Threshold | S&P 500       ")
print(f"------------------------------------------------------------------------------------------------------------------")
print(f"Total Cumulative Return       | {portfolio_total_cumulative_return_monthly:.4f}        | {portfolio_total_cumulative_return_dynamic_costs:.4f}        | {portfolio_total_cumulative_return_dynamic:.4f}         | {portfolio_total_cumulative_return:.4f}        | {s_p_500_total_cumulative_return:.4f}       ")
print(f"Annualized Return             | {portfolio_annualized_return_monthly:.4f}        | {portfolio_annualized_return_dynamic_costs:.4f}        | {portfolio_annualized_return_dynamic:.4f}         | {portfolio_annualized_return:.4f}        | {s_p_500_annualized_return:.4f}       ")
print(f"Annualized Volatility         | {portfolio_annualized_volatility_monthly:.4f}        | {portfolio_annualized_volatility_dynamic_costs:.4f}        | {portfolio_annualized_volatility_dynamic:.4f}         | {portfolio_annualized_volatility:.4f}        | {s_p_500_annualized_volatility:.4f}       ")
print(f"Sharpe Ratio                  | {portfolio_sharpe_ratio_monthly:.4f}        | {portfolio_sharpe_ratio_dynamic_costs:.4f}        | {portfolio_sharpe_ratio_dynamic:.4f}         | {portfolio_sharpe_ratio:.4f}        | {s_p_500_sharpe_ratio:.4f}       ")
print("------------------------------------------------------------------------------------------------------------------")

In [ ]:
# 175.0

import matplotlib.pyplot as plt

plt.figure(figsize=(14, 7))
plt.plot(portfolio_returns_monthly.index, portfolio_returns_monthly['Cumulative Return'], label='Monthly Strategy Cumulative Return', color='orange')
plt.plot(portfolio_returns.index, portfolio_returns['Cumulative Return'], label='Fixed Strategy Cumulative Return', color='blue')
plt.plot(portfolio_returns_dynamic.index, portfolio_returns_dynamic['Cumulative Return'], label='Dynamic Strategy Cumulative Return (no costs)', color='green')
plt.plot(portfolio_returns_dynamic_with_costs.index, portfolio_returns_dynamic_with_costs['Cumulative Return'], label='Dynamic Strategy Cumulative Return (with costs)', color='purple')
plt.plot(s_p_500_cumulative_return.index, s_p_500_cumulative_return, label='S&P 500 Cumulative Return', color='red')
plt.title('Cumulative Returns: All Strategies vs. S&P 500 (Including XLRE)')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend()
plt.show()

In [ ]:
# 176.0

rolling_std_relative_performance = relative_performance.rolling(window=12).std()

multiplier = 0.5 # Adjust this multiplier as needed

dynamic_buy_thresholds = multiplier * rolling_std_relative_performance
dynamic_sell_thresholds = -multiplier * rolling_std_relative_performance

signals_dynamic = pd.DataFrame(index=relative_performance.index)

for sector in relative_performance.columns:
    signals_dynamic[sector] = 'Hold'  # Initialize all to 'Hold'
    # Apply 'Buy' signal where relative performance is above the dynamic buy threshold
    buy_condition = relative_performance[sector] > dynamic_buy_thresholds[sector]
    signals_dynamic.loc[buy_condition, sector] = 'Buy'

    # Apply 'Sell' signal where relative performance is below the dynamic sell threshold
    sell_condition = relative_performance[sector] < dynamic_sell_thresholds[sector]
    signals_dynamic.loc[sell_condition, sector] = 'Sell'

print("Dynamic signals DataFrame regenerated successfully!")
print("First 5 rows of signals_dynamic:")
display(signals_dynamic.head())

# Re-run the backtesting simulation for the dynamic-threshold strategy without costs
portfolio_returns_dynamic = pd.DataFrame(index=signals_dynamic.index, columns=['Portfolio Return'])

sector_etfs = [col for col in df_sector_pct_change.columns if col != "S&P 500"]
num_tradeable_sectors = len(sector_etfs)

returns_aligned_dynamic = df_sector_pct_change[sector_etfs].align(signals_dynamic, join='inner', axis=0)[0]
signals_aligned_dynamic = signals_dynamic.align(returns_aligned_dynamic, join='inner', axis=0)[0]

valid_indices = signals_aligned_dynamic.dropna(how='all').index
signals_aligned_dynamic = signals_aligned_dynamic.loc[valid_indices]
returns_aligned_dynamic = returns_aligned_dynamic.loc[valid_indices]
portfolio_returns_dynamic = portfolio_returns_dynamic.loc[valid_indices]

current_portfolio_weights_at_start_of_period_dynamic = {sector: 1.0 / num_tradeable_sectors for sector in sector_etfs}

historical_weights_dynamic = pd.DataFrame(columns=sector_etfs, index=signals_aligned_dynamic.index)

for i in range(len(signals_aligned_dynamic)):
    current_date = signals_aligned_dynamic.index[i]
    current_signals = signals_aligned_dynamic.loc[current_date] # Signals for the current period
    current_week_returns = returns_aligned_dynamic.loc[current_date] # Returns for the current period

    weights_for_this_period_dynamic = {sector: 0.0 for sector in sector_etfs}

    buy_sectors = [s for s in sector_etfs if current_signals[s] == 'Buy']
    sell_sectors = [s for s in sector_etfs if current_signals[s] == 'Sell']
    hold_sectors = [s for s in sector_etfs if current_signals[s] == 'Hold']

    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_dynamic[sector] = weight_per_active_sector
    else:
        # If all sectors are 'Sell', or no active signals, all weights remain 0.0.
        # Portfolio return for this week will be 0.0.
        pass # weights_for_this_period_dynamic is already initialized to 0.0

    historical_weights_dynamic.loc[current_date] = list(weights_for_this_period_dynamic.values())

    weekly_port_return = sum(weights_for_this_period_dynamic[s] * current_week_returns[s] for s in sector_etfs)
    portfolio_returns_dynamic.loc[current_date, 'Portfolio Return'] = weekly_port_return

    current_portfolio_weights_at_start_of_period_dynamic = weights_for_this_period_dynamic

portfolio_returns_dynamic['Cumulative Return'] = (1 + portfolio_returns_dynamic['Portfolio Return']).cumprod() - 1

print("Dynamic portfolio simulation (without costs) complete.")
print("First 5 rows of portfolio_returns_dynamic:")
display(portfolio_returns_dynamic.head())

# Re-run the backtesting simulation for the dynamic-threshold strategy with transaction costs
portfolio_returns_dynamic_with_costs = pd.DataFrame(index=signals_aligned_dynamic.index, columns=['Portfolio Return'])

transaction_cost_rate = 0.001  # 0.1% transaction cost

previous_period_weights = {sector: 1.0 / num_tradeable_sectors for sector in sector_etfs}

for i in range(len(signals_aligned_dynamic)):
    current_date = signals_aligned_dynamic.index[i]
    current_signals = signals_aligned_dynamic.loc[current_date] # Signals for the current period
    current_week_returns = returns_aligned_dynamic.loc[current_date] # Returns for the current period

    # Determine the weights_for_this_period_dynamic
    weights_for_this_period_dynamic = {sector: 0.0 for sector in sector_etfs}
    buy_sectors = [s for s in sector_etfs if current_signals[s] == 'Buy']
    hold_sectors = [s for s in sector_etfs if current_signals[s] == 'Hold']
    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_dynamic[sector] = weight_per_active_sector

    # Calculate transaction_costs for this period
    transaction_costs = 0.0
    for sector in sector_etfs:
        weight_change = abs(weights_for_this_period_dynamic[sector] - previous_period_weights[sector])
        transaction_costs += weight_change * transaction_cost_rate

    # Calculate weekly_port_return_before_costs
    weekly_port_return_before_costs = sum(weights_for_this_period_dynamic[s] * current_week_returns[s] for s in sector_etfs)

    # Deduct transaction_costs
    weekly_port_return_after_costs = weekly_port_return_before_costs - transaction_costs
    portfolio_returns_dynamic_with_costs.loc[current_date, 'Portfolio Return'] = weekly_port_return_after_costs

    # Update previous_period_weights for the next iteration
    previous_period_weights = weights_for_this_period_dynamic

portfolio_returns_dynamic_with_costs['Cumulative Return'] = (1 + portfolio_returns_dynamic_with_costs['Portfolio Return']).cumprod() - 1

print("Dynamic portfolio simulation with transaction costs complete.")
print("First 5 rows of portfolio_returns_dynamic_with_costs:")
display(portfolio_returns_dynamic_with_costs.head())

In [ ]:
# 177.0

import pandas as pd
import numpy as np

# 1. Resample df_sector to a monthly frequency, taking the last value of each month.
# Use 'ME' for month-end frequency to avoid future warnings.
df_sector_monthly = df_sector.resample('ME').last()

# 2. Calculate the monthly percentage changes of df_sector_monthly
df_sector_monthly_pct_change = df_sector_monthly.pct_change().dropna()

# 3. Compute the 3-month rolling mean of df_sector_monthly_pct_change
rolling_mean_monthly_pct_change = df_sector_monthly_pct_change.rolling(window=3).mean().dropna()

print("Monthly data re-preparation complete!")
print("First 5 rows of df_sector_monthly:")
display(df_sector_monthly.head())
print("\nFirst 5 rows of df_sector_monthly_pct_change:")
display(df_sector_monthly_pct_change.head())
print("\nFirst 5 rows of rolling_mean_monthly_pct_change:")
display(rolling_mean_monthly_pct_change.head())

# 4. Initialize an empty DataFrame called relative_performance_monthly
relative_performance_monthly = pd.DataFrame(index=rolling_mean_monthly_pct_change.index)

# 5. Iterate through each sector (excluding 'S&P 500') and calculate its relative performance
sector_etfs_monthly = [col for col in rolling_mean_monthly_pct_change.columns if col != "S&P 500"]

for sector in sector_etfs_monthly:
    relative_performance_monthly[sector] = rolling_mean_monthly_pct_change[sector] - rolling_mean_monthly_pct_change["S&P 500"]

print("\nMonthly relative performance DataFrame re-created successfully!")
print("First 5 rows of relative_performance_monthly:")
display(relative_performance_monthly.head())

# 6. Define buy_threshold_monthly and sell_threshold_monthly
buy_threshold_monthly = 0.001
sell_threshold_monthly = -0.001

# 7. Initialize an empty DataFrame called signals_monthly
signals_monthly = pd.DataFrame(index=relative_performance_monthly.index)

# 8. Generate 'Buy', 'Sell', or 'Hold' signals for each sector
for sector in relative_performance_monthly.columns:
    signals_monthly[sector] = 'Hold'  # Initialize all to 'Hold'
    signals_monthly.loc[relative_performance_monthly[sector] > buy_threshold_monthly, sector] = 'Buy'
    signals_monthly.loc[relative_performance_monthly[sector] < sell_threshold_monthly, sector] = 'Sell'

print("\nMonthly signals DataFrame re-created successfully!")
print("First 5 rows of signals_monthly:")
display(signals_monthly.head())

# 9. Backtest the monthly trading strategy:
# Initialize an empty DataFrame called portfolio_returns_monthly
portfolio_returns_monthly = pd.DataFrame(index=signals_monthly.index, columns=['Portfolio Return'])

num_tradeable_sectors_monthly = len(sector_etfs_monthly)

# Align monthly returns and signals to ensure they cover the same time period
returns_aligned_monthly = df_sector_monthly_pct_change[sector_etfs_monthly].align(signals_monthly, join='inner', axis=0)[0]
signals_aligned_monthly = signals_monthly.align(returns_aligned_monthly, join='inner', axis=0)[0]

# Initialize current_portfolio_weights_at_start_of_period_monthly
current_portfolio_weights_at_start_of_period_monthly = {sector: 1.0 / num_tradeable_sectors_monthly for sector in sector_etfs_monthly}

# Create a DataFrame to store historical weights for analysis
historical_weights_monthly = pd.DataFrame(columns=sector_etfs_monthly, index=signals_aligned_monthly.index)

# Iterate through each time step (row) in the aligned signals_aligned_monthly DataFrame
for i in range(len(signals_aligned_monthly)):
    current_date = signals_aligned_monthly.index[i]
    current_signals = signals_aligned_monthly.loc[current_date] # Signals for the current period
    current_month_returns = returns_aligned_monthly.loc[current_date] # Returns for the current period

    # `weights_for_this_period_monthly` will hold the actual weights used for the current period's return calculation.
    # Initialize all to 0.0, and then assign based on signals.
    weights_for_this_period_monthly = {sector: 0.0 for sector in sector_etfs_monthly}

    buy_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Buy']
    sell_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Sell']
    hold_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Hold']

    # Determine the 'active' sectors for this period (those not getting a 'Sell' signal).
    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_monthly[sector] = weight_per_active_sector
    else:
        # If all sectors are 'Sell', or no active signals, all weights remain 0.0.
        pass # weights_for_this_period_monthly is already initialized to 0.0

    # Store the weights used for this period calculation in the historical_weights_monthly DataFrame.
    historical_weights_monthly.loc[current_date] = list(weights_for_this_period_monthly.values())

    # Calculate portfolio return for the current month using the determined weights.
    monthly_port_return = sum(weights_for_this_period_monthly[s] * current_month_returns[s] for s in sector_etfs_monthly)
    portfolio_returns_monthly.loc[current_date, 'Portfolio Return'] = monthly_port_return

    # Update `current_portfolio_weights_at_start_of_period_monthly` for the *next* iteration.
    current_portfolio_weights_at_start_of_period_monthly = weights_for_this_period_monthly

# 10. Calculate the Cumulative Return of the portfolio_returns_monthly
portfolio_returns_monthly['Cumulative Return'] = (1 + portfolio_returns_monthly['Portfolio Return']).cumprod() - 1

print("\nMonthly portfolio simulation re-complete.")
# 11. Display the first 5 and last 5 rows of portfolio_returns_monthly
print("First 5 rows of portfolio_returns_monthly:")
display(portfolio_returns_monthly.head())
print("\nLast 5 rows of portfolio_returns_monthly:")
display(portfolio_returns_monthly.tail())

In [ ]:
average_fixed_weekly_trades = fixed_strategy_weekly_trades.mean()
average_dynamic_weekly_trades = dynamic_strategy_weekly_trades.mean()

print(f"\nAverage Weekly Trades (Fixed-Threshold Strategy): {average_fixed_weekly_trades:.2f}")
print(f"Average Weekly Trades (Dynamic-Threshold Strategy): {average_dynamic_weekly_trades:.2f}")

In [ ]:
# 178.0

rf = 0.01 # Re-defining risk-free rate, ensuring it's available in this scope

# --- Recalculate metrics for Monthly Strategy ---
# Ensure df_sector_monthly and df_sector_monthly_pct_change are current
# Resample df_sector to a monthly frequency and calculate monthly percent changes
df_sector_monthly = df_sector.resample('ME').last()
df_sector_monthly_pct_change = df_sector_monthly.pct_change().dropna()
# Compute the 3-month rolling mean of these monthly percent changes
rolling_mean_monthly_pct_change = df_sector_monthly_pct_change.rolling(window=3).mean().dropna()
# Calculate monthly relative performance
relative_performance_monthly = pd.DataFrame(index=rolling_mean_monthly_pct_change.index)
sector_etfs_monthly = [col for col in rolling_mean_monthly_pct_change.columns if col != "S&P 500"]
for sector in sector_etfs_monthly:
    relative_performance_monthly[sector] = rolling_mean_monthly_pct_change[sector] - rolling_mean_monthly_pct_change["S&P 500"]
# Define monthly 'buy', 'sell', or 'hold' signals
buy_threshold_monthly = 0.001
sell_threshold_monthly = -0.001
signals_monthly = pd.DataFrame(index=relative_performance_monthly.index)
for sector in relative_performance_monthly.columns:
    signals_monthly[sector] = 'Hold'  # Initialize all to 'Hold'
    signals_monthly.loc[relative_performance_monthly[sector] > buy_threshold_monthly, sector] = 'Buy'
    signals_monthly.loc[relative_performance_monthly[sector] < sell_threshold_monthly, sector] = 'Sell'
# Backtest this monthly trading strategy
portfolio_returns_monthly = pd.DataFrame(index=signals_monthly.index, columns=['Portfolio Return'])
num_tradeable_sectors_monthly = len(sector_etfs_monthly)
returns_aligned_monthly = df_sector_monthly_pct_change[sector_etfs_monthly].align(signals_monthly, join='inner', axis=0)[0]
signals_aligned_monthly = signals_monthly.align(returns_aligned_monthly, join='inner', axis=0)[0]
current_portfolio_weights_at_start_of_period_monthly = {sector: 1.0 / num_tradeable_sectors_monthly for sector in sector_etfs_monthly}
historical_weights_monthly = pd.DataFrame(columns=sector_etfs_monthly, index=signals_aligned_monthly.index)
for i in range(len(signals_aligned_monthly)):
    current_date = signals_aligned_monthly.index[i]
    current_signals = signals_aligned_monthly.loc[current_date]
    current_month_returns = returns_aligned_monthly.loc[current_date]
    weights_for_this_period_monthly = {sector: 0.0 for sector in sector_etfs_monthly}
    buy_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Buy']
    sell_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Sell']
    hold_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Hold']
    active_sectors_this_period = buy_sectors + hold_sectors
    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_monthly[sector] = weight_per_active_sector
    historical_weights_monthly.loc[current_date] = list(weights_for_this_period_monthly.values())
    monthly_port_return = sum(weights_for_this_period_monthly[s] * current_month_returns[s] for s in sector_etfs_monthly)
    portfolio_returns_monthly.loc[current_date, 'Portfolio Return'] = monthly_port_return
    current_portfolio_weights_at_start_of_period_monthly = weights_for_this_period_monthly
portfolio_returns_monthly['Cumulative Return'] = (1 + portfolio_returns_monthly['Portfolio Return']).cumprod() - 1

num_months_monthly = len(portfolio_returns_monthly)
num_years_monthly = num_months_monthly / 12.0

portfolio_total_cumulative_return_monthly = portfolio_returns_monthly['Cumulative Return'].iloc[-1]
portfolio_annualized_return_monthly = (1 + portfolio_total_cumulative_return_monthly)**(1/num_years_monthly) - 1
portfolio_annualized_volatility_monthly = portfolio_returns_monthly['Portfolio Return'].std() * np.sqrt(12)
portfolio_sharpe_ratio_monthly = (portfolio_annualized_return_monthly - rf) / portfolio_annualized_volatility_monthly

# --- Recalculate metrics for Fixed Strategy --- (using already calculated portfolio_returns)
num_weeks = len(portfolio_returns)
num_years = num_weeks / 52.0
portfolio_total_cumulative_return = portfolio_returns['Cumulative Return'].iloc[-1]
portfolio_annualized_return = (1 + portfolio_total_cumulative_return)**(1/num_years) - 1
portfolio_annualized_volatility = portfolio_returns['Portfolio Return'].std() * np.sqrt(52)
portfolio_sharpe_ratio = (portfolio_annualized_return - rf) / portfolio_annualized_volatility

# Recalculate S&P 500 metrics for consistency with the fixed strategy's time frame
s_p_500_returns_aligned = df_sector_pct_change["S&P 500"].align(portfolio_returns, join='inner', axis=0)[0]
s_p_500_cumulative_return = (1 + s_p_500_returns_aligned).cumprod() - 1
s_p_500_total_cumulative_return = s_p_500_cumulative_return.iloc[-1]
s_p_500_annualized_return = (1 + s_p_500_total_cumulative_return)**(1/num_years) - 1
s_p_500_annualized_volatility = s_p_500_returns_aligned.std() * np.sqrt(52)
s_p_500_sharpe_ratio = (s_p_500_annualized_return - rf) / s_p_500_annualized_volatility

# --- Recalculate metrics for Dynamic Strategy --- (using already calculated portfolio_returns_dynamic)
num_weeks_dynamic = len(portfolio_returns_dynamic)
num_years_dynamic = num_weeks_dynamic / 52.0
portfolio_total_cumulative_return_dynamic = portfolio_returns_dynamic['Cumulative Return'].iloc[-1]
portfolio_annualized_return_dynamic = (1 + portfolio_total_cumulative_return_dynamic)**(1/num_years_dynamic) - 1
portfolio_annualized_volatility_dynamic = portfolio_returns_dynamic['Portfolio Return'].std() * np.sqrt(52)
portfolio_sharpe_ratio_dynamic = (portfolio_annualized_return_dynamic - rf) / portfolio_annualized_volatility_dynamic

# --- Recalculate metrics for Dynamic Strategy with Costs --- (using already calculated portfolio_returns_dynamic_with_costs)
num_weeks_dynamic_costs = len(portfolio_returns_dynamic_with_costs)
num_years_dynamic_costs = num_weeks_dynamic_costs / 52.0
portfolio_total_cumulative_return_dynamic_costs = portfolio_returns_dynamic_with_costs['Cumulative Return'].iloc[-1]
portfolio_annualized_return_dynamic_costs = (1 + portfolio_total_cumulative_return_dynamic_costs)**(1/num_years_dynamic_costs) - 1
portfolio_annualized_volatility_dynamic_costs = portfolio_returns_dynamic_with_costs['Portfolio Return'].std() * np.sqrt(52)
portfolio_sharpe_ratio_dynamic_costs = (portfolio_annualized_return_dynamic_costs - rf) / portfolio_annualized_volatility_dynamic_costs

# Print Summary Comparison
print("\n--- Strategy Performance Comparison (Monthly vs. Dynamic Costs vs. Dynamic vs. Fixed vs. S&P 500) ---")
print(f"Metric                        | Monthly         | Dynamic Costs   | Dynamic         | Fixed Threshold | S&P 500       ")
print(f"------------------------------------------------------------------------------------------------------------------")
print(f"Total Cumulative Return       | {portfolio_total_cumulative_return_monthly:.4f}        | {portfolio_total_cumulative_return_dynamic_costs:.4f}        | {portfolio_total_cumulative_return_dynamic:.4f}         | {portfolio_total_cumulative_return:.4f}        | {s_p_500_total_cumulative_return:.4f}       ")
print(f"Annualized Return             | {portfolio_annualized_return_monthly:.4f}        | {portfolio_annualized_return_dynamic_costs:.4f}        | {portfolio_annualized_return_dynamic:.4f}         | {portfolio_annualized_return:.4f}        | {s_p_500_annualized_return:.4f}       ")
print(f"Annualized Volatility         | {portfolio_annualized_volatility_monthly:.4f}        | {portfolio_annualized_volatility_dynamic_costs:.4f}        | {portfolio_annualized_volatility_dynamic:.4f}         | {portfolio_annualized_volatility:.4f}        | {s_p_500_annualized_volatility:.4f}       ")
print(f"Sharpe Ratio                  | {portfolio_sharpe_ratio_monthly:.4f}        | {portfolio_sharpe_ratio_dynamic_costs:.4f}        | {portfolio_sharpe_ratio_dynamic:.4f}         | {portfolio_sharpe_ratio:.4f}        | {s_p_500_sharpe_ratio:.4f}       ")
print("------------------------------------------------------------------------------------------------------------------")

In [ ]:
# 179.0

import matplotlib.pyplot as plt

plt.figure(figsize=(14, 7))
plt.plot(portfolio_returns_monthly.index, portfolio_returns_monthly['Cumulative Return'], label='Monthly Strategy Cumulative Return', color='orange')
plt.plot(portfolio_returns.index, portfolio_returns['Cumulative Return'], label='Fixed Strategy Cumulative Return', color='blue')
plt.plot(portfolio_returns_dynamic.index, portfolio_returns_dynamic['Cumulative Return'], label='Dynamic Strategy Cumulative Return (no costs)', color='green')
plt.plot(portfolio_returns_dynamic_with_costs.index, portfolio_returns_dynamic_with_costs['Cumulative Return'], label='Dynamic Strategy Cumulative Return (with costs)', color='purple')
plt.plot(s_p_500_cumulative_return.index, s_p_500_cumulative_return, label='S&P 500 Cumulative Return', color='red')
plt.title('Cumulative Returns: All Strategies vs. S&P 500 (Including XLRE)')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend()
plt.show()

In [ ]:
# 180.0

import pandas as pd

df_sector = pd.read_csv("Sector_ETF_Data.csv", index_col="Date", parse_dates=True)

# Correct for missing values using ffill() and then bfill()
df_sector = df_sector.ffill()
df_sector = df_sector.bfill()

print("Missing values after correction:")
display(df_sector.isnull().sum())

print("DataFrame head after missing value correction:")
display(df_sector.head())

In [ ]:
# 181.0

import pandas as pd
import yfinance as yf
import fredapi # Import fredapi, though not directly used in this specific cell, it's a necessary library from context.

tickers = ["XLK", "XLC", "XLV", "XLF", "XLI", "XLB", "XLU", "XLE", "XLY", "XLP", "SPY", "XLRE"]

# Mapping from tickers to desired sector names
ticker_to_sector = {
    "XLK": "Technology",
    "XLC": "Communications",
    "XLV": "Healthcare",
    "XLF": "Financial",
    "XLI": "Industrial",
    "XLB": "Material",
    "XLU": "Utilities",
    "XLE": "Energy",
    "XLY": "CDiscretionary",
    "XLP": "CStaples",
    "SPY": "S&P 500",
    "XLRE": "Real Estate"
}

# Set the date range
start_date = "2010-01-01"
end_date = "2025-12-30"

# Initialize an empty dictionary to store ETF data
sector_data = {}

# Download historical data for each ETF, explicitly setting auto_adjust=True.
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date, interval="1wk", auto_adjust=True, progress=False)
    if not data.empty:
        sector_data[ticker] = data["Close"]

# If valid data exists, combine it into a DataFrame and flatten the headers.
if sector_data:
    # Concatenate the Series objects; using keys creates a MultiIndex header.
    df_sector = pd.concat(sector_data.values(), axis=1, keys=sector_data.keys())

    # Flatten the MultiIndex columns by extracting the first level (tickers) and mapping to sector names.
    df_sector.columns = [ticker_to_sector.get(ticker, ticker) for ticker in df_sector.columns.get_level_values(0)]

    # Save the DataFrame to CSV
    df_sector.to_csv("Sector_ETF_Data.csv")
    print("Sector ETF data saved successfully!")
else:
    print("No valid data retrieved for any tickers.")

In [ ]:
# 182.0

!pip install fredapi

In [ ]:
# 183.0

import pandas as pd
import yfinance as yf
import fredapi # Import fredapi, though not directly used in this specific cell, it's a necessary library from context.

tickers = ["XLK", "XLC", "XLV", "XLF", "XLI", "XLB", "XLU", "XLE", "XLY", "XLP", "SPY", "XLRE"]

# Mapping from tickers to desired sector names
ticker_to_sector = {
    "XLK": "Technology",
    "XLC": "Communications",
    "XLV": "Healthcare",
    "XLF": "Financial",
    "XLI": "Industrial",
    "XLB": "Material",
    "XLU": "Utilities",
    "XLE": "Energy",
    "XLY": "CDiscretionary",
    "XLP": "CStaples",
    "SPY": "S&P 500",
    "XLRE": "Real Estate"
}

# Set the date range
start_date = "2010-01-01"
end_date = "2025-12-30"

# Initialize an empty dictionary to store ETF data
sector_data = {}

# Download historical data for each ETF, explicitly setting auto_adjust=True.
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date, interval="1wk", auto_adjust=True, progress=False)
    if not data.empty:
        sector_data[ticker] = data["Close"]

# If valid data exists, combine it into a DataFrame and flatten the headers.
if sector_data:
    # Concatenate the Series objects; using keys creates a MultiIndex header.
    df_sector = pd.concat(sector_data.values(), axis=1, keys=sector_data.keys())

    # Flatten the MultiIndex columns by extracting the first level (tickers) and mapping to sector names.
    df_sector.columns = [ticker_to_sector.get(ticker, ticker) for ticker in df_sector.columns.get_level_values(0)]

    # Save the DataFrame to CSV
    df_sector.to_csv("Sector_ETF_Data.csv")
    print("Sector ETF data saved successfully!")
else:
    print("No valid data retrieved for any tickers.")

Sector ETF data saved successfully!


In [ ]:
# 184.0

import pandas as pd

df_sector = pd.read_csv("Sector_ETF_Data.csv", index_col="Date", parse_dates=True)

# Correct for missing values using ffill() and then bfill()
df_sector = df_sector.ffill()
df_sector = df_sector.bfill()

print("Missing values after correction:")
display(df_sector.isnull().sum())

print("DataFrame head after missing value correction:")
display(df_sector.head())

In [ ]:
# 185.0

import pandas as pd
from fredapi import Fred

# FRED API key - Ensure this is correctly set or retrieved from a secure source.
# For demonstration, using the API key from the provided notebook context.
fred = Fred(api_key='e8fb9c414f6ab8f1a06a5472466a31dc')

# Define date range for weekly index, consistent with previous data fetches
start_date = "2010-01-01"
end_date = "2025-12-30"
weekly_index = pd.date_range(start=start_date, end=end_date, freq='W-FRI').strftime('%Y-%m-%d')

# Define FRED series IDs and their corresponding labels
series_dict = {
    'GDP': 'GDP',
    'Inflation (CPI)': 'CPIAUCSL',
    'Unemployment Rate': 'UNRATE',
    'Interest Rate (Fed Funds)': 'FEDFUNDS'
}

# Create a DataFrame with weekly index to store macroeconomic data
macro_data = pd.DataFrame(index=pd.to_datetime(weekly_index))
macro_data.index.name = "Date"

# Fetch and interpolate data for each series
for label, series_id in series_dict.items():
    data = fred.get_series(series_id)
    if data is None: # Handle cases where FRED API might return None
        print(f"Warning: Could not fetch series {series_id} for {label}")
        continue

    data = data.to_frame(name=label)
    data.index = pd.to_datetime(data.index)

    # Resample to daily, interpolating or forward-filling as specified
    if label == 'Interest Rate (Fed Funds)':
        daily = data.resample('D').ffill()
    else:
        # For other series, interpolate missing values to daily frequency
        daily = data.resample('D').interpolate(method='linear')

    # Reindex the daily data to the predefined weekly index and interpolate remaining missing values
    # Use limit_direction='both' for interpolation to handle NaNs at ends of series
    weekly_processed = daily.reindex(macro_data.index).interpolate(method='linear', limit_direction='both')
    macro_data[label] = weekly_processed[label]

# Save the completed macroeconomic DataFrame to a CSV file
macro_data.to_csv("Macroeconomic_Data.csv", index=True)

print("Macroeconomic Data fetched, processed, and saved successfully to 'Macroeconomic_Data.csv'!")

Macroeconomic Data fetched, processed, and saved successfully to 'Macroeconomic_Data.csv'!


In [ ]:
# 186.0

df_sector_pct_change = df_sector.pct_change().dropna()

rolling_mean_pct_change = df_sector_pct_change.rolling(window=12).mean().dropna()

relative_performance = pd.DataFrame(index=rolling_mean_pct_change.index)

sector_etfs = [col for col in rolling_mean_pct_change.columns if col != "S&P 500"]

for sector in sector_etfs:
    relative_performance[sector] = rolling_mean_pct_change[sector] - rolling_mean_pct_change["S&P 500"]

print("df_sector_pct_change, rolling_mean_pct_change, and relative_performance recalculated successfully!")
print("First 5 rows of relative_performance:")
display(relative_performance.head())

In [ ]:
# 187.0

buy_threshold = 0.001
sell_threshold = -0.001

signals = pd.DataFrame(index=relative_performance.index)

for sector in relative_performance.columns:
    signals[sector] = 'Hold'  # Initialize all to 'Hold'
    signals.loc[relative_performance[sector] > buy_threshold, sector] = 'Buy'
    signals.loc[relative_performance[sector] < sell_threshold, sector] = 'Sell'

print("Fixed-threshold signals DataFrame regenerated successfully!")
print("First 5 rows of signals:")
display(signals.head())

# Re-run the backtesting simulation for the fixed-threshold strategy
# 1. Initialize an empty DataFrame called portfolio_returns
portfolio_returns = pd.DataFrame(index=signals.index, columns=['Portfolio Return'])

# Get list of sector ETFs (excluding S&P 500)
sector_etfs = [col for col in df_sector_pct_change.columns if col != "S&P 500"]
num_tradeable_sectors = len(sector_etfs)

# Align returns and signals to ensure they cover the same time period
# This ensures both DataFrames start and end on the same dates.
returns_aligned = df_sector_pct_change[sector_etfs].align(signals, join='inner', axis=0)[0]
signals_aligned = signals.align(returns_aligned, join='inner', axis=0)[0]

# 2. Initialize current_portfolio_weights_at_start_of_period
# These are the weights assumed at the start of the *first* period for which we have signals.
current_portfolio_weights_at_start_of_period = {sector: 1.0 / num_tradeable_sectors for sector in sector_etfs}

# Create a DataFrame to store historical weights for analysis (optional, but good for inspection)
historical_weights = pd.DataFrame(columns=sector_etfs, index=signals_aligned.index)

# 3. Iterate through each time step (row) in the aligned signals DataFrame
for i in range(len(signals_aligned)):
    current_date = signals_aligned.index[i]
    current_signals = signals_aligned.loc[current_date] # Signals for the current period
    current_week_returns = returns_aligned.loc[current_date] # Returns for the current period

    # `weights_for_this_period` will hold the actual weights used for the current period's return calculation.
    # Initialize all to 0.0, and then assign based on signals.
    weights_for_this_period = {sector: 0.0 for sector in sector_etfs}

    buy_sectors = [s for s in sector_etfs if current_signals[s] == 'Buy']
    sell_sectors = [s for s in sector_etfs if current_signals[s] == 'Sell']
    hold_sectors = [s for s in sector_etfs if current_signals[s] == 'Hold']

    # Determine the 'active' sectors for this period (those not getting a 'Sell' signal).
    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period[sector] = weight_per_active_sector
        # Sectors not in active_sectors_this_period (i.e., sell_sectors) implicitly retain 0.0 weight.
    else:
        # If all sectors are 'Sell', or no active signals, all weights remain 0.0.
        # Portfolio return for this week will be 0.0.
        pass # weights_for_this_period is already initialized to 0.0

    # Store the weights used for this period calculation in the historical_weights DataFrame.
    historical_weights.loc[current_date] = list(weights_for_this_period.values())

    # Calculate portfolio return for the current week using the determined weights.
    weekly_port_return = sum(weights_for_this_period[s] * current_week_returns[s] for s in sector_etfs)
    portfolio_returns.loc[current_date, 'Portfolio Return'] = weekly_port_return

    # Update `current_portfolio_weights_at_start_of_period` for the *next* iteration.
    # This reflects the portfolio composition at the end of the current period, which is the start of the next.
    current_portfolio_weights_at_start_of_period = weights_for_this_period

# 4. Calculate the cumulative returns of the portfolio_returns
# (1 + Return) cumprod() - 1 gives the cumulative return based on compounding.
portfolio_returns['Cumulative Return'] = (1 + portfolio_returns['Portfolio Return']).cumprod() - 1

print("Fixed-threshold backtesting simulation complete.")
print("First 5 rows of portfolio_returns:")
display(portfolio_returns.head())

In [ ]:
# 188.0

rolling_std_relative_performance = relative_performance.rolling(window=12).std()

multiplier = 0.5 # Adjust this multiplier as needed

dynamic_buy_thresholds = multiplier * rolling_std_relative_performance
dynamic_sell_thresholds = -multiplier * rolling_std_relative_performance

signals_dynamic = pd.DataFrame(index=relative_performance.index)

for sector in relative_performance.columns:
    signals_dynamic[sector] = 'Hold'  # Initialize all to 'Hold'
    # Apply 'Buy' signal where relative performance is above the dynamic buy threshold
    buy_condition = relative_performance[sector] > dynamic_buy_thresholds[sector]
    signals_dynamic.loc[buy_condition, sector] = 'Buy'

    # Apply 'Sell' signal where relative performance is below the dynamic sell threshold
    sell_condition = relative_performance[sector] < dynamic_sell_thresholds[sector]
    signals_dynamic.loc[sell_condition, sector] = 'Sell'

print("Dynamic signals DataFrame regenerated successfully!")
print("First 5 rows of signals_dynamic:")
display(signals_dynamic.head())

# Re-run the backtesting simulation for the dynamic-threshold strategy without costs
portfolio_returns_dynamic = pd.DataFrame(index=signals_dynamic.index, columns=['Portfolio Return'])

sector_etfs = [col for col in df_sector_pct_change.columns if col != "S&P 500"]
num_tradeable_sectors = len(sector_etfs)

returns_aligned_dynamic = df_sector_pct_change[sector_etfs].align(signals_dynamic, join='inner', axis=0)[0]
signals_aligned_dynamic = signals_dynamic.align(returns_aligned_dynamic, join='inner', axis=0)[0]

valid_indices = signals_aligned_dynamic.dropna(how='all').index
signals_aligned_dynamic = signals_aligned_dynamic.loc[valid_indices]
returns_aligned_dynamic = returns_aligned_dynamic.loc[valid_indices]
portfolio_returns_dynamic = portfolio_returns_dynamic.loc[valid_indices]

current_portfolio_weights_at_start_of_period_dynamic = {sector: 1.0 / num_tradeable_sectors for sector in sector_etfs}

historical_weights_dynamic = pd.DataFrame(columns=sector_etfs, index=signals_aligned_dynamic.index)

for i in range(len(signals_aligned_dynamic)):
    current_date = signals_aligned_dynamic.index[i]
    current_signals = signals_aligned_dynamic.loc[current_date]
    current_week_returns = returns_aligned_dynamic.loc[current_date]

    weights_for_this_period_dynamic = {sector: 0.0 for sector in sector_etfs}

    buy_sectors = [s for s in sector_etfs if current_signals[s] == 'Buy']
    sell_sectors = [s for s in sector_etfs if current_signals[s] == 'Sell']
    hold_sectors = [s for s in sector_etfs if current_signals[s] == 'Hold']

    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_dynamic[sector] = weight_per_active_sector

    historical_weights_dynamic.loc[current_date] = list(weights_for_this_period_dynamic.values())

    weekly_port_return = sum(weights_for_this_period_dynamic[s] * current_week_returns[s] for s in sector_etfs)
    portfolio_returns_dynamic.loc[current_date, 'Portfolio Return'] = weekly_port_return

    current_portfolio_weights_at_start_of_period_dynamic = weights_for_this_period_dynamic

portfolio_returns_dynamic['Cumulative Return'] = (1 + portfolio_returns_dynamic['Portfolio Return']).cumprod() - 1

print("Dynamic portfolio simulation (without costs) complete.")
print("First 5 rows of portfolio_returns_dynamic:")
display(portfolio_returns_dynamic.head())

# Re-run the backtesting simulation for the dynamic-threshold strategy with transaction costs
portfolio_returns_dynamic_with_costs = pd.DataFrame(index=signals_aligned_dynamic.index, columns=['Portfolio Return'])

transaction_cost_rate = 0.001  # 0.1% transaction cost

previous_period_weights = {sector: 1.0 / num_tradeable_sectors for sector in sector_etfs}

for i in range(len(signals_aligned_dynamic)):
    current_date = signals_aligned_dynamic.index[i]
    current_signals = signals_aligned_dynamic.loc[current_date] # Signals for the current period
    current_week_returns = returns_aligned_dynamic.loc[current_date] # Returns for the current period

    # Determine the weights_for_this_period_dynamic
    weights_for_this_period_dynamic = {sector: 0.0 for sector in sector_etfs}
    buy_sectors = [s for s in sector_etfs if current_signals[s] == 'Buy']
    hold_sectors = [s for s in sector_etfs if current_signals[s] == 'Hold']
    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_dynamic[sector] = weight_per_active_sector

    # Calculate transaction_costs for this period
    transaction_costs = 0.0
    for sector in sector_etfs:
        weight_change = abs(weights_for_this_period_dynamic[sector] - previous_period_weights[sector])
        transaction_costs += weight_change * transaction_cost_rate

    # Calculate weekly_port_return_before_costs
    weekly_port_return_before_costs = sum(weights_for_this_period_dynamic[s] * current_week_returns[s] for s in sector_etfs)

    # Deduct transaction_costs
    weekly_port_return_after_costs = weekly_port_return_before_costs - transaction_costs
    portfolio_returns_dynamic_with_costs.loc[current_date, 'Portfolio Return'] = weekly_port_return_after_costs

    # Update previous_period_weights for the next iteration
    previous_period_weights = weights_for_this_period_dynamic

portfolio_returns_dynamic_with_costs['Cumulative Return'] = (1 + portfolio_returns_dynamic_with_costs['Portfolio Return']).cumprod() - 1

print("Dynamic portfolio simulation with transaction costs complete.")
print("First 5 rows of portfolio_returns_dynamic_with_costs:")
display(portfolio_returns_dynamic_with_costs.head())

In [ ]:
# 189.0

import pandas as pd
import numpy as np

# 1. Resample df_sector to a monthly frequency, taking the last value of each month.
# Use 'ME' for month-end frequency to avoid future warnings.
df_sector_monthly = df_sector.resample('ME').last()

# 2. Calculate the monthly percentage changes of df_sector_monthly
df_sector_monthly_pct_change = df_sector_monthly.pct_change().dropna()

# 3. Compute the 3-month rolling mean of df_sector_monthly_pct_change
rolling_mean_monthly_pct_change = df_sector_monthly_pct_change.rolling(window=3).mean().dropna()

print("Monthly data re-preparation complete!")
print("First 5 rows of df_sector_monthly:")
display(df_sector_monthly.head())
print("\nFirst 5 rows of df_sector_monthly_pct_change:")
display(df_sector_monthly_pct_change.head())
print("\nFirst 5 rows of rolling_mean_monthly_pct_change:")
display(rolling_mean_monthly_pct_change.head())

# 4. Initialize an empty DataFrame called relative_performance_monthly
relative_performance_monthly = pd.DataFrame(index=rolling_mean_monthly_pct_change.index)

# 5. Iterate through each sector (excluding 'S&P 500') and calculate its relative performance
sector_etfs_monthly = [col for col in rolling_mean_monthly_pct_change.columns if col != "S&P 500"]

for sector in sector_etfs_monthly:
    relative_performance_monthly[sector] = rolling_mean_monthly_pct_change[sector] - rolling_mean_monthly_pct_change["S&P 500"]

print("\nMonthly relative performance DataFrame re-created successfully!")
print("First 5 rows of relative_performance_monthly:")
display(relative_performance_monthly.head())

# 6. Define buy_threshold_monthly and sell_threshold_monthly
buy_threshold_monthly = 0.001
sell_threshold_monthly = -0.001

# 7. Initialize an empty DataFrame called signals_monthly
signals_monthly = pd.DataFrame(index=relative_performance_monthly.index)

# 8. Generate 'Buy', 'Sell', or 'Hold' signals for each sector
for sector in relative_performance_monthly.columns:
    signals_monthly[sector] = 'Hold'  # Initialize all to 'Hold'
    signals_monthly.loc[relative_performance_monthly[sector] > buy_threshold_monthly, sector] = 'Buy'
    signals_monthly.loc[relative_performance_monthly[sector] < sell_threshold_monthly, sector] = 'Sell'

print("\nMonthly signals DataFrame re-created successfully!")
print("First 5 rows of signals_monthly:")
display(signals_monthly.head())

# 9. Backtest the monthly trading strategy:
# Initialize an empty DataFrame called portfolio_returns_monthly
portfolio_returns_monthly = pd.DataFrame(index=signals_monthly.index, columns=['Portfolio Return'])

num_tradeable_sectors_monthly = len(sector_etfs_monthly)

# Align monthly returns and signals to ensure they cover the same time period
returns_aligned_monthly = df_sector_monthly_pct_change[sector_etfs_monthly].align(signals_monthly, join='inner', axis=0)[0]
signals_aligned_monthly = signals_monthly.align(returns_aligned_monthly, join='inner', axis=0)[0]

# Initialize current_portfolio_weights_at_start_of_period_monthly
current_portfolio_weights_at_start_of_period_monthly = {sector: 1.0 / num_tradeable_sectors_monthly for sector in sector_etfs_monthly}

# Create a DataFrame to store historical weights for analysis
historical_weights_monthly = pd.DataFrame(columns=sector_etfs_monthly, index=signals_aligned_monthly.index)

# Iterate through each time step (row) in the aligned signals_aligned_monthly DataFrame
for i in range(len(signals_aligned_monthly)):
    current_date = signals_aligned_monthly.index[i]
    current_signals = signals_aligned_monthly.loc[current_date] # Signals for the current period
    current_month_returns = returns_aligned_monthly.loc[current_date] # Returns for the current period

    # `weights_for_this_period_monthly` will hold the actual weights used for the current period's return calculation.
    # Initialize all to 0.0, and then assign based on signals.
    weights_for_this_period_monthly = {sector: 0.0 for sector in sector_etfs_monthly}

    buy_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Buy']
    sell_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Sell']
    hold_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Hold']

    # Determine the 'active' sectors for this period (those not getting a 'Sell' signal).
    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_monthly[sector] = weight_per_active_sector
    else:
        # If all sectors are 'Sell', or no active signals, all weights remain 0.0.
        pass # weights_for_this_period_monthly is already initialized to 0.0

    # Store the weights used for this period calculation in the historical_weights_monthly DataFrame.
    historical_weights_monthly.loc[current_date] = list(weights_for_this_period_monthly.values())

    # Calculate portfolio return for the current month using the determined weights.
    monthly_port_return = sum(weights_for_this_period_monthly[s] * current_month_returns[s] for s in sector_etfs_monthly)
    portfolio_returns_monthly.loc[current_date, 'Portfolio Return'] = monthly_port_return

    # Update `current_portfolio_weights_at_start_of_period_monthly` for the *next* iteration.
    current_portfolio_weights_at_start_of_period_monthly = weights_for_this_period_monthly

# 10. Calculate the Cumulative Return of the portfolio_returns_monthly
portfolio_returns_monthly['Cumulative Return'] = (1 + portfolio_returns_monthly['Portfolio Return']).cumprod() - 1

print("\nMonthly portfolio simulation re-complete.")
# 11. Display the first 5 and last 5 rows of portfolio_returns_monthly
print("First 5 rows of portfolio_returns_monthly:")
display(portfolio_returns_monthly.head())
print("\nLast 5 rows of portfolio_returns_monthly:")
display(portfolio_returns_monthly.tail())

In [ ]:
# 190.0

import pandas as pd

df_sector = pd.read_csv("Sector_ETF_Data.csv", index_col="Date", parse_dates=True)

# Correct for missing values using ffill() and then bfill()
df_sector = df_sector.ffill()
df_sector = df_sector.bfill()

print("Missing values after correction:")
display(df_sector.isnull().sum())

print("DataFrame head after missing value correction:")
display(df_sector.head())

In [ ]:
# 191.0

# Filter signals for December 2025
dec_2025_signals = signals_monthly.loc['2025-12-01':'2025-12-31']

# Extract the signals for that specific month as a Series
dec_2025_signals_series = dec_2025_signals.iloc[0]

buy_sectors = dec_2025_signals_series[dec_2025_signals_series == 'Buy'].index.tolist()
sell_sectors = dec_2025_signals_series[dec_2025_signals_series == 'Sell'].index.tolist()
hold_sectors = dec_2025_signals_series[dec_2025_signals_series == 'Hold'].index.tolist()

# Create a DataFrame for the recommendations
recommendations_data = {
    'Signal Type': ['Buy', 'Sell', 'Hold'],
    'Recommended Sectors (Dec 2025)': [
        ', '.join(buy_sectors) if buy_sectors else 'None',
        ', '.join(sell_sectors) if sell_sectors else 'None',
        ', '.join(hold_sectors) if hold_sectors else 'None'
    ]
}
recommendations_df = pd.DataFrame(recommendations_data)

print("Sector Recommendations for December 2025:")
display(recommendations_df)

print("\nNote: The current models in this notebook are designed to predict macroeconomic indicators (GDP, Inflation, Unemployment Rate, Interest Rate) and economic phases for sector rotation strategy. They do not directly output a predicted S&P 500 percentage return for future months.")

In [ ]:
# 192.0

rf = 0.01 # Re-defining risk-free rate, ensuring it's available in this scope

# --- Recalculate metrics for Monthly Strategy ---
# The df_sector_monthly, df_sector_monthly_pct_change, rolling_mean_monthly_pct_change,
# relative_performance_monthly, signals_monthly, and portfolio_returns_monthly are already
# correctly calculated and updated by the previous code block (cell `cb4626b1`).

num_months_monthly = len(portfolio_returns_monthly)
num_years_monthly = num_months_monthly / 12.0

portfolio_total_cumulative_return_monthly = portfolio_returns_monthly['Cumulative Return'].iloc[-1]
portfolio_annualized_return_monthly = (1 + portfolio_total_cumulative_return_monthly)**(1/num_years_monthly) - 1
portfolio_annualized_volatility_monthly = portfolio_returns_monthly['Portfolio Return'].std() * np.sqrt(12)
portfolio_sharpe_ratio_monthly = (portfolio_annualized_return_monthly - rf) / portfolio_annualized_volatility_monthly

# --- Recalculate metrics for Fixed Strategy --- (using already calculated portfolio_returns)
# The portfolio_returns dataframe is already correctly calculated and updated by cell `b7efcd7e`.
num_weeks = len(portfolio_returns)
num_years = num_weeks / 52.0

portfolio_total_cumulative_return = portfolio_returns['Cumulative Return'].iloc[-1]
portfolio_annualized_return = (1 + portfolio_total_cumulative_return)**(1/num_years) - 1
portfolio_annualized_volatility = portfolio_returns['Portfolio Return'].std() * np.sqrt(52)
portfolio_sharpe_ratio = (portfolio_annualized_return - rf) / portfolio_annualized_volatility

# Recalculate S&P 500 metrics for consistency with the fixed strategy's time frame
# df_sector_pct_change should be updated from `37b9da73`
s_p_500_returns_aligned = df_sector_pct_change["S&P 500"].align(portfolio_returns, join='inner', axis=0)[0]
s_p_500_cumulative_return = (1 + s_p_500_returns_aligned).cumprod() - 1
s_p_500_total_cumulative_return = s_p_500_cumulative_return.iloc[-1]
s_p_500_annualized_return = (1 + s_p_500_total_cumulative_return)**(1/num_years) - 1
s_p_500_annualized_volatility = s_p_500_returns_aligned.std() * np.sqrt(52)
s_p_500_sharpe_ratio = (s_p_500_annualized_return - rf) / s_p_500_annualized_volatility

# --- Recalculate metrics for Dynamic Strategy --- (using already calculated portfolio_returns_dynamic)
# The portfolio_returns_dynamic dataframe is already correctly calculated and updated by cell `fb0015c0`.
num_weeks_dynamic = len(portfolio_returns_dynamic)
num_years_dynamic = num_weeks_dynamic / 52.0

portfolio_total_cumulative_return_dynamic = portfolio_returns_dynamic['Cumulative Return'].iloc[-1]
portfolio_annualized_return_dynamic = (1 + portfolio_total_cumulative_return_dynamic)**(1/num_years_dynamic) - 1
portfolio_annualized_volatility_dynamic = portfolio_returns_dynamic['Portfolio Return'].std() * np.sqrt(52)
portfolio_sharpe_ratio_dynamic = (portfolio_annualized_return_dynamic - rf) / portfolio_annualized_volatility_dynamic

# --- Recalculate metrics for Dynamic Strategy with Costs --- (using already calculated portfolio_returns_dynamic_with_costs)
# The portfolio_returns_dynamic_with_costs dataframe is already correctly calculated and updated by cell `fb0015c0`.
num_weeks_dynamic_costs = len(portfolio_returns_dynamic_with_costs)
num_years_dynamic_costs = num_weeks_dynamic_costs / 52.0

portfolio_total_cumulative_return_dynamic_costs = portfolio_returns_dynamic_with_costs['Cumulative Return'].iloc[-1]
portfolio_annualized_return_dynamic_costs = (1 + portfolio_total_cumulative_return_dynamic_costs)**(1/num_years_dynamic_costs) - 1
portfolio_annualized_volatility_dynamic_costs = portfolio_returns_dynamic_with_costs['Portfolio Return'].std() * np.sqrt(52)
portfolio_sharpe_ratio_dynamic_costs = (portfolio_annualized_return_dynamic_costs - rf) / portfolio_annualized_volatility_dynamic_costs

# Print Summary Comparison
print("\n--- Strategy Performance Comparison (Monthly vs. Dynamic Costs vs. Dynamic vs. Fixed vs. S&P 500) ---")
print(f"Metric                        | Monthly         | Dynamic Costs   | Dynamic         | Fixed Threshold | S&P 500       ")
print(f"------------------------------------------------------------------------------------------------------------------")
print(f"Total Cumulative Return       | {portfolio_total_cumulative_return_monthly:.4f}        | {portfolio_total_cumulative_return_dynamic_costs:.4f}        | {portfolio_total_cumulative_return_dynamic:.4f}         | {portfolio_total_cumulative_return:.4f}        | {s_p_500_total_cumulative_return:.4f}       ")
print(f"Annualized Return             | {portfolio_annualized_return_monthly:.4f}        | {portfolio_annualized_return_dynamic_costs:.4f}        | {portfolio_annualized_return_dynamic:.4f}         | {portfolio_annualized_return:.4f}        | {s_p_500_annualized_return:.4f}       ")
print(f"Annualized Volatility         | {portfolio_annualized_volatility_monthly:.4f}        | {portfolio_annualized_volatility_dynamic_costs:.4f}        | {portfolio_annualized_volatility_dynamic:.4f}         | {portfolio_annualized_volatility:.4f}        | {s_p_500_annualized_volatility:.4f}       ")
print(f"Sharpe Ratio                  | {portfolio_sharpe_ratio_monthly:.4f}        | {portfolio_sharpe_ratio_dynamic_costs:.4f}        | {portfolio_sharpe_ratio_dynamic:.4f}         | {portfolio_sharpe_ratio:.4f}        | {s_p_500_sharpe_ratio:.4f}       ")
print("------------------------------------------------------------------------------------------------------------------")

In [ ]:
# 193.0

import matplotlib.pyplot as plt

plt.figure(figsize=(14, 7))
plt.plot(portfolio_returns_monthly.index, portfolio_returns_monthly['Cumulative Return'], label='Monthly Strategy Cumulative Return', color='orange')
plt.plot(portfolio_returns.index, portfolio_returns['Cumulative Return'], label='Fixed Strategy Cumulative Return', color='blue')
plt.plot(portfolio_returns_dynamic.index, portfolio_returns_dynamic['Cumulative Return'], label='Dynamic Strategy Cumulative Return (no costs)', color='green')
plt.plot(portfolio_returns_dynamic_with_costs.index, portfolio_returns_dynamic_with_costs['Cumulative Return'], label='Dynamic Strategy Cumulative Return (with costs)', color='purple')
plt.plot(s_p_500_cumulative_return.index, s_p_500_cumulative_return, label='S&P 500 Cumulative Return', color='red')
plt.title('Cumulative Returns: All Strategies vs. S&P 500 (Including XLRE)')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend()
plt.show()

In [ ]:
# 194.0

import pandas as pd
import yfinance as yf

tickers = ["XLK", "XLC", "XLV", "XLF", "XLI", "XLB", "XLU", "XLE", "XLY", "XLP", "SPY", "XLRE"]

# Mapping from tickers to desired sector names
ticker_to_sector = {
    "XLK": "Technology",
    "XLC": "Communications",
    "XLV": "Healthcare",
    "XLF": "Financial",
    "XLI": "Industrial",
    "XLB": "Material",
    "XLU": "Utilities",
    "XLE": "Energy",
    "XLY": "CDiscretionary",
    "XLP": "CStaples",
    "SPY": "S&P 500",
    "XLRE": "Real Estate"
}

# Set the date range
start_date = "2010-01-01"
end_date = "2025-12-30"

# Initialize an empty dictionary to store ETF data
sector_data = {}

# Download historical data for each ETF, explicitly setting auto_adjust=True.
for ticker in tickers:
    data = yf.download(ticker, start=start_date, end=end_date, interval="1wk", auto_adjust=True, progress=False)
    if not data.empty:
        sector_data[ticker] = data["Close"]

# If valid data exists, combine it into a DataFrame and flatten the headers.
if sector_data:
    # Concatenate the Series objects; using keys creates a MultiIndex header.
    df_sector = pd.concat(sector_data.values(), axis=1, keys=sector_data.keys())

    # Flatten the MultiIndex columns by extracting the first level (tickers) and mapping to sector names.
    df_sector.columns = [ticker_to_sector.get(ticker, ticker) for ticker in df_sector.columns.get_level_values(0)]

    # Save the DataFrame to CSV
    df_sector.to_csv("Sector_ETF_Data.csv")
    print("Sector ETF data re-saved successfully!")
else:
    print("No valid data retrieved for any tickers.")

# Reload the 'Sector_ETF_Data.csv' into df_sector
df_sector = pd.read_csv("Sector_ETF_Data.csv", index_col="Date", parse_dates=True)

# Correct for missing values using ffill() and then bfill()
df_sector = df_sector.ffill()
df_sector = df_sector.bfill()

print("\nMissing values after correction:")
display(df_sector.isnull().sum())

print("\nDataFrame head after missing value correction:")
display(df_sector.head())

In [ ]:
# 195.0

import pandas as pd
import numpy as np

# 1. Resample df_sector to a monthly frequency, taking the last value of each month.
# Use 'ME' for month-end frequency to avoid future warnings.
df_sector_monthly = df_sector.resample('ME').last()

# 2. Calculate the monthly percentage changes of df_sector_monthly
df_sector_monthly_pct_change = df_sector_monthly.pct_change().dropna()

# 3. Compute the 3-month rolling mean of df_sector_monthly_pct_change
rolling_mean_monthly_pct_change = df_sector_monthly_pct_change.rolling(window=3).mean().dropna()

print("Monthly data re-preparation complete!")
print("First 5 rows of df_sector_monthly:")
display(df_sector_monthly.head())
print("\nFirst 5 rows of df_sector_monthly_pct_change:")
display(df_sector_monthly_pct_change.head())
print("\nFirst 5 rows of rolling_mean_monthly_pct_change:")
display(rolling_mean_monthly_pct_change.head())

# 4. Initialize an empty DataFrame called relative_performance_monthly
relative_performance_monthly = pd.DataFrame(index=rolling_mean_monthly_pct_change.index)

# 5. Iterate through each sector (excluding 'S&P 500') and calculate its relative performance
sector_etfs_monthly = [col for col in rolling_mean_monthly_pct_change.columns if col != "S&P 500"]

for sector in sector_etfs_monthly:
    relative_performance_monthly[sector] = rolling_mean_monthly_pct_change[sector] - rolling_mean_monthly_pct_change["S&P 500"]

print("\nMonthly relative performance DataFrame re-created successfully!")
print("First 5 rows of relative_performance_monthly:")
display(relative_performance_monthly.head())

# 6. Define buy_threshold_monthly and sell_threshold_monthly
buy_threshold_monthly = 0.001
sell_threshold_monthly = -0.001

# 7. Initialize an empty DataFrame called signals_monthly
signals_monthly = pd.DataFrame(index=relative_performance_monthly.index)

# 8. Generate 'Buy', 'Sell', or 'Hold' signals for each sector
for sector in relative_performance_monthly.columns:
    signals_monthly[sector] = 'Hold'  # Initialize all to 'Hold'
    signals_monthly.loc[relative_performance_monthly[sector] > buy_threshold_monthly, sector] = 'Buy'
    signals_monthly.loc[relative_performance_monthly[sector] < sell_threshold_monthly, sector] = 'Sell'

print("\nMonthly signals DataFrame re-created successfully!")
print("First 5 rows of signals_monthly:")
display(signals_monthly.head())

# 9. Backtest the monthly trading strategy:
# Initialize an empty DataFrame called portfolio_returns_monthly
portfolio_returns_monthly = pd.DataFrame(index=signals_monthly.index, columns=['Portfolio Return'])

num_tradeable_sectors_monthly = len(sector_etfs_monthly)

# Align monthly returns and signals to ensure they cover the same time period
returns_aligned_monthly = df_sector_monthly_pct_change[sector_etfs_monthly].align(signals_monthly, join='inner', axis=0)[0]
signals_aligned_monthly = signals_monthly.align(returns_aligned_monthly, join='inner', axis=0)[0]

# Initialize current_portfolio_weights_at_start_of_period_monthly
current_portfolio_weights_at_start_of_period_monthly = {sector: 1.0 / num_tradeable_sectors_monthly for sector in sector_etfs_monthly}

# Create a DataFrame to store historical weights for analysis
historical_weights_monthly = pd.DataFrame(columns=sector_etfs_monthly, index=signals_aligned_monthly.index)

# Iterate through each time step (row) in the aligned signals_aligned_monthly DataFrame
for i in range(len(signals_aligned_monthly)):
    current_date = signals_aligned_monthly.index[i]
    current_signals = signals_aligned_monthly.loc[current_date] # Signals for the current period
    current_month_returns = returns_aligned_monthly.loc[current_date] # Returns for the current period

    # `weights_for_this_period_monthly` will hold the actual weights used for the current period's return calculation.
    # Initialize all to 0.0, and then assign based on signals.
    weights_for_this_period_monthly = {sector: 0.0 for sector in sector_etfs_monthly}

    buy_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Buy']
    sell_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Sell']
    hold_sectors = [s for s in sector_etfs_monthly if current_signals[s] == 'Hold']

    # Determine the 'active' sectors for this period (those not getting a 'Sell' signal).
    active_sectors_this_period = buy_sectors + hold_sectors

    if len(active_sectors_this_period) > 0:
        weight_per_active_sector = 1.0 / len(active_sectors_this_period)
        for sector in active_sectors_this_period:
            weights_for_this_period_monthly[sector] = weight_per_active_sector
    else:
        # If all sectors are 'Sell', or no active signals, all weights remain 0.0.
        pass # weights_for_this_period_monthly is already initialized to 0.0

    # Store the weights used for this period calculation in the historical_weights_monthly DataFrame.
    historical_weights_monthly.loc[current_date] = list(weights_for_this_period_monthly.values())

    # Calculate portfolio return for the current month using the determined weights.
    monthly_port_return = sum(weights_for_this_period_monthly[s] * current_month_returns[s] for s in sector_etfs_monthly)
    portfolio_returns_monthly.loc[current_date, 'Portfolio Return'] = monthly_port_return

    # Update `current_portfolio_weights_at_start_of_period_monthly` for the *next* iteration.
    current_portfolio_weights_at_start_of_period_monthly = weights_for_this_period_monthly

# 10. Calculate the Cumulative Return of the portfolio_returns_monthly
portfolio_returns_monthly['Cumulative Return'] = (1 + portfolio_returns_monthly['Portfolio Return']).cumprod() - 1

print("\nMonthly portfolio simulation re-complete.")
# 11. Display the first 5 and last 5 rows of portfolio_returns_monthly
print("First 5 rows of portfolio_returns_monthly:")
display(portfolio_returns_monthly.head())
print("\nLast 5 rows of portfolio_returns_monthly:")
display(portfolio_returns_monthly.tail())

In [ ]:
# 196.0

import pandas as pd

df_sector = pd.read_csv("Sector_ETF_Data.csv", index_col="Date", parse_dates=True)

# Correct for missing values using ffill() and then bfill()
df_sector = df_sector.ffill()
df_sector = df_sector.bfill()

print("Missing values after correction:")
display(df_sector.isnull().sum())

print("DataFrame head after missing value correction:")
display(df_sector.head())

In [ ]:
# 197.0

import pandas as pd
import numpy as np

# 1. Resample df_sector to a monthly frequency, taking the last value of each month.
# Use 'ME' for month-end frequency to avoid future warnings.
df_sector_monthly = df_sector.resample('ME').last()

# 2. Calculate the monthly percentage changes of df_sector_monthly
df_sector_monthly_pct_change = df_sector_monthly.pct_change().dropna()

# 3. Compute the 3-month rolling mean of df_sector_monthly_pct_change
rolling_mean_monthly_pct_change = df_sector_monthly_pct_change.rolling(window=3).mean().dropna()

# 4. Initialize an empty DataFrame called relative_performance_monthly
relative_performance_monthly = pd.DataFrame(index=rolling_mean_monthly_pct_change.index)

# 5. Iterate through each sector (excluding 'S&P 500') and calculate its relative performance
sector_etfs_monthly = [col for col in rolling_mean_monthly_pct_change.columns if col != "S&P 500"]

for sector in sector_etfs_monthly:
    relative_performance_monthly[sector] = rolling_mean_monthly_pct_change[sector] - rolling_mean_monthly_pct_change["S&P 500"]

# 6. Define buy_threshold_monthly and sell_threshold_monthly
buy_threshold_monthly = 0.001
sell_threshold_monthly = -0.001

# 7. Initialize an empty DataFrame called signals_monthly
signals_monthly = pd.DataFrame(index=relative_performance_monthly.index)

# 8. Generate 'Buy', 'Sell', or 'Hold' signals for each sector
for sector in relative_performance_monthly.columns:
    signals_monthly[sector] = 'Hold'  # Initialize all to 'Hold'
    signals_monthly.loc[relative_performance_monthly[sector] > buy_threshold_monthly, sector] = 'Buy'
    signals_monthly.loc[relative_performance_monthly[sector] < sell_threshold_monthly, sector] = 'Sell'

# Filter signals for January 2026
jan_2026_signals = signals_monthly.loc['2026-01-01':'2026-01-31']

buy_sectors = jan_2026_signals[jan_2026_signals == 'Buy'].columns.tolist()
sell_sectors = jan_2026_signals[jan_2026_signals == 'Sell'].columns.tolist()
hold_sectors = jan_2026_signals[jan_2026_signals == 'Hold'].columns.tolist()

# Create a DataFrame for the recommendations
recommendations_data = {
    'Signal Type': ['Buy', 'Sell', 'Hold'],
    'Recommended Sectors (Jan 2026)': [
        ', '.join(buy_sectors) if buy_sectors else 'None',
        ', '.join(sell_sectors) if sell_sectors else 'None',
        ', '.join(hold_sectors) if hold_sectors else 'None'
    ]
}
recommendations_df = pd.DataFrame(recommendations_data)

print("Sector Recommendations for January 2026:")
display(recommendations_df)

print("\nNote: The current models in this notebook are designed to predict macroeconomic indicators (GDP, Inflation, Unemployment Rate, Interest Rate) and economic phases for sector rotation strategy. They do not directly output a predicted S&P 500 percentage return for future months.")